In this notebook I will create a training table equivalent to ml_model_run_details

# Define Library

In [1]:
# %% [markdown]
# # Jupyter Notebook Loading Header
#
# This is a custom loading header for Jupyter Notebooks in Visual Studio Code.
# It includes common imports and settings to get you started quickly.
# %% [markdown]
## Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.cloud import bigquery
from google.cloud import storage
import os
import tempfile
import time
from datetime import datetime
import uuid
import joblib
import uuid

import gcsfs
import duckdb as dd
import pickle
import joblib
from typing import Union
import io
path = r'C:\Users\Dwaipayan\AppData\Roaming\gcloud\legacy_credentials\dchakroborti@tonikbank.com\adc.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = path
client = bigquery.Client(project='prj-prod-dataplatform')
os.environ["GOOGLE_CLOUD_PROJECT"] = "prj-prod-dataplatform"

# %% [markdown]
## Configure Settings
# Set options or configurations as needed
pd.set_option('display.max_columns', None)
pd.set_option("Display.max_rows", 100)

### Function

#### expand_calc_features

In [2]:
import pandas as pd
import json

def expand_calc_features(df):
    """
    Expand the calcFeatures JSON column into separate columns and return the complete DataFrame.

    Parameters:
    df (pd.DataFrame): Input DataFrame with calcFeatures column containing JSON data

    Returns:
    pd.DataFrame: Expanded DataFrame with all original columns plus JSON features as separate columns
    """

    # Make a copy to avoid modifying the original DataFrame
    df_expanded = df.copy()

    # Parse the calcFeatures JSON column
    calc_features_list = []

    for idx, calc_features_str in enumerate(df['calcFeatures']):
        try:
            # Parse the JSON string
            features_dict = json.loads(calc_features_str.replace("'", '"'))  # Replace single quotes with double quotes for valid JSON
            calc_features_list.append(features_dict)
        except (json.JSONDecodeError, AttributeError) as e:
            # If parsing fails, create an empty dict and print warning
            print(f"Warning: Could not parse calcFeatures at index {idx}: {e}")
            calc_features_list.append({})

    # Create DataFrame from the parsed JSON data
    calc_features_df = pd.DataFrame(calc_features_list)

    # Add prefix to JSON-derived columns to avoid conflicts
    calc_features_df = calc_features_df.add_prefix('calc_')

    # Reset index to ensure proper alignment
    df_expanded = df_expanded.reset_index(drop=True)
    calc_features_df = calc_features_df.reset_index(drop=True)

    # Combine original DataFrame with expanded calcFeatures
    result_df = pd.concat([df_expanded, calc_features_df], axis=1)

    return result_df


#### expand_calc_features_robust

In [3]:
import pandas as pd
import json

def expand_calc_features_robust(df):
    """
    Expand the calcFeatures JSON column into separate columns with better error handling.

    Parameters:
    df (pd.DataFrame): Input DataFrame with calcFeatures column containing JSON data

    Returns:
    pd.DataFrame: Expanded DataFrame with all original columns plus JSON features as separate columns
    """

    # Make a copy to avoid modifying the original DataFrame
    df_expanded = df.copy()

    # Parse the calcFeatures JSON column
    calc_features_data = []

    for idx, row in df.iterrows():
        calc_features_str = row['calcFeatures']

        if pd.isna(calc_features_str) or calc_features_str == '':
            calc_features_data.append({})
            continue

        try:
            # Clean the string and parse JSON
            cleaned_str = calc_features_str.replace("'", '"').replace('None', 'null').replace('True', 'true').replace('False', 'false')
            features_dict = json.loads(cleaned_str)
            calc_features_data.append(features_dict)
        except Exception as e:
            print(f"Warning: Could not parse calcFeatures at index {idx}: {e}")
            print(f"Problematic string: {calc_features_str[:100]}...")  # Print first 100 chars
            calc_features_data.append({})

    # Create DataFrame from the parsed JSON data
    calc_features_df = pd.DataFrame(calc_features_data)

    # Add prefix to JSON-derived columns to avoid conflicts with existing columns
    calc_features_df = calc_features_df.add_prefix('feat_')

    # Combine DataFrames
    result_df = pd.concat([df_expanded, calc_features_df], axis=1)

    print(f"Original DataFrame shape: {df.shape}")
    print(f"Expanded DataFrame shape: {result_df.shape}")
    print(f"Added {len(calc_features_df.columns)} new columns from calcFeatures")

    return result_df

#### transform_data

In [4]:
# import pandas as pd
# import json
# import uuid
# from datetime import datetime
# from typing import List

# def transform_data(d1: pd.DataFrame, feature_column: List[str], a='demo_score', modelDisplayName = 'Cash_beta_trench1_Demo_backscore', subscription_name = 'sil_march 25 models'):
#     # Read the input CSV file
#     df = d1.copy()
    
#     # Create the output DataFrame with the required structure
#     output_data = []
    
#     for _, row in df.iterrows():
#         # Create the calcFeature JSON with all the feature columns
#         feature_columns = feature_column
        
#         calc_feature = {}
#         for col in feature_columns:
#             if col in row and pd.notna(row[col]):
#                 # Convert Timestamp objects to string
#                 if isinstance(row[col], pd.Timestamp):
#                     calc_feature[col] = row[col].isoformat()
#                 else:
#                     calc_feature[col] = row[col]
        
       
#         # Get current timestamp
#         current_time = datetime.now().isoformat()
        
#         # Create the output row
#         output_row = {
#             "customerId": row['customer_id'],
#             "digitalLoanAccountId": row['digitalLoanAccountId'],
#             "crifApplicationId": str(uuid.uuid4()),  # Generate random UUID
#             "prediction": row.get(a, 0),
#             "start_time": current_time,
#             "end_time": current_time,
#             "modelDisplayName":modelDisplayName,
#             "modelVersionId":"v1",
#             "subscription_name": subscription_name,
#             "message_id": str(uuid.uuid4()),  # Generate random UUID
#             "publish_time": current_time,
#             "attributes": "{}",  # Empty JSON object
#             "calcFeature": json.dumps(calc_feature, default=str)  # Use default=str to handle non-serializable objects
            
#         }
        
#         output_data.append(output_row)
    
#     # Create DataFrame from the output data
#     output_df = pd.DataFrame(output_data)
    
#     return output_df

# # Example usage:
# # transformeddata = 'cash_beta_trench1_applied_loans_backscored_20241001_20250831'
# # transform_data(f'{LOCALPATH}/{transformeddata}.csv')

In [5]:
import pandas as pd
import json
import uuid
from datetime import datetime
from typing import List

def transform_data(
    d1: pd.DataFrame, 
    feature_column: List[str], 
    a: str = 'demo_score', 
    modelDisplayName: str = 'Cash_beta_trench1_Demo_backscore', 
    tc: str = "", 
    subscription_name: str = 'sil_march 25 models'
) -> pd.DataFrame:
    """
    Transforms input data into a structured format suitable for model scoring output.

    Parameters:
    - d1 (pd.DataFrame): Input DataFrame containing raw data.
    - feature_column (List[str]): List of column names to include in the 'calcFeature' JSON.
    - a (str): Column name containing the prediction score. Default is 'demo_score'.
    - modelDisplayName (str): Name of the model used for scoring.
    - tc (str): Trench category (optional).
    - do (str): Device operating system. Default is 'android'.
    - subscription_name (str): Name of the subscription or model group.

    Returns:
    - pd.DataFrame: Transformed DataFrame with structured output.
    """

    # Make a copy of the input DataFrame to avoid modifying the original
    df = d1.copy()
    
    # Initialize an empty list to store transformed rows
    output_data = []
    
    # Iterate over each row in the DataFrame
    for _, row in df.iterrows():
        # Initialize dictionary to hold feature values
        calc_feature = {}
        
        # Loop through each feature column and extract its value from the row
        for col in feature_column:
            if col in row and pd.notna(row[col]):
                # Convert datetime values to ISO format strings
                if isinstance(row[col], pd.Timestamp):
                    calc_feature[col] = row[col].isoformat()
                else:
                    calc_feature[col] = row[col]
        
        # Get the current timestamp for start_time, end_time, and publish_time
        current_time = datetime.now().isoformat()
        
        # Construct the output row dictionary with required fields
        output_row = {
            "customerId": row['customer_id'],  # Unique customer identifier
            "digitalLoanAccountId": row['digitalLoanAccountId'],  # Loan account ID
            "crifApplicationId": str(uuid.uuid4()),  # Random UUID for application ID
            "prediction": row.get(a, 0),  # Prediction score from specified column
            "start_time": current_time,  # Timestamp when processing starts
            "end_time": current_time,    # Timestamp when processing ends
            "modelDisplayName": modelDisplayName,  # Name of the model used
            "modelVersionId": "v1",  # Static model version
            "calcFeature": json.dumps(calc_feature, default=str),  # Features as JSON string
            "subscription_name": subscription_name,  # Subscription name
            "message_id": str(uuid.uuid4()),  # Random UUID for message ID
            "publish_time": current_time,  # Timestamp when message is published
            "attributes": "{}",  # Placeholder for additional attributes
            "trenchCategory": tc,  # Optional trench category
            "deviceOs": row['osType'],  # Device operating system
        }
        
        # Append the transformed row to the output list
        output_data.append(output_row)
    
    # Convert the list of dictionaries to a DataFrame
    output_df = pd.DataFrame(output_data)
    
    # Return the transformed DataFrame
    return output_df


# transform_datav2

In [6]:
import pandas as pd
import json
import uuid
from datetime import datetime
from typing import List

def transform_datav2(
    d1: pd.DataFrame, 
    feature_column: List[str], 
    a: str = 'demo_score', 
    modelDisplayName: str = 'Cash_beta_trench1_Demo_backscore', 
    tc: str = "", 
    subscription_name: str = 'sil_march 25 models'
) -> pd.DataFrame:
    """
    Transforms input data into a structured format suitable for model scoring output.

    Parameters:
    - d1 (pd.DataFrame): Input DataFrame containing raw data.
    - feature_column (List[str]): List of column names to include in the 'calcFeature' JSON.
    - a (str): Column name containing the prediction score. Default is 'demo_score'.
    - modelDisplayName (str): Name of the model used for scoring.
    - tc (str): Trench category (optional).
    - do (str): Device operating system. Default is 'android'.
    - subscription_name (str): Name of the subscription or model group.

    Returns:
    - pd.DataFrame: Transformed DataFrame with structured output.
    """

    # Make a copy of the input DataFrame to avoid modifying the original
    df = d1.copy()
    
    # Initialize an empty list to store transformed rows
    output_data = []
    
    # Iterate over each row in the DataFrame
    for _, row in df.iterrows():
        # Initialize dictionary to hold feature values
        calc_feature = {}
        
        # Loop through each feature column and extract its value from the row
        for col in feature_column:
            if col in row and pd.notna(row[col]):
                # Convert datetime values to ISO format strings
                if isinstance(row[col], pd.Timestamp):
                    calc_feature[col] = row[col].isoformat()
                else:
                    calc_feature[col] = row[col]
        
        # Get the current timestamp for start_time, end_time, and publish_time
        current_time = datetime.now().isoformat()
        
        # Construct the output row dictionary with required fields
        output_row = {
            "customerId": row['customer_id'],  # Unique customer identifier
            "digitalLoanAccountId": row['digitalLoanAccountId'],  # Loan account ID
            "crifApplicationId": str(uuid.uuid4()),  # Random UUID for application ID
            "prediction": row.get(a, 0),  # Prediction score from specified column
            "start_time": current_time,  # Timestamp when processing starts
            "end_time": current_time,    # Timestamp when processing ends
            "modelDisplayName": modelDisplayName,  # Name of the model used
            "modelVersionId": "v2",  # Static model version
            "calcFeature": json.dumps(calc_feature, default=str),  # Features as JSON string
            "subscription_name": subscription_name,  # Subscription name
            "message_id": str(uuid.uuid4()),  # Random UUID for message ID
            "publish_time": current_time,  # Timestamp when message is published
            "attributes": "{}",  # Placeholder for additional attributes
            "trenchCategory": tc,  # Optional trench category
            "deviceOs": row['osType'],  # Device operating system
        }
        
        # Append the transformed row to the output list
        output_data.append(output_row)
    
    # Convert the list of dictionaries to a DataFrame
    output_df = pd.DataFrame(output_data)
    
    # Return the transformed DataFrame
    return output_df


#### PSI Functions new

In [7]:
## Updated on 27-10-2025 - Modified for Training Period Baseline
import pandas as pd
import numpy as np
from typing import List, Dict, Tuple
import warnings
warnings.filterwarnings('ignore')

def identify_feature_types(df: pd.DataFrame, feature_list: List[str]) -> Dict[str, List[str]]:
    """
    Identify categorical and numerical features from the feature list.

    Parameters:
    -----------
    df : pd.DataFrame
        Input dataframe
    feature_list : List[str]
        List of features to classify

    Returns:
    --------
    Dict with 'categorical' and 'numerical' keys containing respective feature lists
    """
    categorical_features = []
    numerical_features = []

    for feature in feature_list:
        if feature not in df.columns:
            print(f"Warning: Feature '{feature}' not found in dataframe")
            continue

        # Check if feature is numeric
        if pd.api.types.is_numeric_dtype(df[feature]):
            # If unique values are less than 15 and all integers, treat as categorical
            unique_vals = df[feature].nunique()
            if unique_vals < 15 and df[feature].dropna().apply(lambda x: x == int(x) if isinstance(x, (int, float)) else False).all():
                categorical_features.append(feature)
            else:
                numerical_features.append(feature)
        else:
            categorical_features.append(feature)

    return {
        'categorical': categorical_features,
        'numerical': numerical_features
    }


def create_bins_for_features(df: pd.DataFrame,
                             numerical_features: List[str],
                             categorical_features: List[str],
                             train_period_df: pd.DataFrame) -> Dict:
    """
    Create bins for numerical features (deciles with fallback) and categorical features (top 6 + others)
    based on the entire training period data.

    Parameters:
    -----------
    df : pd.DataFrame
        Full input dataframe
    numerical_features : List[str]
        List of numerical features
    categorical_features : List[str]
        List of categorical features
    train_period_df : pd.DataFrame
        Training period dataframe (June 2024 to March 2025)

    Returns:
    --------
    Dictionary containing binning information for each feature
    """
    binning_info = {}

    # Create bins for numerical features with fallback strategy
    for feature in numerical_features:
        valid_data = train_period_df[feature].dropna()

        if len(valid_data) == 0:
            binning_info[feature] = {'type': 'numerical', 'bins': None, 'bin_ranges': {}}
            continue

        bins = None
        bin_count = None

        # Try 10 bins (deciles)
        try:
            test_bins = np.percentile(valid_data, np.arange(0, 101, 10))
            test_bins = np.unique(test_bins)
            if len(test_bins) >= 11:  # 11 edges = 10 bins
                bins = test_bins
                bin_count = 10
        except Exception as e:
            pass

        # If 10 bins not possible, try 5 bins
        if bins is None:
            try:
                test_bins = np.percentile(valid_data, np.arange(0, 101, 20))
                test_bins = np.unique(test_bins)
                if len(test_bins) >= 6:  # 6 edges = 5 bins
                    bins = test_bins
                    bin_count = 5
            except Exception as e:
                pass

        # If 5 bins not possible, try 3 bins
        if bins is None:
            try:
                test_bins = np.percentile(valid_data, [0, 33.33, 66.67, 100])
                test_bins = np.unique(test_bins)
                if len(test_bins) >= 4:  # 4 edges = 3 bins
                    bins = test_bins
                    bin_count = 3
            except Exception as e:
                pass

        # If still no bins possible, use equal distance bins of 5
        if bins is None:
            print(f"Warning: Feature '{feature}' has insufficient variance - cannot create standard bins")
            print(f"Feature '{feature}': Using equal distance bins of 5")

            min_val = valid_data.min()
            max_val = valid_data.max()

            # Create 5 equal distance bins
            bins = np.linspace(min_val, max_val, 6)  # 6 edges = 5 bins
            bins = np.unique(bins)
            bin_count = len(bins) - 1

            # If all values are the same, add slight buffer
            if bin_count == 1:
                bins = np.array([min_val - 0.1, min_val, min_val + 0.1])
                bin_count = 2
                print(f"Feature '{feature}': Constant value ({min_val}). Created 2 equal distance bins with buffer")

        # Add infinity edges to capture all values
        bins = bins.copy()
        bins[0] = -np.inf
        bins[-1] = np.inf

        print(f"Feature '{feature}': Created {bin_count} bins")

        # Create bin ranges dictionary
        bin_ranges = {}
        for i in range(len(bins)-1):
            bin_name = f"Bin_{i+1}"
            bin_ranges[bin_name] = {
                'min': bins[i],
                'max': bins[i+1],
                'range_str': f"[{bins[i]:.2f}, {bins[i+1]:.2f}]" if not np.isinf(bins[i]) and not np.isinf(bins[i+1]) else f"({bins[i]}, {bins[i+1]})"
            }

        binning_info[feature] = {
            'type': 'numerical',
            'bins': bins,
            'bin_ranges': bin_ranges,
            'bin_count': bin_count
        }

    # Create bins for categorical features (top 6 + others) using training period
    for feature in categorical_features:
        value_counts = train_period_df[feature].value_counts()
        unique_categories = value_counts.index.tolist()
        print(f"Unique categories: {unique_categories}")

        if len(unique_categories) <= 6:
            # Treat each category as a separate bin
            top_categories = unique_categories
        else:
            # Use top 6 categories only
            top_categories = value_counts.nlargest(6).index.tolist()

        print(f"Top categories for feature '{feature}': {top_categories}")

        binning_info[feature] = {
                'type': 'categorical',
                'top_categories': top_categories,
                'bin_ranges': {}  # No ranges for categorical
            }

    return binning_info


def apply_binning(df: pd.DataFrame,
                  feature: str,
                  binning_info: Dict) -> pd.Series:
    """
    Apply binning to a feature based on binning information.

    Parameters:
    -----------
    df : pd.DataFrame
        Input dataframe
    feature : str
        Feature name
    binning_info : Dict
        Binning information for the feature

    Returns:
    --------
    pd.Series with binned values
    """
    if binning_info['type'] == 'numerical':
        if binning_info['bins'] is None:
            return pd.Series(['Missing'] * len(df), index=df.index)

        bins = binning_info['bins']
        labels = [f"Bin_{i+1}" for i in range(len(bins)-1)]

        binned = pd.cut(df[feature],
                       bins=bins,
                       labels=labels,
                       include_lowest=True,
                       duplicates='drop')

        # Handle nulls - convert to string and then replace
        binned = binned.astype(str)
        binned[df[feature].isna()] = 'Missing'

        return binned

    else:  # categorical
        top_cats = binning_info['top_categories']

        # Convert to string for consistent comparison
        if pd.api.types.is_categorical_dtype(df[feature]):
            feature_data = df[feature].astype(str)
        else:
            feature_data = df[feature].astype(str)

        # Replace NaN string representation with 'Missing'
        feature_data = feature_data.replace('nan', 'Missing')

        # Convert top_cats to strings for comparison
        top_cats_str = [str(cat) for cat in top_cats]

        # Apply binning logic: use category name if in top_cats, else 'Others' (except for Missing)
        binned = feature_data.apply(lambda x: x if x in top_cats_str else ('Others' if x != 'Missing' else 'Missing'))

        return binned


def calculate_psi(expected_pct: pd.Series,
                  actual_pct: pd.Series,
                  epsilon: float = 0.0001) -> float:
    """
    Calculate Population Stability Index with proper epsilon handling and renormalization.

    Parameters:
    -----------
    expected_pct : pd.Series
        Expected (baseline) percentages
    actual_pct : pd.Series
        Actual percentages
    epsilon : float
        Small value to avoid log(0)

    Returns:
    --------
    PSI value
    """
    # Align indices
    all_bins = expected_pct.index.union(actual_pct.index)
    expected_pct = expected_pct.reindex(all_bins, fill_value=0)
    actual_pct = actual_pct.reindex(all_bins, fill_value=0)

    # Only add epsilon where values are zero
    expected_pct = expected_pct.apply(lambda x: epsilon if x == 0 else x)
    actual_pct = actual_pct.apply(lambda x: epsilon if x == 0 else x)

    # Renormalize to ensure they sum to 1 after adding epsilon
    expected_pct = expected_pct / expected_pct.sum()
    actual_pct = actual_pct / actual_pct.sum()

    # Calculate PSI
    psi_value = np.sum((actual_pct - expected_pct) * np.log(actual_pct / expected_pct))

    return psi_value


def calculate_month_on_month_psi(df: pd.DataFrame,
                                 feature_list: List[str],
                                 segment_columns: List[str],
                                 month_col: str = 'Application_month',
                                 data_selection_col: str = 'Data_selection',
                                 account_id_col: str = 'digitalLoanAccountId') -> pd.DataFrame:
    """
    Calculate PSI for each feature comparing training period (June 2024 to March 2025)
    vs each month after March 2025, overall and by segments.

    Parameters:
    -----------
    df : pd.DataFrame
        Input dataframe
    feature_list : List[str]
        List of features to calculate PSI for
    segment_columns : List[str]
        List of segment columns
    month_col : str
        Name of month column
    data_selection_col : str
        Name of data selection column (identifies train period)
    account_id_col : str
        Name of account ID column for counting distinct accounts

    Returns:
    --------
    pd.DataFrame with PSI values with one row per feature-month-segment combination
    """
    # Create a copy to avoid modifying original
    df = df.copy()

    # Identify training and test periods
    train_df = df[df[data_selection_col] == 'Train'].copy()
    test_df = df[df[data_selection_col] != 'Train'].copy()

    if len(train_df) == 0:
        raise ValueError("No training data found. Check Data_selection column.")

    print(f"Training period: {train_df[month_col].min()} to {train_df[month_col].max()}")
    print(f"Test period: {test_df[month_col].min()} to {test_df[month_col].max()}")

    # Identify feature types
    feature_types = identify_feature_types(df, feature_list)

    # Create binning strategy based on training period
    binning_info = create_bins_for_features(
        df,
        feature_types['numerical'],
        feature_types['categorical'],
        train_df
    )

    # Get sorted test months
    test_months = sorted(test_df[month_col].unique())

    results = []

    # Calculate overall PSI
    for feature in feature_list:
        if feature not in df.columns:
            continue

        # Apply binning to entire dataset
        df[f'{feature}_binned'] = apply_binning(df, feature, binning_info[feature])
        # print(f"Feature binned {df[f'{feature}_binned']}")
        # Get training period distribution (baseline)
        train_baseline = df[df[data_selection_col] == 'Train'][f'{feature}_binned'].value_counts(normalize=True)

        # Calculate PSI for each test month
        for month in test_months:
            actual_dist = df[df[month_col] == month][f'{feature}_binned'].value_counts(normalize=True)
            psi_value = calculate_psi(train_baseline, actual_dist)

            # Calculate average percentages across all bins
            expected_avg_pct = train_baseline.mean() * 100
            actual_avg_pct = actual_dist.mean() * 100

            # # Count distinct accounts for segment
            # base_segment_count = train_segment[account_id_col].nunique()
            # actual_segment_count = actual_segment[account_id_col].nunique()


            results.append({
                'Feature': feature,
                'Feature_Type': binning_info[feature]['type'],
                'Segment_Column': 'Overall',
                'Segment_Value': 'All',
                'Month': f"{month}",
                'Base_Month': 'Train (Jun 2024 - Mar 2025)',
                'Current_Month': month,
                'Expected_Percentage': expected_avg_pct,
                'Actual_Percentage': actual_avg_pct,
                'PSI': psi_value
            })

    # Calculate PSI by segments
    for segment_col in segment_columns:
        if segment_col not in df.columns:
            continue

        segments = df[segment_col].dropna().unique()

        for segment_val in segments:
            segment_df = df[df[segment_col] == segment_val]

            for feature in feature_list:
                if feature not in df.columns:
                    continue

                # Get training period distribution for segment
                train_segment = segment_df[segment_df[data_selection_col] == 'Train']
                if len(train_segment) == 0:
                    continue

                train_baseline = train_segment[f'{feature}_binned'].value_counts(normalize=True)

                # Calculate PSI for each test month
                for month in test_months:
                    actual_segment = segment_df[segment_df[month_col] == month]
                    if len(actual_segment) == 0:
                        continue

                    actual_dist = actual_segment[f'{feature}_binned'].value_counts(normalize=True)
                    psi_value = calculate_psi(train_baseline, actual_dist)

                    # Calculate average percentages across all bins
                    expected_avg_pct = train_baseline.mean() * 100
                    actual_avg_pct = actual_dist.mean() * 100

                    # Count distinct accounts for segment
                    base_segment_count = train_segment[account_id_col].nunique()
                    actual_segment_count = actual_segment[account_id_col].nunique()

                    results.append({
                        'Feature': feature,
                        'Feature_Type': binning_info[feature]['type'],
                        'Segment_Column': segment_col,
                        'Segment_Value': segment_val,
                        'Month': f"{month}",
                        'Base_Month': 'Train (Jun 2024 - Mar 2025)',
                        'Current_Month': month,
                        'Base_Count': base_segment_count,
                        'Actual_Count': actual_segment_count,
                        'Expected_Percentage': expected_avg_pct,
                        'Actual_Percentage': actual_avg_pct,
                        'PSI': psi_value
                    })

    return pd.DataFrame(results)


def calculate_bin_level_psi(df: pd.DataFrame,
                            feature_list: List[str],
                            segment_columns: List[str],
                            month_col: str = 'Application_month',
                            data_selection_col: str = 'Data_selection',
                            account_id_col: str = 'digitalLoanAccountId') -> pd.DataFrame:
    """
    Calculate bin-level PSI for each feature comparing training period
    vs each month after March 2025, overall and by segments.

    Parameters:
    -----------
    df : pd.DataFrame
        Input dataframe
    feature_list : List[str]
        List of features to calculate PSI for
    segment_columns : List[str]
        List of segment columns
    month_col : str
        Name of month column
    data_selection_col : str
        Name of data selection column
    account_id_col : str
        Name of account ID column for counting distinct accounts

    Returns:
    --------
    pd.DataFrame with bin-level PSI details including bin ranges
    """
    # Create a copy to avoid modifying original
    df = df.copy()

    # Identify training and test periods
    train_df = df[df[data_selection_col] == 'Train'].copy()
    test_df = df[df[data_selection_col] != 'Train'].copy()

    if len(train_df) == 0:
        raise ValueError("No training data found. Check Data_selection column.")

    print(f"Training period: {train_df[month_col].min()} to {train_df[month_col].max()}")
    print(f"Test period: {test_df[month_col].min()} to {test_df[month_col].max()}")

    # Identify feature types
    feature_types = identify_feature_types(df, feature_list)

    # Create binning strategy based on training period
    binning_info = create_bins_for_features(
        df,
        feature_types['numerical'],
        feature_types['categorical'],
        train_df
    )

    # Get sorted test months
    test_months = sorted(test_df[month_col].unique())

    results = []
    epsilon = 0.0001

    # Calculate overall bin-level PSI
    for feature in feature_list:
        if feature not in df.columns:
            continue

        # Apply binning to entire dataset
        df[f'{feature}_binned'] = apply_binning(df, feature, binning_info[feature])
        # print(df[f'{feature}_binned'])

        # Get training period distribution (baseline)
        train_baseline = df[df[data_selection_col] == 'Train'][f'{feature}_binned'].value_counts(normalize=True)

        # Calculate bin-level PSI for each test month
        for month in test_months:
            month_data = df[df[month_col] == month]
            actual_dist = month_data[f'{feature}_binned'].value_counts(normalize=True)

            # Count distinct accounts
            base_count = df[df[data_selection_col] == 'Train'][account_id_col].nunique()
            actual_count = month_data[account_id_col].nunique()

            # Get all bins
            all_bins = train_baseline.index.union(actual_dist.index)

            for bin_name in all_bins:
                # Simplified epsilon logic - no redundancy
                expected_pct = train_baseline.get(bin_name, 0)
                actual_pct = actual_dist.get(bin_name, 0)

                # Add epsilon only if zero
                expected_pct = epsilon if expected_pct == 0 else expected_pct
                actual_pct = epsilon if actual_pct == 0 else actual_pct

                # Calculate bin-level PSI
                bin_psi = (actual_pct - expected_pct) * np.log(actual_pct / expected_pct)

                # Get bin range information
                bin_ranges = binning_info[feature]['bin_ranges']
                if bin_name in bin_ranges:
                    bin_min = bin_ranges[bin_name]['min']
                    bin_max = bin_ranges[bin_name]['max']
                    bin_range = bin_ranges[bin_name]['range_str']
                else:
                    # For categorical or special bins (Missing, Others)
                    bin_min = None
                    bin_max = None
                    bin_range = bin_name

                results.append({
                    'Feature': feature,
                    'Feature_Type': binning_info[feature]['type'],
                    'Segment_Column': 'Overall',
                    'Segment_Value': 'All',
                    'Month': f"{month}",
                    'Base_Month': 'Train (Jun 2024 - Mar 2025)',
                    'Current_Month': month,
                    'Base_Count': base_count,
                    'Actual_Count': actual_count,
                    'Bin': bin_name,
                    'Bin_Range': bin_range,
                    'Bin_Min': bin_min,
                    'Bin_Max': bin_max,
                    'Base_Percentage': (train_baseline.get(bin_name, 0) * 100),
                    'Actual_Percentage': (actual_dist.get(bin_name, 0) * 100),
                    'Bin_PSI': bin_psi
                })

    # Calculate bin-level PSI by segments
    for segment_col in segment_columns:
        if segment_col not in df.columns:
            continue

        segments = df[segment_col].dropna().unique()

        for segment_val in segments:
            segment_df = df[df[segment_col] == segment_val]

            for feature in feature_list:
                if feature not in df.columns:
                    continue

                # Get training period distribution for segment
                train_segment = segment_df[segment_df[data_selection_col] == 'Train']
                if len(train_segment) == 0:
                    continue

                train_baseline = train_segment[f'{feature}_binned'].value_counts(normalize=True)

                # Calculate bin-level PSI for each test month
                for month in test_months:
                    actual_segment = segment_df[segment_df[month_col] == month]
                    if len(actual_segment) == 0:
                        continue

                    actual_dist = actual_segment[f'{feature}_binned'].value_counts(normalize=True)

                    # Count distinct accounts for segment
                    base_segment_count = train_segment[account_id_col].nunique()
                    actual_segment_count = actual_segment[account_id_col].nunique()

                    # Get all bins
                    all_bins = train_baseline.index.union(actual_dist.index)

                    for bin_name in all_bins:
                        # Simplified epsilon logic - no redundancy
                        expected_pct = train_baseline.get(bin_name, 0)
                        actual_pct = actual_dist.get(bin_name, 0)

                        # Add epsilon only if zero
                        expected_pct = epsilon if expected_pct == 0 else expected_pct
                        actual_pct = epsilon if actual_pct == 0 else actual_pct

                        # Calculate bin-level PSI
                        bin_psi = (actual_pct - expected_pct) * np.log(actual_pct / expected_pct)

                        # Get bin range information
                        bin_ranges = binning_info[feature]['bin_ranges']
                        if bin_name in bin_ranges:
                            bin_min = bin_ranges[bin_name]['min']
                            bin_max = bin_ranges[bin_name]['max']
                            bin_range = bin_ranges[bin_name]['range_str']
                        else:
                            # For categorical or special bins (Missing, Others)
                            bin_min = None
                            bin_max = None
                            bin_range = bin_name

                        results.append({
                            'Feature': feature,
                            'Feature_Type': binning_info[feature]['type'],
                            'Segment_Column': segment_col,
                            'Segment_Value': segment_val,
                            'Month': f"{month}",
                            'Base_Month': 'Train (Jun 2024 - Mar 2025)',
                            'Current_Month': month,
                            'Base_Count': base_segment_count,
                            'Actual_Count': actual_segment_count,
                            'Bin': bin_name,
                            'Bin_Range': bin_range,
                            'Bin_Min': bin_min,
                            'Bin_Max': bin_max,
                            'Base_Percentage': (train_baseline.get(bin_name, 0) * 100),
                            'Actual_Percentage': (actual_dist.get(bin_name, 0) * 100),
                            'Bin_PSI': bin_psi
                        })

    return pd.DataFrame(results)

# SIL V1

## SIL

#### Query from risk_mart.sil_risk_ds_master_20230101_20250309_v2

##### 'Alpha - CIC-SIL-Model'

In [8]:
sq = """select distinct
    r.customerId customer_id ,
    r.digitalLoanAccountId,
    r.cic_score,
    r.cic_Personal_Loans_granted_contracts_amt_24M,
    r.cic_days_since_last_inquiry, 
    r.cic_cnt_active_contracts,
    r.cic_vel_contract_nongranted_cnt_12on24,
    r.cic_max_amt_granted_24M, 
    r.cic_zero_non_granted_ever_flag,
    r.cic_tot_active_contracts_util,
    r.cic_vel_contract_granted_amt_12on24,
    r.cic_zero_granted_ever_flag,
    case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
    when lower(loanmaster.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
from risk_mart.sil_risk_ds_master_20230101_20250309_v2 r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId

where cic_score is not null;"""
data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")

Job ID 70b0b925-48da-4732-aad1-5c45867b2181 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (204453, 13)


In [9]:
feature_column = ['cic_Personal_Loans_granted_contracts_amt_24M',
       'cic_days_since_last_inquiry', 'cic_cnt_active_contracts',
       'cic_vel_contract_nongranted_cnt_12on24', 'cic_max_amt_granted_24M',
       'cic_zero_non_granted_ever_flag', 'cic_tot_active_contracts_util',
       'cic_vel_contract_granted_amt_12on24', 'cic_zero_granted_ever_flag']

In [10]:
dfd = transform_data(data, feature_column, a='cic_score', modelDisplayName='Alpha - CIC-SIL-Model') 
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs
0,3217983,e52282e7-6be3-474c-8223-b7d28352e38b,c2d98b8b-48c1-457e-af25-6d768cd70014,0.186045,2025-12-13T11:19:11.844078,2025-12-13T11:19:11.844078,Alpha - CIC-SIL-Model,v1,"{""cic_zero_non_granted_ever_flag"": 0, ""cic_zer...",sil_march 25 models,e5f8ec61-f4df-413b-8ae3-2dd70ef34403,2025-12-13T11:19:11.844078,{},,android
1,2861476,fd6f1853-8bbe-474c-805a-ded9ae18643b,1dd21ecb-71fc-41f5-a320-b7814ac28976,0.200952,2025-12-13T11:19:11.844608,2025-12-13T11:19:11.844608,Alpha - CIC-SIL-Model,v1,"{""cic_days_since_last_inquiry"": 0.0, ""cic_vel_...",sil_march 25 models,3ed7afd7-8e4f-456f-9a9d-53b89a67858a,2025-12-13T11:19:11.844608,{},,android
2,2961258,a039d429-0f93-4d8e-abe0-f9a802378903,a48c7a2c-029e-4c73-858e-3556083460da,0.126058,2025-12-13T11:19:11.844608,2025-12-13T11:19:11.844608,Alpha - CIC-SIL-Model,v1,"{""cic_days_since_last_inquiry"": 1720.0, ""cic_z...",sil_march 25 models,d43e3766-22f0-494b-b3cd-d77b0a0654ec,2025-12-13T11:19:11.844608,{},,android
3,3103351,3cca9259-4977-4790-a873-18ffc63a94c2,b88d16dd-7304-4ed0-911a-9d84209019fe,0.199273,2025-12-13T11:19:11.845149,2025-12-13T11:19:11.845149,Alpha - CIC-SIL-Model,v1,"{""cic_days_since_last_inquiry"": 0.0, ""cic_vel_...",sil_march 25 models,3a43adb7-76bc-4884-9744-a18f9b818ffd,2025-12-13T11:19:11.845149,{},,android
4,2488152,3c6ff6a1-6aff-4150-a8f7-6074d0b0c933,cc959883-8456-4d85-a3cd-d3ee25045a89,0.090228,2025-12-13T11:19:11.845149,2025-12-13T11:19:11.845149,Alpha - CIC-SIL-Model,v1,"{""cic_days_since_last_inquiry"": 209.0, ""cic_ve...",sil_march 25 models,199fc23f-6813-4547-89c7-a019ce1094e8,2025-12-13T11:19:11.845149,{},,android


In [11]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_TRUNCATE",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=3532155c-c0f4-4622-974d-62e3d488d2e9>

##### Alpha Sil Stack Model 

In [12]:
sq = """ 
select distinct 
r.customerId customer_id ,
r.digitalLoanAccountId,
r.alpha_stack_score,
r.beta_demo_score,
r.cic_score,
r.apps_score,
r.credo_gen_score,
    case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
    when lower(loanmaster.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
from `risk_mart.sil_risk_ds_master_20230101_20250309_v2` r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
where alpha_stack_score is not null
;
"""

data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")

Job ID 1302fb32-cccb-4e83-b809-f18471d42fd7 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (204453, 8)


In [14]:
data.columns
data.rename(columns={'beta_demo_score':'sb_demo_score', 'cic_score':'s_cic_score', 
                    'apps_score':'s_apps_score', 'credo_gen_score':'s_credo_score'}, inplace = True)
data.columns

Index(['customer_id', 'digitalLoanAccountId', 'alpha_stack_score',
       'sb_demo_score', 's_cic_score', 's_apps_score', 's_credo_score',
       'osType'],
      dtype='object')

In [15]:
feature_column = ['sb_demo_score',
       's_cic_score', 's_apps_score',
       's_credo_score']

dfd = transform_data(data, feature_column, a='alpha_stack_score', modelDisplayName='Alpha - StackingModel') 
dfd.head()


,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs
0,3217983,e52282e7-6be3-474c-8223-b7d28352e38b,aefbedd3-3ad5-41ac-b977-4dade59712cc,0.196466,2025-12-13T11:25:07.045384,2025-12-13T11:25:07.045384,Alpha - StackingModel,v1,"{""sb_demo_score"": 0.07787201388170713, ""s_cic_...",sil_march 25 models,29b50e23-45db-4a28-af1b-ed5b69965544,2025-12-13T11:25:07.045384,{},,android
1,3171119,ca27a27c-7661-4b9b-a5e0-007aa5ea2597,7e8d328d-f45e-43c0-9c57-dfa3b9af8f10,0.061847,2025-12-13T11:25:07.045384,2025-12-13T11:25:07.045384,Alpha - StackingModel,v1,"{""sb_demo_score"": 0.0612781115702501, ""s_cic_s...",sil_march 25 models,4110a3ef-f8b3-41ab-bb69-54c849dd7a99,2025-12-13T11:25:07.045384,{},,android
2,3103351,3cca9259-4977-4790-a873-18ffc63a94c2,b0c0e67d-831d-4a8c-a008-21b8906a4cec,0.327127,2025-12-13T11:25:07.045384,2025-12-13T11:25:07.045384,Alpha - StackingModel,v1,"{""sb_demo_score"": 0.15832345263460662, ""s_cic_...",sil_march 25 models,d99cfa86-c07d-4ae3-a4fc-0bf45526b75e,2025-12-13T11:25:07.045384,{},,android
3,3305803,8129dc45-bd8d-45d3-a394-af00bbfcab26,18e60479-7bfc-4d7c-af0b-403e4a5dafb5,0.087333,2025-12-13T11:25:07.045384,2025-12-13T11:25:07.045384,Alpha - StackingModel,v1,"{""sb_demo_score"": 0.11155911021611067, ""s_cic_...",sil_march 25 models,62334e01-fe29-416d-a04e-e95a5f37da51,2025-12-13T11:25:07.045384,{},,android
4,2999329,41a6562a-3c3b-44dc-a12d-face70a3cbf9,0da8534f-dfb0-4656-8c8a-0f7d3f1c3a6c,0.123988,2025-12-13T11:25:07.045384,2025-12-13T11:25:07.045384,Alpha - StackingModel,v1,"{""sb_demo_score"": 0.0892740704934226, ""s_cic_s...",sil_march 25 models,510d4374-b210-4c1f-9344-20b4f31f7f2d,2025-12-13T11:25:07.045384,{},,android


In [16]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=6d6f95c9-482f-4615-be9d-8891cbad5690>

##### Beta Sil App Score

In [17]:
sq = """ 
select distinct
r.customerId customer_id ,
r.digitalLoanAccountId,
r.apps_score,
r.app_cnt_absence_tag_30d,
r.app_cnt_absence_tag_90d ,
r.app_cnt_business_ever ,
r.app_cnt_competitors_30d ,
r.app_cnt_competitors_90d ,
r.app_cnt_education_ever ,
r.app_cnt_finance_7d ,
r.app_cnt_finance_90d ,
r.app_cnt_music_and_audio_ever ,
r.app_cnt_payday_90d ,
r.app_cnt_rated_for_3plus_ever ,
r.app_cnt_travel_and_local_ever ,
r.app_first_competitors_install_to_apply_days ,
r.app_first_payday_install_to_apply_days ,
r.app_median_time_bw_installed_mins_30d ,
r.app_vel_finance_30_over_365 ,
    case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
    when lower(loanmaster.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
from `risk_mart.sil_risk_ds_master_20230101_20250309_v2` r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
where apps_score is not null
"""
data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")

Job ID 54f54263-bfaf-4f06-bc9d-61322342da6d successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (317385, 20)


In [18]:
feature_column = ['app_cnt_rated_for_3plus_ever',
       'app_cnt_education_ever', 'app_cnt_business_ever',
       'app_cnt_music_and_audio_ever',
       'app_cnt_travel_and_local_ever', 'app_cnt_finance_7d',
       'app_cnt_competitors_30d', 'app_cnt_absence_tag_30d',
        'app_cnt_absence_tag_90d',
       'app_cnt_finance_90d', 'app_cnt_competitors_90d',
       'app_cnt_payday_90d',
       'app_median_time_bw_installed_mins_30d',
       'app_first_competitors_install_to_apply_days',
       'app_first_payday_install_to_apply_days',
       'app_vel_finance_30_over_365']

dfd = transform_data(data, feature_column, a='apps_score', modelDisplayName='Beta - AppsScoreModel') 
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs
0,2931012,b68a903c-98cb-4ec1-9efc-3d0b6dee6cbe,89317338-bbd2-4bea-89b1-4fb29b7b2153,0.716736,2025-12-13T11:26:00.184712,2025-12-13T11:26:00.184712,Beta - AppsScoreModel,v1,"{""app_cnt_rated_for_3plus_ever"": 12.0, ""app_cn...",sil_march 25 models,93f548a3-9101-4cf4-8874-1f8ffcd5dd00,2025-12-13T11:26:00.184712,{},,android
1,2481451,9ad43f75-21eb-4c02-a249-c1bb44b34f43,f9745ba9-951c-4bba-8522-7ce6d377c8c2,0.416705,2025-12-13T11:26:00.184712,2025-12-13T11:26:00.184712,Beta - AppsScoreModel,v1,"{""app_cnt_rated_for_3plus_ever"": 21.0, ""app_cn...",sil_march 25 models,293e12e2-328d-43d8-9ea3-01afda3184b2,2025-12-13T11:26:00.184712,{},,android
2,3300562,2165006c-b9d4-4e3b-93f2-b6bd17512121,af7bdf60-8045-4ed9-a357-c6b7a77b3b03,0.671526,2025-12-13T11:26:00.184712,2025-12-13T11:26:00.184712,Beta - AppsScoreModel,v1,"{""app_cnt_rated_for_3plus_ever"": 23.0, ""app_cn...",sil_march 25 models,963ca938-8b52-4e81-bba3-40c518aab06c,2025-12-13T11:26:00.184712,{},,android
3,2361640,c72636f0-f785-4033-a1fc-691cc7ba4b09,a79b2b2c-b44b-4c63-9a22-a3e2339114ed,0.546899,2025-12-13T11:26:00.184712,2025-12-13T11:26:00.184712,Beta - AppsScoreModel,v1,"{""app_cnt_rated_for_3plus_ever"": 12.0, ""app_cn...",sil_march 25 models,7d71b40d-b039-413a-b9c0-0a76f75fe42a,2025-12-13T11:26:00.184712,{},,android
4,3296246,80377b51-d4f0-4fb9-91f5-88d7cbd5ac94,6dd299bd-4bc4-4041-acb2-f0be000e0c5a,0.605626,2025-12-13T11:26:00.184712,2025-12-13T11:26:00.184712,Beta - AppsScoreModel,v1,"{""app_cnt_rated_for_3plus_ever"": 18.0, ""app_cn...",sil_march 25 models,37bbbbd2-9acc-4535-96f3-0d6c2fa5efeb,2025-12-13T11:26:00.184712,{},,android


In [19]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=e6ac2d6d-7b57-4c40-bf80-e9f066bc3a88>

##### Beta SIL Demo Score

In [20]:
sq = """
select distinct
r.customerId customer_id ,
r.digitalLoanAccountId,
r.beta_demo_score,
r.beta_de_ln_vas_opted_flag ,
r.beta_de_ln_doc_type_rolled ,
r.beta_de_ln_marital_status ,
r.beta_de_ln_age_bin ,
r.beta_de_ln_province_bin ,
r.beta_de_ln_ref2_type ,
r.beta_de_ln_education_level ,
r.beta_de_ln_ref1_type ,
r.beta_de_ln_industry_new_bin ,
r.beta_de_ln_appln_day_of_week ,
r.beta_de_onb_name_email_match_score ,
r.beta_de_ln_employment_type_new_bin ,
r.beta_de_ln_telconame ,
r.beta_de_time_bw_onb_loan_appln_mins ,
r.beta_de_ln_source_of_funds_new_bin ,
r.beta_de_ln_brand_bin ,
r.beta_de_ln_email_primary_domain ,
    case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
    when lower(loanmaster.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
from `risk_mart.sil_risk_ds_master_20230101_20250309_v2` r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
where beta_demo_score is not null
;
"""
data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")

Job ID b4333853-bb39-48d1-9654-7eb6be257343 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (349207, 21)


In [21]:
feature_column = ['beta_de_ln_vas_opted_flag',
       'beta_de_ln_doc_type_rolled', 'beta_de_ln_marital_status',
       'beta_de_ln_age_bin', 'beta_de_ln_province_bin',
       'beta_de_ln_ref2_type', 'beta_de_ln_education_level',
       'beta_de_ln_ref1_type', 'beta_de_ln_industry_new_bin',
       'beta_de_ln_appln_day_of_week',
       'beta_de_onb_name_email_match_score',
       'beta_de_ln_employment_type_new_bin', 'beta_de_ln_telconame',
       'beta_de_time_bw_onb_loan_appln_mins',
       'beta_de_ln_source_of_funds_new_bin', 'beta_de_ln_brand_bin',
       'beta_de_ln_email_primary_domain']

dfd = transform_data(data, feature_column, a='beta_demo_score', modelDisplayName='Beta - DemoScoreModel') 
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs
0,3162919,03576b1d-2138-4151-b8a6-c8b5364822ba,eb297dc5-0014-4f7f-9b11-f04cc5078d7f,0.196803,2025-12-13T11:28:09.625151,2025-12-13T11:28:09.625151,Beta - DemoScoreModel,v1,"{""beta_de_ln_vas_opted_flag"": ""1"", ""beta_de_ln...",sil_march 25 models,5a339656-0837-41f4-83d5-d36af0de70e3,2025-12-13T11:28:09.625151,{},,ios
1,2771545,d80f9a20-412b-4c15-baa2-54a731abd120,c5d3db66-ae05-423a-ac5a-0bfe7808061d,0.198499,2025-12-13T11:28:09.625151,2025-12-13T11:28:09.625151,Beta - DemoScoreModel,v1,"{""beta_de_ln_vas_opted_flag"": ""0"", ""beta_de_ln...",sil_march 25 models,70b1aaae-2acb-470c-8f19-273f42127a0e,2025-12-13T11:28:09.625151,{},,android
2,3200870,43e4deb5-55ee-42ee-bd8a-2ba698c8a443,afa40980-ebb7-4f21-848d-c9ea066ab98a,0.140128,2025-12-13T11:28:09.625151,2025-12-13T11:28:09.625151,Beta - DemoScoreModel,v1,"{""beta_de_ln_vas_opted_flag"": ""1"", ""beta_de_ln...",sil_march 25 models,6353048e-fd9a-49d1-8fc1-39e5f34b4be0,2025-12-13T11:28:09.625151,{},,android
3,2880547,dbf7d2f1-1365-4cda-9d46-120340cf446f,87823331-8605-45e0-9d01-7e624c95e1d9,0.185714,2025-12-13T11:28:09.625151,2025-12-13T11:28:09.625151,Beta - DemoScoreModel,v1,"{""beta_de_ln_vas_opted_flag"": ""1"", ""beta_de_ln...",sil_march 25 models,b38748f7-c8d1-4a42-967b-c5f8ae669a3f,2025-12-13T11:28:09.625151,{},,android
4,2828984,259a7f89-68e7-42bd-ba9f-ec4a99051526,0ba1e6d4-153d-4ec4-9eb2-adbaa4c6791f,0.141709,2025-12-13T11:28:09.633374,2025-12-13T11:28:09.633374,Beta - DemoScoreModel,v1,"{""beta_de_ln_vas_opted_flag"": ""1"", ""beta_de_ln...",sil_march 25 models,a6fed57f-4983-44b8-8c43-090aea63945d,2025-12-13T11:28:09.633374,{},,android


In [22]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=3d9fbf6e-3704-4675-bff6-f3491e8e301e>

##### Beta SIL STACK Score Model

In [23]:
sq = """ 
select  distinct
r.customerId customer_id ,
r.digitalLoanAccountId,
r.beta_stack_score,
r.apps_score,
r.credo_gen_score,
r.beta_demo_score,
    case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
    when lower(loanmaster.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
from `risk_mart.sil_risk_ds_master_20230101_20250309_v2` r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
where beta_stack_score is not null
"""
data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")

Job ID b54c2769-959f-44e4-8698-f540f1f31d3f successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (349207, 7)


In [24]:
feature_column = ['apps_score', 'credo_gen_score', 'beta_demo_score']
dfd = transform_data(data, feature_column, a='beta_stack_score', modelDisplayName='Beta - StackScoreModel') 
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs
0,2482999,03fb073f-760c-4ba9-bcd4-b3fff2108c77,2cd39756-375a-4b71-a228-be049dc47792,0.249859,2025-12-13T11:30:25.531755,2025-12-13T11:30:25.531755,Beta - StackScoreModel,v1,"{""apps_score"": 0.6665963232507546, ""credo_gen_...",sil_march 25 models,e9458faf-2186-4552-9502-f6f5ee9222be,2025-12-13T11:30:25.531755,{},,android
1,2974885,9dfaa4c6-381f-4b14-84ee-e406886f9500,5145272a-0d96-4fb4-b48f-49ff59394bf1,0.012713,2025-12-13T11:30:25.531755,2025-12-13T11:30:25.531755,Beta - StackScoreModel,v1,"{""apps_score"": 0.31590694029617394, ""credo_gen...",sil_march 25 models,c45980f9-1ab1-43fa-bdb1-7409feda8b5b,2025-12-13T11:30:25.531755,{},,android
2,2283846,5a00936a-6c54-4128-98d4-08dae58b0c89,d0e78fd2-8366-42ad-a4ad-83f848985fbc,0.382341,2025-12-13T11:30:25.531755,2025-12-13T11:30:25.531755,Beta - StackScoreModel,v1,"{""apps_score"": 0.7075294646384661, ""credo_gen_...",sil_march 25 models,25aaf6b4-818e-4f9b-acb5-f4d46b9726f2,2025-12-13T11:30:25.531755,{},,android
3,3217983,e52282e7-6be3-474c-8223-b7d28352e38b,1221518d-a32c-42e2-828f-044e7288a231,0.107526,2025-12-13T11:30:25.533765,2025-12-13T11:30:25.533765,Beta - StackScoreModel,v1,"{""apps_score"": 0.5786625970137902, ""credo_gen_...",sil_march 25 models,5b968065-3ea4-436c-9d20-b757258f0106,2025-12-13T11:30:25.533765,{},,android
4,2184080,38c5bbe3-3690-49ad-bacd-5c81c1c4da74,924bf31b-f42a-4df8-9196-c8d59edc13f3,0.101342,2025-12-13T11:30:25.533765,2025-12-13T11:30:25.533765,Beta - StackScoreModel,v1,"{""apps_score"": 0.5681554218898229, ""credo_gen_...",sil_march 25 models,efdfb311-64e9-4c56-aa5d-09e8bf6d34fc,2025-12-13T11:30:25.533765,{},,android


In [25]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=ba8050f0-0058-42e3-a2f5-b4f09f6fdeef>

##### Alpha  - IncomeEstimationModel

In [26]:
sq = """  
Select 
distinct
r.customerId customer_id ,
r.digitalLoanAccountId,
r.alpha_estimated_income,
r.inc_alpha_cic_credit_avg_credit_limit,
r.inc_alpha_cic_max_active_contracts_amt,
r.inc_alpha_ln_company_name,
r.inc_alpha_ln_age,
r.inc_alpha_doc_type_rolled,
r.inc_alpha_ln_brand,
r.inc_alpha_ln_city,
r.inc_alpha_ln_cnt_dependents,
r.inc_alpha_ln_education_level,
r.inc_alpha_ln_employment_type_new,
r.inc_alpha_ln_gender,
r.inc_alpha_ln_industry_new,
r.inc_alpha_ln_loan_prod_type,
r.inc_alpha_ln_marital_status_new,
r.inc_alpha_ln_nature_of_work_new,
r.inc_alpha_ln_osversion_bin,
r.inc_alpha_ln_purpose,
r.inc_alpha_ln_source_of_funds_new,
r.inc_alpha_loan_monthly_income,
r.inc_alpha_encoded_company_name_grouped,
    case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
    when lower(loanmaster.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
from `risk_mart.sil_risk_ds_master_20230101_20250309_v2` r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
where r.alpha_estimated_income is not null
;
"""
data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")


Job ID 5bfa4411-8520-4be1-bbf1-f841af8c1ff0 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (349207, 24)


In [27]:
feature_column = ['inc_alpha_cic_credit_avg_credit_limit',
       'inc_alpha_cic_max_active_contracts_amt', 'inc_alpha_ln_age',
       'inc_alpha_doc_type_rolled', 'inc_alpha_ln_brand', 'inc_alpha_ln_city',
       'inc_alpha_ln_cnt_dependents', 'inc_alpha_ln_education_level',
       'inc_alpha_ln_employment_type_new', 'inc_alpha_ln_gender',
       'inc_alpha_ln_industry_new', 'inc_alpha_ln_loan_prod_type',
       'inc_alpha_ln_marital_status_new', 'inc_alpha_ln_nature_of_work_new',
       'inc_alpha_ln_osversion_bin', 'inc_alpha_ln_purpose',
       'inc_alpha_ln_source_of_funds_new',
       'inc_alpha_encoded_company_name_grouped']
dfd = transform_data(data, feature_column, a='alpha_estimated_income', modelDisplayName='Alpha  - IncomeEstimationModel') 
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs
0,2767325,5f14e5a7-f079-4b26-b7c5-27d544b0d0c8,03afaf89-ee68-4d9a-b788-7548251fd4f7,16000.494737,2025-12-13T11:31:46.353258,2025-12-13T11:31:46.353258,Alpha - IncomeEstimationModel,v1,"{""inc_alpha_cic_credit_avg_credit_limit"": 0.0,...",sil_march 25 models,27affafa-426d-4b6f-8ae2-4198840934c8,2025-12-13T11:31:46.353258,{},,android
1,2794973,f086611e-cb57-4d59-bc1a-aaeab764a40c,45030187-75f1-4bfd-acc7-3fd22a385c8f,30274.149490,2025-12-13T11:31:46.353946,2025-12-13T11:31:46.353946,Alpha - IncomeEstimationModel,v1,"{""inc_alpha_cic_credit_avg_credit_limit"": 0.0,...",sil_march 25 models,87d2b55e-9067-48cc-a58f-405943b9fc03,2025-12-13T11:31:46.353946,{},,android
2,2638253,02295d9f-6c43-474b-970c-2311e8199cbf,90a386dc-a6b2-4a67-8e72-7749375fa56e,26274.119480,2025-12-13T11:31:46.353946,2025-12-13T11:31:46.353946,Alpha - IncomeEstimationModel,v1,"{""inc_alpha_cic_credit_avg_credit_limit"": 0.0,...",sil_march 25 models,1d65649f-5164-42b8-8870-1e3961cc3de0,2025-12-13T11:31:46.353946,{},,ios
3,2783108,484d5de2-0017-4294-979c-c2c2b7505f8e,7ad6b2e9-2e41-4618-bf4f-be8ebed05828,32555.265227,2025-12-13T11:31:46.354512,2025-12-13T11:31:46.354512,Alpha - IncomeEstimationModel,v1,"{""inc_alpha_cic_credit_avg_credit_limit"": 2550...",sil_march 25 models,c005fa30-c6f6-4429-8dae-2298e3795da5,2025-12-13T11:31:46.354512,{},,android
4,2451964,58e4f0e2-c922-4085-a726-2ba98a14af5e,afbd89bc-2d55-417e-b0cc-a60b6b7dbf23,29085.788158,2025-12-13T11:31:46.354982,2025-12-13T11:31:46.354982,Alpha - IncomeEstimationModel,v1,"{""inc_alpha_cic_credit_avg_credit_limit"": 0.0,...",sil_march 25 models,f8474b51-9f13-4980-85e6-f825cff908c7,2025-12-13T11:31:46.354982,{},,android


In [28]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=4c133d2d-c9b3-4828-87f5-346d5a772ba8>

##### Beta - IncomeEstimationModelm

In [29]:
sq = """ 
select
distinct
r.customerId customer_id ,
r.digitalLoanAccountId,
r.beta_estimated_income,
r.inc_beta_ln_loan_type,
r.inc_beta_ln_education_level,
r.inc_beta_ln_employment_type_new,
r.inc_beta_ln_industry_new,
r.inc_beta_ln_age,
r.inc_beta_ln_brand,
r.inc_beta_ln_city,
r.inc_beta_ln_purpose,
r.inc_beta_ln_osversion_bin,
r.inc_beta_ln_postal_code,
r.inc_beta_ln_gender,
r.inc_beta_ln_doc_type_rolled,
r.inc_beta_ln_cnt_dependents,
r.inc_beta_ln_source_of_funds_new,
r.inc_beta_ln_marital_status_new,
r.inc_beta_encoded_company_name_grouped,
    case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
    when lower(loanmaster.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
from `risk_mart.sil_risk_ds_master_20230101_20250309_v2` r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
where r.alpha_estimated_income is not null
;
"""

# data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
data = client.query(sq).result().to_arrow().to_pandas()
print(f"The shape of the dataframe is:\t {data.shape}")

The shape of the dataframe is:	 (349207, 20)


In [30]:
feature_column = ['inc_beta_ln_loan_type',
       'inc_beta_ln_education_level', 'inc_beta_ln_employment_type_new',
       'inc_beta_ln_industry_new', 'inc_beta_ln_age', 'inc_beta_ln_brand',
       'inc_beta_ln_city', 'inc_beta_ln_purpose', 'inc_beta_ln_osversion_bin',
       'inc_beta_ln_postal_code', 'inc_beta_ln_gender',
       'inc_beta_ln_doc_type_rolled', 'inc_beta_ln_cnt_dependents',
       'inc_beta_ln_source_of_funds_new', 'inc_beta_ln_marital_status_new',
       'inc_beta_encoded_company_name_grouped',]

dfd = transform_data(data, feature_column, a='beta_estimated_income', modelDisplayName='Beta - IncomeEstimationModel') 
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs
0,3044281,d80398df-9a74-4bf5-8203-002db8a02714,5855ee44-f301-4e20-8b45-fd7ff917a9b0,8380.813101,2025-12-13T11:33:13.851728,2025-12-13T11:33:13.851728,Beta - IncomeEstimationModel,v1,"{""inc_beta_ln_loan_type"": ""SIL-Instore"", ""inc_...",sil_march 25 models,160e7b5c-6658-4104-9607-98593884027e,2025-12-13T11:33:13.851728,{},,ios
1,2847981,41791fe3-4efc-4bb6-85e6-0ad54d9e5676,1588ba41-a564-40da-b9ee-4a0f04d5eeb7,8873.529832,2025-12-13T11:33:13.852732,2025-12-13T11:33:13.852732,Beta - IncomeEstimationModel,v1,"{""inc_beta_ln_loan_type"": ""SIL-Instore"", ""inc_...",sil_march 25 models,6645ecfa-9dbd-4c2f-807d-a3b6b962fbed,2025-12-13T11:33:13.852732,{},,android
2,3201733,0cb9594d-b391-4500-985f-cc7be47038da,ef427474-d745-4a13-b81d-a7b070a7e1cd,8437.662500,2025-12-13T11:33:13.852732,2025-12-13T11:33:13.852732,Beta - IncomeEstimationModel,v1,"{""inc_beta_ln_loan_type"": ""SIL-Instore"", ""inc_...",sil_march 25 models,b775f5a4-f10c-4f55-ad20-95bbf4cacae4,2025-12-13T11:33:13.852732,{},,ios
3,3070086,b2f845db-8f6b-45a3-8446-0423246d5401,f33f370e-cb8c-4a98-b1bf-33e34bc4c2e4,7633.286544,2025-12-13T11:33:13.852732,2025-12-13T11:33:13.852732,Beta - IncomeEstimationModel,v1,"{""inc_beta_ln_loan_type"": ""SIL-Instore"", ""inc_...",sil_march 25 models,41256b30-f61a-418f-9758-385766e2c476,2025-12-13T11:33:13.852732,{},,android
4,2342891,c3cf07aa-6c5d-41d7-8938-c128aa801279,db542b51-4bf3-4206-a8a2-7c4e9b88b464,8239.238095,2025-12-13T11:33:13.852732,2025-12-13T11:33:13.852732,Beta - IncomeEstimationModel,v1,"{""inc_beta_ln_loan_type"": ""SIL-Instore"", ""inc_...",sil_march 25 models,035f729e-e908-4c58-a9fd-d9cd385c8028,2025-12-13T11:33:13.852732,{},,android


In [31]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=3f4ac4fa-2379-45b8-9bba-cd74f1afeb62>

# Cash V1

## Cash

##### Alpha Cash Stack Model

###### Trench 1

In [32]:
sq = """ 
select 
  r.customer_id,
  r.digitalLoanAccountId, 
  r.stack_score,
  r.demo_score,
  r.apps_score,
  r.credo_score,
  r.cic_score,
  r.stack_score_norm,
  r.ln_os_type osType,
from worktable_data_analysis.cash_alpha_trench1_applied_loans_backscored_20241001_20250930 r;
"""
data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")

Job ID 3752f892-605d-475c-aa1a-78f45774c3bc successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (67807, 9)


In [33]:
feature_column = ['demo_score', 'apps_score', 'credo_score', 'cic_score', 'stack_score_norm']

dfd = transform_data(data, feature_column, a='stack_score', modelDisplayName='Alpha-Cash-Stack-Model', tc='Trench 1', subscription_name = 'Cash September 25 Models') 
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs
0,3539915,04faa4fc-40df-4c35-8fd4-7873f4f27d7a,b2f5712e-d3c8-49d3-a7d8-03322774d348,0.669111,2025-12-13T11:34:23.872096,2025-12-13T11:34:23.872096,Alpha-Cash-Stack-Model,v1,"{""demo_score"": 0.5744555950383494, ""credo_scor...",Cash September 25 Models,b57ec27d-fa56-41a6-aa91-3a953f7b64b5,2025-12-13T11:34:23.872096,{},Trench 1,iOS
1,3556922,e30aad68-3baa-4439-a5cc-8a742648b200,512e37bc-8670-402f-b79c-fbb619a32156,0.316267,2025-12-13T11:34:23.872096,2025-12-13T11:34:23.872096,Alpha-Cash-Stack-Model,v1,"{""demo_score"": 0.44932706141455475, ""credo_sco...",Cash September 25 Models,cb8d14b1-f823-4535-bbf0-20ee84ef5c0a,2025-12-13T11:34:23.872096,{},Trench 1,iOS
2,3242405,16e5ac4b-e966-4212-9f7b-9d24bb15a8dc,44c305ed-d53a-48fa-afd2-55d05f158047,0.311066,2025-12-13T11:34:23.873096,2025-12-13T11:34:23.873096,Alpha-Cash-Stack-Model,v1,"{""demo_score"": 0.40506523274601347, ""credo_sco...",Cash September 25 Models,43b341e4-6358-48a6-8260-334c8bc2dd17,2025-12-13T11:34:23.873096,{},Trench 1,iOS
3,3646632,30b64b4a-5982-4e8c-b96e-e064aa605845,022f2734-db1c-4500-9cd1-10597ececf6b,0.533312,2025-12-13T11:34:23.873096,2025-12-13T11:34:23.873096,Alpha-Cash-Stack-Model,v1,"{""demo_score"": 0.5261059729214972, ""credo_scor...",Cash September 25 Models,2b20cd06-1550-4f31-b25e-785e06e62ccc,2025-12-13T11:34:23.873096,{},Trench 1,iOS
4,3454325,c96d5134-cf15-4393-a867-2220c7f176cc,67106cb9-9887-4f5f-b0d8-2066c9375dd8,0.433226,2025-12-13T11:34:23.873096,2025-12-13T11:34:23.873096,Alpha-Cash-Stack-Model,v1,"{""demo_score"": 0.39591531683005676, ""credo_sco...",Cash September 25 Models,3d46f84e-e096-4f5e-89e4-fe5a854cb4b8,2025-12-13T11:34:23.873096,{},Trench 1,iOS


In [34]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=9f09d78a-0f5b-43f5-a40f-269a5297390d>

###### Trench 2

In [35]:
sq = """ 
select 
  r.customer_id,
  r.digitalLoanAccountId, 
  r.stack_score,
  r.demo_score,
  r.apps_score,
  r.credo_score,
  r.cic_score,
  r.stack_score_norm,
  r.ln_os_type osType,
from worktable_data_analysis.cash_alpha_trench2_applied_loans_backscored_20241001_20250930 r;
"""
data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")

Job ID ebe53612-bbc6-42ec-9f19-a52f4d58cf1c successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (42982, 9)


In [36]:
feature_column = ['demo_score', 'apps_score', 'credo_score', 'cic_score', 'stack_score_norm']

dfd = transform_data(data, feature_column, a='stack_score', modelDisplayName='Alpha-Cash-Stack-Model', tc='Trench 2', subscription_name = 'Cash September 25 Models') 
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs
0,3239336,9386c275-dbf0-4c0a-8d02-8071f592b37d,ad2a44e2-9fce-4494-a469-2d0c558eee73,0.672123,2025-12-13T11:34:41.717484,2025-12-13T11:34:41.717484,Alpha-Cash-Stack-Model,v1,"{""demo_score"": 0.6247828133286182, ""credo_scor...",Cash September 25 Models,acee6310-8dd1-477a-89ff-ba58bdaf06e2,2025-12-13T11:34:41.717484,{},Trench 2,iOS
1,2234712,d5b9351b-4d02-4052-8296-8248cce581fe,9782a6c7-68ff-466c-a889-1f0baa1a7f33,0.618780,2025-12-13T11:34:41.717484,2025-12-13T11:34:41.717484,Alpha-Cash-Stack-Model,v1,"{""demo_score"": 0.4714931796785411, ""credo_scor...",Cash September 25 Models,b0f778f5-cb85-41fc-87e8-c5f962a6627f,2025-12-13T11:34:41.717484,{},Trench 2,iOS
2,1147827,d5fd2224-f704-414f-8294-4fc98451fa61,15bff0be-d4d4-4b56-84e4-ecd71386e58d,0.513876,2025-12-13T11:34:41.718483,2025-12-13T11:34:41.718483,Alpha-Cash-Stack-Model,v1,"{""demo_score"": 0.5756542624288734, ""credo_scor...",Cash September 25 Models,0a01118d-54f9-46e1-afaa-9c3a36a09f9e,2025-12-13T11:34:41.718483,{},Trench 2,iOS
3,1924318,db06a8d7-5eaa-42f0-8bca-c2709a7bfe39,b7a83703-a7e9-48f2-8dc2-6d1b2111302a,0.687939,2025-12-13T11:34:41.718483,2025-12-13T11:34:41.718483,Alpha-Cash-Stack-Model,v1,"{""demo_score"": 0.575024075027459, ""credo_score...",Cash September 25 Models,d282dd7e-8fae-4c8a-ac4c-e2a881e97511,2025-12-13T11:34:41.718483,{},Trench 2,iOS
4,3042999,517586f3-eb4c-4401-9ccf-f12da8d552fc,e3292b5c-9505-4752-91f4-06c4f18eb294,0.585185,2025-12-13T11:34:41.718483,2025-12-13T11:34:41.718483,Alpha-Cash-Stack-Model,v1,"{""demo_score"": 0.646694460802266, ""credo_score...",Cash September 25 Models,37cb1e52-5f4e-4f2b-bb98-a4c94bc287fe,2025-12-13T11:34:41.718483,{},Trench 2,iOS


In [37]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=6ab63116-d9bc-464b-b660-19cb60a19fe8>

###### Trench 3

In [38]:
sq = """ 
select 
  r.customer_id,
  r.digitalLoanAccountId, 
  r.stack_score,
  r.demo_score,
  r.apps_score,
  r.credo_score,
  r.cic_score,
  r.stack_score_norm,
  r.ln_os_type osType,
from worktable_data_analysis.cash_alpha_trench3_applied_loans_backscored_20241001_20250930 r;
"""
data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")

Job ID 13c90dff-09a0-476c-93b8-48edfa4eeaad successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (18269, 9)


In [39]:
feature_column = ['demo_score', 'apps_score', 'credo_score', 'cic_score', 'stack_score_norm']

dfd = transform_data(data, feature_column, a='stack_score', modelDisplayName='Alpha-Cash-Stack-Model', tc='Trench 3', subscription_name = 'Cash September 25 Models') 
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs
0,3169262,96426bd5-17d6-4fe0-bd14-3d7aa84dbe34,81d819f0-2bcf-44bc-9eab-8fd53d0a24ed,0.523043,2025-12-13T11:34:53.559216,2025-12-13T11:34:53.559216,Alpha-Cash-Stack-Model,v1,"{""demo_score"": 0.424909388603803, ""credo_score...",Cash September 25 Models,afb56977-b844-49e5-ab30-f0b1054548d7,2025-12-13T11:34:53.559216,{},Trench 3,iOS
1,1494932,4c192f35-a31e-438e-8fd0-ec7273a5ae39,672e2b65-afc1-4f0c-9edf-91ec82e90b7e,0.415618,2025-12-13T11:34:53.559216,2025-12-13T11:34:53.559216,Alpha-Cash-Stack-Model,v1,"{""demo_score"": 0.3984364213949569, ""credo_scor...",Cash September 25 Models,2c0f2cd2-8da6-46f2-89da-ad3f7bfccfd0,2025-12-13T11:34:53.559216,{},Trench 3,iOS
2,2735400,fb47df8c-0a1a-45c1-a10e-d2b53b9e2453,cc601d3f-84ae-486b-9cce-216bfeeff531,0.373629,2025-12-13T11:34:53.559216,2025-12-13T11:34:53.559216,Alpha-Cash-Stack-Model,v1,"{""demo_score"": 0.21634670864737435, ""credo_sco...",Cash September 25 Models,29624165-dabd-4432-b942-852b0dd8fc09,2025-12-13T11:34:53.559216,{},Trench 3,iOS
3,1331628,ff48b3ad-9111-4ffe-9a58-4e838afc0410,8830a79c-c1dd-403b-a65f-db3356715716,0.400812,2025-12-13T11:34:53.559216,2025-12-13T11:34:53.559216,Alpha-Cash-Stack-Model,v1,"{""demo_score"": 0.49639525992168604, ""credo_sco...",Cash September 25 Models,86a9d9ef-8bf3-4757-9549-d9c64927b120,2025-12-13T11:34:53.559216,{},Trench 3,iOS
4,2225608,485aac6c-1684-4698-be83-c08fdb74a1fd,0a4212d8-06a8-4910-995b-b35fd653e6e0,0.397827,2025-12-13T11:34:53.559216,2025-12-13T11:34:53.559216,Alpha-Cash-Stack-Model,v1,"{""demo_score"": 0.5367553576956362, ""credo_scor...",Cash September 25 Models,a4033e9c-3d90-4a91-821e-7aaaac45738a,2025-12-13T11:34:53.559216,{},Trench 3,iOS


In [40]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=6a696245-ced5-4a0a-b140-2ece6babcd9b>

##### Alpha Cash CIC Model

###### Trench 1

In [41]:
sq = """ 
select 
lmt.customerId customer_id,
r.digitalLoanAccountId,
r.ca_cic_score,
r.max_age_all_contracts_snapshot,
r.ratio_overdue_contracts_to_granted_contracts,
r.ScoreRange,
r.ln_loan_level_user_type,
r.has_ever_been_overdue,
r.latest_granted_contract_overdue_flag,
r.ratio_closed_over_new_granted_cnt_24M,
r.ratio_risky_contracts_to_granted_contracts,
r.Short_and_Term_Loans_granted_contracts_cnt_24M,
r.flg_zero_non_granted_ever,
r.Personal_Loans_granted_contracts_amt_24M,
r.CreditAvgCreditLimit,
r.flg_zero_granted_ever,
    case when lower(coalesce(lmt.osversion_v2, lmt.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(lmt.osversion_v2, lmt.osVersion)) like '%os%' then 'ios'
    when lower(lmt.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
from worktable_data_analysis.cash_alpha_cic_all_applied_backscored_20240901_20250930 r
left join `risk_credit_mis.loan_master_table` lmt on lmt.digitalLoanAccountId = r.digitalLoanAccountId
where trench_category = 'Trench 1'
;
"""

data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")


Job ID b15e2c9e-af3f-40ae-9a97-287da1f6fcf5 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (77967, 17)


In [42]:
data.columns

Index(['customer_id', 'digitalLoanAccountId', 'ca_cic_score',
       'max_age_all_contracts_snapshot',
       'ratio_overdue_contracts_to_granted_contracts', 'ScoreRange',
       'ln_loan_level_user_type', 'has_ever_been_overdue',
       'latest_granted_contract_overdue_flag',
       'ratio_closed_over_new_granted_cnt_24M',
       'ratio_risky_contracts_to_granted_contracts',
       'Short_and_Term_Loans_granted_contracts_cnt_24M',
       'flg_zero_non_granted_ever', 'Personal_Loans_granted_contracts_amt_24M',
       'CreditAvgCreditLimit', 'flg_zero_granted_ever', 'osType'],
      dtype='object')

In [43]:
feature_column = ['max_age_all_contracts_snapshot',
       'ratio_overdue_contracts_to_granted_contracts', 'ScoreRange',
       'ln_loan_level_user_type', 'has_ever_been_overdue',
       'latest_granted_contract_overdue_flag',
       'ratio_closed_over_new_granted_cnt_24M',
       'ratio_risky_contracts_to_granted_contracts',
       'Short_and_Term_Loans_granted_contracts_cnt_24M',
       'flg_zero_non_granted_ever', 'Personal_Loans_granted_contracts_amt_24M',
       'CreditAvgCreditLimit', 'flg_zero_granted_ever', 'ca_cic_score']

dfd = transform_data(data, feature_column, a='ca_cic_score', modelDisplayName='Alpha-Cash-CIC-Model', tc='Trench 1', subscription_name = 'Cash September 25 Models') 
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs
0,3487664,6756279c-638a-456c-93bc-f573291bb369,f2f3d894-7078-48b4-93cb-48a80257b6e7,0.646333,2025-12-13T11:35:02.354685,2025-12-13T11:35:02.354685,Alpha-Cash-CIC-Model,v1,"{""max_age_all_contracts_snapshot"": 871.0, ""rat...",Cash September 25 Models,7e826c1b-ea79-4359-8836-221fd8ae3f9e,2025-12-13T11:35:02.354685,{},Trench 1,android
1,3028475,60419b04-bf65-4722-87ac-afe0590fee5f,5084e7c1-b121-4732-b659-ba5f9cf85c80,0.589209,2025-12-13T11:35:02.355693,2025-12-13T11:35:02.355693,Alpha-Cash-CIC-Model,v1,"{""max_age_all_contracts_snapshot"": 445.0, ""rat...",Cash September 25 Models,5ae13e12-b50a-4588-93cc-e7d0fa95a665,2025-12-13T11:35:02.355693,{},Trench 1,ios
2,3683637,ede2cfe8-2b80-4f2d-9add-f85e4fe2209b,2e34e18a-8ea5-480f-8328-c4d69b7e8ac8,0.382389,2025-12-13T11:35:02.355693,2025-12-13T11:35:02.355693,Alpha-Cash-CIC-Model,v1,"{""max_age_all_contracts_snapshot"": 2571.0, ""ra...",Cash September 25 Models,85f21d27-e43e-4beb-b1d8-4654cf806c29,2025-12-13T11:35:02.355693,{},Trench 1,ios
3,3331343,f9642f9d-cef6-42e6-ae80-6a7787ed3dfe,0d51b643-0f45-4960-b367-a44b3ca7552b,0.297572,2025-12-13T11:35:02.355693,2025-12-13T11:35:02.355693,Alpha-Cash-CIC-Model,v1,"{""max_age_all_contracts_snapshot"": 3655.0, ""ra...",Cash September 25 Models,639f9337-0197-4753-8bed-3a0fa892cc40,2025-12-13T11:35:02.355693,{},Trench 1,android
4,3675358,aa9cf08f-0e37-4f34-a7bf-ae2e3fb7983c,fd347ad6-88a0-4049-a02a-2614850b3b3d,0.489649,2025-12-13T11:35:02.356688,2025-12-13T11:35:02.356688,Alpha-Cash-CIC-Model,v1,"{""max_age_all_contracts_snapshot"": 875.0, ""rat...",Cash September 25 Models,417bf304-f594-4647-9a89-f4e2e4cd7932,2025-12-13T11:35:02.356688,{},Trench 1,ios


In [44]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=d01fa140-6cd0-4759-bd98-aa09e097377b>

###### Trench 2

In [45]:
sq = """ 
select 
lmt.customerId customer_id,
r.digitalLoanAccountId,
r.ca_cic_score,
r.max_age_all_contracts_snapshot,
r.ratio_overdue_contracts_to_granted_contracts,
r.ScoreRange,
r.ln_loan_level_user_type,
r.has_ever_been_overdue,
r.latest_granted_contract_overdue_flag,
r.ratio_closed_over_new_granted_cnt_24M,
r.ratio_risky_contracts_to_granted_contracts,
r.Short_and_Term_Loans_granted_contracts_cnt_24M,
r.flg_zero_non_granted_ever,
r.Personal_Loans_granted_contracts_amt_24M,
r.CreditAvgCreditLimit,
r.flg_zero_granted_ever,
    case when lower(coalesce(lmt.osversion_v2, lmt.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(lmt.osversion_v2, lmt.osVersion)) like '%os%' then 'ios'
    when lower(lmt.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
from worktable_data_analysis.cash_alpha_cic_all_applied_backscored_20240901_20250930 r
left join `risk_credit_mis.loan_master_table` lmt on lmt.digitalLoanAccountId = r.digitalLoanAccountId
where trench_category = 'Trench 2'
;
"""


data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")

Job ID 759d4572-f9c0-42b7-a9eb-29c376433165 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (48479, 17)


In [46]:
feature_column = ['max_age_all_contracts_snapshot',
       'ratio_overdue_contracts_to_granted_contracts', 'ScoreRange',
       'ln_loan_level_user_type', 'has_ever_been_overdue',
       'latest_granted_contract_overdue_flag',
       'ratio_closed_over_new_granted_cnt_24M',
       'ratio_risky_contracts_to_granted_contracts',
       'Short_and_Term_Loans_granted_contracts_cnt_24M',
       'flg_zero_non_granted_ever', 'Personal_Loans_granted_contracts_amt_24M',
       'CreditAvgCreditLimit', 'flg_zero_granted_ever', 'ca_cic_score']

dfd = transform_data(data, feature_column, a='ca_cic_score', modelDisplayName='Alpha-Cash-CIC-Model', tc='Trench 2', subscription_name = 'Cash September 25 Models') 
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs
0,3280522,ffbcbc6d-9360-471f-95ad-ef6cf34cf4a3,c400483d-57f6-4121-ad37-154a5c65b77d,0.737991,2025-12-13T11:35:23.499309,2025-12-13T11:35:23.499309,Alpha-Cash-CIC-Model,v1,"{""max_age_all_contracts_snapshot"": 888.0, ""rat...",Cash September 25 Models,a618b994-ff86-4b8b-a1b0-2aeb04766a82,2025-12-13T11:35:23.499309,{},Trench 2,ios
1,1621519,302dd95f-115d-43d1-94d1-6fe9972cbc7b,eb7c72c3-5840-42cc-b465-79ba46ce9e58,0.461257,2025-12-13T11:35:23.499309,2025-12-13T11:35:23.499309,Alpha-Cash-CIC-Model,v1,"{""max_age_all_contracts_snapshot"": 2180.0, ""ra...",Cash September 25 Models,be279656-e9af-4a09-9f06-8f862ae7b49d,2025-12-13T11:35:23.499309,{},Trench 2,android
2,1901317,b3c19e51-9449-4d9f-af51-ccae3d032ebe,11d5ea33-cc93-425e-a9db-98bdae771a3e,0.495179,2025-12-13T11:35:23.499309,2025-12-13T11:35:23.499309,Alpha-Cash-CIC-Model,v1,"{""max_age_all_contracts_snapshot"": 1801.0, ""ra...",Cash September 25 Models,43af3a35-a62e-4ebc-8954-4b0ff90e4831,2025-12-13T11:35:23.499309,{},Trench 2,ios
3,2591654,47b3b1dc-9ed8-4c1c-9974-b5c447a3890a,484aca0d-44ce-4bbb-b3ce-f7a489c70e7e,0.498306,2025-12-13T11:35:23.500309,2025-12-13T11:35:23.500309,Alpha-Cash-CIC-Model,v1,"{""max_age_all_contracts_snapshot"": 1782.0, ""ra...",Cash September 25 Models,d37d2ed5-afed-4460-b441-4d7fbff8ab3e,2025-12-13T11:35:23.500309,{},Trench 2,android
4,3408113,d8fbec16-20a5-4282-9b18-ca4a08cc35ff,5587387b-1324-4c77-9359-b4be9b8918d7,0.542989,2025-12-13T11:35:23.500309,2025-12-13T11:35:23.500309,Alpha-Cash-CIC-Model,v1,"{""max_age_all_contracts_snapshot"": 1129.0, ""ra...",Cash September 25 Models,8370090d-19b2-4e37-b56a-2b6b1b54e486,2025-12-13T11:35:23.500309,{},Trench 2,ios


In [47]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=f26890c9-f0ba-4848-ad4d-2679a526b286>

###### Trench 3

In [48]:
sq = """ 
select 
lmt.customerId customer_id,
r.digitalLoanAccountId,
r.ca_cic_score,
r.max_age_all_contracts_snapshot,
r.ratio_overdue_contracts_to_granted_contracts,
r.ScoreRange,
r.ln_loan_level_user_type,
r.has_ever_been_overdue,
r.latest_granted_contract_overdue_flag,
r.ratio_closed_over_new_granted_cnt_24M,
r.ratio_risky_contracts_to_granted_contracts,
r.Short_and_Term_Loans_granted_contracts_cnt_24M,
r.flg_zero_non_granted_ever,
r.Personal_Loans_granted_contracts_amt_24M,
r.CreditAvgCreditLimit,
r.flg_zero_granted_ever,
    case when lower(coalesce(lmt.osversion_v2, lmt.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(lmt.osversion_v2, lmt.osVersion)) like '%os%' then 'ios'
    when lower(lmt.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
from worktable_data_analysis.cash_alpha_cic_all_applied_backscored_20240901_20250930 r
left join `risk_credit_mis.loan_master_table` lmt on lmt.digitalLoanAccountId = r.digitalLoanAccountId
where trench_category = 'Trench 3'
;
"""


data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")

Job ID 8e5b7b08-166b-423e-b110-aeafde6acbad successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (19300, 17)


In [49]:
feature_column = ['max_age_all_contracts_snapshot',
       'ratio_overdue_contracts_to_granted_contracts', 'ScoreRange',
       'ln_loan_level_user_type', 'has_ever_been_overdue',
       'latest_granted_contract_overdue_flag',
       'ratio_closed_over_new_granted_cnt_24M',
       'ratio_risky_contracts_to_granted_contracts',
       'Short_and_Term_Loans_granted_contracts_cnt_24M',
       'flg_zero_non_granted_ever', 'Personal_Loans_granted_contracts_amt_24M',
       'CreditAvgCreditLimit', 'flg_zero_granted_ever', 'ca_cic_score']

dfd = transform_data(data, feature_column, a='ca_cic_score', modelDisplayName='Alpha-Cash-CIC-Model', tc='Trench 3', subscription_name = 'Cash September 25 Models') 
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs
0,1349208,3380cf70-0656-456d-9c30-c7debe04ef44,690d5daa-2a98-4a85-9e5a-ab5aa26e6e51,0.354534,2025-12-13T11:35:38.123120,2025-12-13T11:35:38.123120,Alpha-Cash-CIC-Model,v1,"{""max_age_all_contracts_snapshot"": 4024.0, ""ra...",Cash September 25 Models,95a591cb-f53b-40b6-909e-fb2825648536,2025-12-13T11:35:38.123120,{},Trench 3,android
1,1599395,7bffb0ec-ac37-44e5-8ee3-2e949b6e3742,ec2342ed-0d8a-432e-899d-05309e565dd7,0.543521,2025-12-13T11:35:38.124118,2025-12-13T11:35:38.124118,Alpha-Cash-CIC-Model,v1,"{""max_age_all_contracts_snapshot"": 931.0, ""rat...",Cash September 25 Models,2bae312d-d33a-444b-b631-188aeee5636c,2025-12-13T11:35:38.124118,{},Trench 3,android
2,2151440,0d97a16c-915b-4d37-ada5-a67052ff5a50,6a799414-f5ca-4d84-8f82-74421cb6a463,0.490993,2025-12-13T11:35:38.124118,2025-12-13T11:35:38.124118,Alpha-Cash-CIC-Model,v1,"{""max_age_all_contracts_snapshot"": 1803.0, ""ra...",Cash September 25 Models,13ba860e-ecc0-404b-ba8d-6cf09d2d6b5a,2025-12-13T11:35:38.124118,{},Trench 3,ios
3,1587934,7ac9c876-d7b6-4cd2-b18e-09500475c236,fdc0795e-7b99-4782-86b3-e111ec1bedcb,0.536526,2025-12-13T11:35:38.124118,2025-12-13T11:35:38.124118,Alpha-Cash-CIC-Model,v1,"{""max_age_all_contracts_snapshot"": 1203.0, ""ra...",Cash September 25 Models,5170d109-81b5-40c6-a35c-77f36dffaa89,2025-12-13T11:35:38.124118,{},Trench 3,ios
4,2087171,2d75e9bd-d0d2-4269-9e5c-22052d402f8e,e3af6a70-39f7-4ac9-b391-76cd018ab726,0.443535,2025-12-13T11:35:38.125120,2025-12-13T11:35:38.125120,Alpha-Cash-CIC-Model,v1,"{""max_age_all_contracts_snapshot"": 1711.0, ""ra...",Cash September 25 Models,bbc62ee0-4141-4d7c-9667-f17e55c8c171,2025-12-13T11:35:38.125120,{},Trench 3,android


In [50]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=3e6f3b6b-8c9e-40ca-99dc-a3229cc39b69>

##### Beta-Cash-Demo-Model

###### Trench 1

In [51]:
sq = """ 
select 
r.customerId customer_id,
r.digitalLoanAccountId,
r.c_demo_score Beta_Cash_Demo_Score,
r.ln_vas_opted_flag, 
r.ln_self_dec_income, 
r.ln_age,
r.ln_source_funds_new_bin, 
r.ln_loan_level_user_type,
r.ln_industry_new_cat_bin, 
r.ln_marital_status,
r.ln_doc_type_rolled, 
r.ln_education_level,
r.ln_ref2_type, 
r.ln_email_primary_domain, 
r.ln_province_bin,
    case when lower(coalesce(lmt.osversion_v2, lmt.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(lmt.osversion_v2, lmt.osVersion)) like '%os%' then 'ios'
    when lower(lmt.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
from worktable_data_analysis.cash_beta_demo_all_applied_backscored_20241001_20250930 r
left join `risk_credit_mis.loan_master_table` lmt on lmt.digitalLoanAccountId = r.digitalLoanAccountId
where trench_category = 'Trench 1'
;

"""

data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")

Job ID d504ed52-faad-42d9-b75f-e0efddf8777e successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (324441, 16)


In [52]:
feature_column = ['ln_vas_opted_flag',
       'ln_self_dec_income', 'ln_age',
       'ln_source_funds_new_bin', 'ln_loan_level_user_type',
       'ln_industry_new_cat_bin',
       'ln_marital_status',
       'ln_doc_type_rolled',
       'ln_education_level',
       'ln_ref2_type', 'ln_email_primary_domain',
       'ln_province_bin','Beta_Cash_Demo_Score']

dfd = transform_data(data, feature_column, a='Beta_Cash_Demo_Score', modelDisplayName='Beta-Cash-Demo-Model', tc='Trench 1', subscription_name = 'Cash September 25 Models') 
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs
0,3046754,811952cc-898d-4b60-a722-15a79c89e6db,a573a496-b4d3-484f-ae0f-f58051e8feb6,0.405182,2025-12-13T11:35:52.371744,2025-12-13T11:35:52.371744,Beta-Cash-Demo-Model,v1,"{""ln_vas_opted_flag"": ""1"", ""ln_self_dec_income...",Cash September 25 Models,faaa1e5a-5b30-4f4e-b6c5-55767c1e9b03,2025-12-13T11:35:52.371744,{},Trench 1,ios
1,3066437,4ad0413d-f0d4-4d55-9f8c-6ba62e0b5fd8,bd6390ad-ecaa-485b-86ed-ad2999e040ea,0.239467,2025-12-13T11:35:52.372974,2025-12-13T11:35:52.372974,Beta-Cash-Demo-Model,v1,"{""ln_vas_opted_flag"": ""0"", ""ln_self_dec_income...",Cash September 25 Models,4c3ee76d-16ec-43a2-aab6-5c60d8b4ac3b,2025-12-13T11:35:52.372974,{},Trench 1,android
2,3526721,1641164d-91fb-4fc1-a7b6-fbd3fb3bb542,8324eadd-b56a-42a1-a9f5-b947a38b7905,0.376546,2025-12-13T11:35:52.372974,2025-12-13T11:35:52.372974,Beta-Cash-Demo-Model,v1,"{""ln_vas_opted_flag"": ""1"", ""ln_self_dec_income...",Cash September 25 Models,6bea17a3-46ee-44ee-b79d-3ca3c79fe3e5,2025-12-13T11:35:52.372974,{},Trench 1,ios
3,3238938,e812291c-d5d1-48d4-a0d7-41539e40bf67,6906bb0c-5f3d-41f0-96c8-8ebd00c9b457,0.306892,2025-12-13T11:35:52.372974,2025-12-13T11:35:52.372974,Beta-Cash-Demo-Model,v1,"{""ln_vas_opted_flag"": ""1"", ""ln_self_dec_income...",Cash September 25 Models,d8bc4449-585b-4628-ab31-e6bd7d0e7e9d,2025-12-13T11:35:52.372974,{},Trench 1,ios
4,3042966,4212b7f4-4c69-40ef-b0a8-4a534f67d3bd,eb585988-5e2b-4f8e-b2e4-1cf56d090ed2,0.535914,2025-12-13T11:35:52.372974,2025-12-13T11:35:52.372974,Beta-Cash-Demo-Model,v1,"{""ln_vas_opted_flag"": ""1"", ""ln_self_dec_income...",Cash September 25 Models,ec197e21-fc19-44ce-adb5-c023c9ba450c,2025-12-13T11:35:52.372974,{},Trench 1,android


In [53]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=869a120d-bd6e-45ab-bc40-316510e4dcb5>

###### Trench 2

In [54]:
sq = """ 
select 
r.customerId customer_id,
r.digitalLoanAccountId,
r.c_demo_score Beta_Cash_Demo_Score,
r.ln_vas_opted_flag, 
r.ln_self_dec_income, 
r.ln_age,
r.ln_source_funds_new_bin, 
r.ln_loan_level_user_type,
r.ln_industry_new_cat_bin, 
r.ln_marital_status,
r.ln_doc_type_rolled, 
r.ln_education_level,
r.ln_ref2_type, 
r.ln_email_primary_domain, 
r.ln_province_bin,
    case when lower(coalesce(lmt.osversion_v2, lmt.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(lmt.osversion_v2, lmt.osVersion)) like '%os%' then 'ios'
    when lower(lmt.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
from worktable_data_analysis.cash_beta_demo_all_applied_backscored_20241001_20250930 r
left join `risk_credit_mis.loan_master_table` lmt on lmt.digitalLoanAccountId = r.digitalLoanAccountId
where trench_category = 'Trench 2'
;

"""

data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")

Job ID 79731e39-89a5-4ad0-9a34-5f6d6e78a74d successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (124539, 16)


In [55]:
feature_column = ['ln_vas_opted_flag',
       'ln_self_dec_income', 'ln_age',
       'ln_source_funds_new_bin', 'ln_loan_level_user_type',
       'ln_industry_new_cat_bin',
       'ln_marital_status',
       'ln_doc_type_rolled',
       'ln_education_level',
       'ln_ref2_type', 'ln_email_primary_domain',
       'ln_province_bin','Beta_Cash_Demo_Score']

dfd = transform_data(data, feature_column, a='Beta_Cash_Demo_Score', modelDisplayName='Beta-Cash-Demo-Model', tc='Trench 2', subscription_name = 'Cash September 25 Models') 
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs
0,3236619,2a442bd0-3901-45d6-8e2d-b569b5b197c6,7f5f13b6-4fa8-40cb-a47a-39b7b9b21abb,0.597785,2025-12-13T11:37:06.265537,2025-12-13T11:37:06.265537,Beta-Cash-Demo-Model,v1,"{""ln_vas_opted_flag"": ""1"", ""ln_self_dec_income...",Cash September 25 Models,72df6343-7026-4ace-83d3-59627d6ee9d5,2025-12-13T11:37:06.265537,{},Trench 2,ios
1,2551613,a0dabc8d-991d-4ce5-a096-996ea273464e,ba933e2e-0789-49c1-9ebb-94fcb7590771,0.363883,2025-12-13T11:37:06.265537,2025-12-13T11:37:06.265537,Beta-Cash-Demo-Model,v1,"{""ln_vas_opted_flag"": ""0"", ""ln_self_dec_income...",Cash September 25 Models,6aa54260-1dd0-475d-bd43-13fa45c611a2,2025-12-13T11:37:06.265537,{},Trench 2,android
2,2615974,fefd16c5-939b-484c-92d7-7711529c5734,e1de6f57-1bec-4d9d-8418-2f78502b5df8,0.343075,2025-12-13T11:37:06.265537,2025-12-13T11:37:06.265537,Beta-Cash-Demo-Model,v1,"{""ln_vas_opted_flag"": ""0"", ""ln_self_dec_income...",Cash September 25 Models,daf41eb5-e0bf-4e9d-959a-8587a17d3651,2025-12-13T11:37:06.265537,{},Trench 2,android
3,2593438,aa708450-0e76-4551-b676-60439c08ef3f,1116e8b5-9f65-4df8-b6d2-359748cf4027,0.420591,2025-12-13T11:37:06.265537,2025-12-13T11:37:06.265537,Beta-Cash-Demo-Model,v1,"{""ln_vas_opted_flag"": ""1"", ""ln_self_dec_income...",Cash September 25 Models,aa18c01a-378d-4897-bf6a-9705ee0c1f4f,2025-12-13T11:37:06.265537,{},Trench 2,ios
4,2604641,30f735c2-e6bd-4016-a6ec-14fda81b132c,ac018e2f-9443-4791-928f-b398b5e632b9,0.486882,2025-12-13T11:37:06.265537,2025-12-13T11:37:06.265537,Beta-Cash-Demo-Model,v1,"{""ln_vas_opted_flag"": ""1"", ""ln_self_dec_income...",Cash September 25 Models,626b0ccc-bb86-4726-a1cc-0cf7689d0811,2025-12-13T11:37:06.265537,{},Trench 2,android


In [56]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=910ae9ce-48fc-4d4b-ada1-092c4bf630fb>

###### Trench 3

In [57]:
sq = """ 
select 
r.customerId customer_id,
r.digitalLoanAccountId,
r.c_demo_score Beta_Cash_Demo_Score,
r.ln_vas_opted_flag, 
r.ln_self_dec_income, 
r.ln_age,
r.ln_source_funds_new_bin, 
r.ln_loan_level_user_type,
r.ln_industry_new_cat_bin, 
r.ln_marital_status,
r.ln_doc_type_rolled, 
r.ln_education_level,
r.ln_ref2_type, 
r.ln_email_primary_domain, 
r.ln_province_bin,
    case when lower(coalesce(lmt.osversion_v2, lmt.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(lmt.osversion_v2, lmt.osVersion)) like '%os%' then 'ios'
    when lower(lmt.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
from worktable_data_analysis.cash_beta_demo_all_applied_backscored_20241001_20250930 r
left join `risk_credit_mis.loan_master_table` lmt on lmt.digitalLoanAccountId = r.digitalLoanAccountId
where trench_category = 'Trench 3'
;

"""

data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")

Job ID 71f57b9a-e047-4d9d-8b10-7d0286e2858c successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (42441, 16)


In [58]:
feature_column = ['ln_vas_opted_flag',
       'ln_self_dec_income', 'ln_age',
       'ln_source_funds_new_bin', 'ln_loan_level_user_type',
       'ln_industry_new_cat_bin',
       'ln_marital_status',
       'ln_doc_type_rolled',
       'ln_education_level',
       'ln_ref2_type', 'ln_email_primary_domain',
       'ln_province_bin','Beta_Cash_Demo_Score']

dfd = transform_data(data, feature_column, a='Beta_Cash_Demo_Score', modelDisplayName='Beta-Cash-Demo-Model', tc='Trench 3', subscription_name = 'Cash September 25 Models') 
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs
0,2802143,5b5791dc-8118-4492-b96f-72b2ee29198e,f33ae697-80f2-40ba-8443-6df24e355bf5,0.270333,2025-12-13T11:37:34.821200,2025-12-13T11:37:34.821200,Beta-Cash-Demo-Model,v1,"{""ln_vas_opted_flag"": ""0"", ""ln_self_dec_income...",Cash September 25 Models,0413b2d4-2469-4d2e-9889-c971af4e8106,2025-12-13T11:37:34.821200,{},Trench 3,ios
1,2901731,9c51f67b-cd70-4ca4-81fd-7acd9d03264b,de5dab3b-52d1-4cf1-a761-a9908e4d5f6d,0.336419,2025-12-13T11:37:34.821200,2025-12-13T11:37:34.821200,Beta-Cash-Demo-Model,v1,"{""ln_vas_opted_flag"": ""0"", ""ln_self_dec_income...",Cash September 25 Models,70179d60-ff5a-4daf-bc33-c6a71fa71660,2025-12-13T11:37:34.821200,{},Trench 3,android
2,2808659,bc39aec4-2bda-4fe0-b2ef-77fcde942f6c,5e8f4549-07c3-4b80-8bc7-5ce8304ba19c,0.329838,2025-12-13T11:37:34.821200,2025-12-13T11:37:34.821200,Beta-Cash-Demo-Model,v1,"{""ln_vas_opted_flag"": ""1"", ""ln_self_dec_income...",Cash September 25 Models,995c8524-93cb-4d2b-8f89-2979052172fb,2025-12-13T11:37:34.821200,{},Trench 3,android
3,2989109,f89f36f6-08b5-4afc-8a58-1ce06da1d335,8cd31c3c-8260-47f9-bb3d-47c53e908e66,0.343360,2025-12-13T11:37:34.822303,2025-12-13T11:37:34.822303,Beta-Cash-Demo-Model,v1,"{""ln_vas_opted_flag"": ""1"", ""ln_self_dec_income...",Cash September 25 Models,aa2a8e46-8e81-41ea-94b4-4f124b0dbb8d,2025-12-13T11:37:34.822303,{},Trench 3,android
4,1822399,176991d3-78e0-4714-982b-f638e29f2d48,833844bd-2bc0-461f-acd6-c4454a0e8df0,0.274429,2025-12-13T11:37:34.822303,2025-12-13T11:37:34.822303,Beta-Cash-Demo-Model,v1,"{""ln_vas_opted_flag"": ""1"", ""ln_self_dec_income...",Cash September 25 Models,0785cf86-aeb0-4b73-9ca3-e8373ec61825,2025-12-13T11:37:34.822303,{},Trench 3,ios


In [59]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=0a293e97-e4dc-45c5-8a97-f662ba61f44b>

##### Beta-Cash-Stack-Model

In [60]:
sq = """ 
select r.customer_id,
r.digitalLoanAccountId,
r.demo_score,
r.apps_score,
r.credo_score,
r.stack_score Beta_cash_stack_score,
r.stack_score_norm,
    case when lower(coalesce(lmt.osversion_v2, lmt.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(lmt.osversion_v2, lmt.osVersion)) like '%os%' then 'ios'
    when lower(lmt.deviceType) like '%andro%' then 'android'
    else 'ios' end osType
 from worktable_data_analysis.cash_beta_trench1_applied_loans_backscored_20241001_20250930 r
 left join `risk_credit_mis.loan_master_table` lmt on lmt.digitalLoanAccountId = r.digitalLoanAccountId
"""
data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")

Job ID 01a27d32-c57d-472b-b889-ea9a0f2393e4 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (324344, 8)


In [61]:
feature_column = ['demo_score',
       'apps_score', 'credo_score',
       'stack_score', 'stack_score_norm']

dfd = transform_data(data, feature_column, a='Beta_cash_stack_score', modelDisplayName='Beta-Cash-Stack-Model', tc='Trench 1', subscription_name = 'Cash September 25 Models') 
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs
0,3285203,236c9f9b-ff89-422f-a0cb-bcdbea4246a2,0dd86f37-b4e6-4a83-8934-95b2191f27d1,0.772580,2025-12-13T11:37:52.702602,2025-12-13T11:37:52.702602,Beta-Cash-Stack-Model,v1,"{""demo_score"": 0.361722674477409, ""apps_score""...",Cash September 25 Models,3c67f7cc-7936-4554-9363-06437f9b7d8f,2025-12-13T11:37:52.702602,{},Trench 1,android
1,3207375,28352167-2686-456e-86c9-b2a8f358f8bd,02c2980f-1c09-474a-9e17-176ec148ac3b,0.486115,2025-12-13T11:37:52.702602,2025-12-13T11:37:52.702602,Beta-Cash-Stack-Model,v1,"{""demo_score"": 0.4588918958078889, ""apps_score...",Cash September 25 Models,5ddff940-20b1-4a65-b314-465557a864eb,2025-12-13T11:37:52.702602,{},Trench 1,android
2,3319605,159a25a9-69be-47ba-bcfe-501677acb883,901f3b60-2679-4b9c-81be-a2e388de3959,0.396408,2025-12-13T11:37:52.702602,2025-12-13T11:37:52.702602,Beta-Cash-Stack-Model,v1,"{""demo_score"": 0.5158387578848971, ""apps_score...",Cash September 25 Models,4356598f-3d63-47ae-bdca-52b4ac36584b,2025-12-13T11:37:52.702602,{},Trench 1,android
3,3627968,8074f044-16c9-4fe0-8d45-9258cc640bd2,9c12dcbe-1e95-4025-a5fc-27440a944395,0.312250,2025-12-13T11:37:52.702602,2025-12-13T11:37:52.702602,Beta-Cash-Stack-Model,v1,"{""demo_score"": 0.302756645045552, ""apps_score""...",Cash September 25 Models,1178fa32-cecb-440d-8036-acc411cfb01c,2025-12-13T11:37:52.702602,{},Trench 1,android
4,2981169,13c6fea0-6c91-481f-9a7b-3799d0807138,72155a50-1c9d-4121-9131-dcae51b5d595,0.951329,2025-12-13T11:37:52.702602,2025-12-13T11:37:52.702602,Beta-Cash-Stack-Model,v1,"{""demo_score"": 0.6061958009468499, ""apps_score...",Cash September 25 Models,cb6b36b9-5d34-42e5-93ab-9f1394b4be0a,2025-12-13T11:37:52.702602,{},Trench 1,android


In [62]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=b4b219fc-2075-4834-977c-0ca46c9c8e55>

###### Trench 2

In [63]:
sq = """ 
select r.customer_id,
r.digitalLoanAccountId,
r.demo_score,
r.apps_score,
r.credo_score,
r.stack_score Beta_cash_stack_score,
r.stack_score_norm,
    case when lower(coalesce(lmt.osversion_v2, lmt.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(lmt.osversion_v2, lmt.osVersion)) like '%os%' then 'ios'
    when lower(lmt.deviceType) like '%andro%' then 'android'
    else 'ios' end osType
 from worktable_data_analysis.cash_beta_trench2_applied_loans_backscored_20241001_20250930 r
 left join `risk_credit_mis.loan_master_table` lmt on lmt.digitalLoanAccountId = r.digitalLoanAccountId
"""
data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")

Job ID 4c7f76ae-0d68-4b1d-9286-aee317250c73 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (121682, 8)


In [64]:
feature_column = ['demo_score',
       'apps_score', 'credo_score',
       'stack_score', 'stack_score_norm']

dfd = transform_data(data, feature_column, a='Beta_cash_stack_score', modelDisplayName='Beta-Cash-Stack-Model', tc='Trench 2', subscription_name = 'Cash September 25 Models') 
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs
0,2054720,f913c755-f382-45f2-b4fd-b9363190243f,57b2ad2f-9015-4674-ad10-848246ab6a40,0.215816,2025-12-13T11:38:36.021342,2025-12-13T11:38:36.021342,Beta-Cash-Stack-Model,v1,"{""demo_score"": 0.640104698063176, ""apps_score""...",Cash September 25 Models,b1bd0ad2-a04d-41eb-9b8a-52fbd6e01702,2025-12-13T11:38:36.021342,{},Trench 2,android
1,1174234,55a74941-0529-449c-ac5b-4ef2529a193b,b78d8c3a-2ab9-4ecc-8dc9-1459ac9d8017,0.153348,2025-12-13T11:38:36.021342,2025-12-13T11:38:36.021342,Beta-Cash-Stack-Model,v1,"{""demo_score"": 0.6483502793704244, ""apps_score...",Cash September 25 Models,fccdd6e5-518b-4182-9f27-18966294f075,2025-12-13T11:38:36.021342,{},Trench 2,android
2,2120167,cdf3ff9e-f646-49fa-8846-410587308b66,a50289b6-89cd-461b-b12d-13dacded05e2,0.452166,2025-12-13T11:38:36.021342,2025-12-13T11:38:36.021342,Beta-Cash-Stack-Model,v1,"{""demo_score"": 0.5867201013917412, ""apps_score...",Cash September 25 Models,e0115dac-71ff-415b-9e8d-a1e7702c2b05,2025-12-13T11:38:36.021342,{},Trench 2,android
3,3100477,29da8000-1606-4ecf-a05a-2df838bf2ea5,d1d4f8f2-250e-4e5d-89a6-c948e9e5f18d,0.635149,2025-12-13T11:38:36.021342,2025-12-13T11:38:36.021342,Beta-Cash-Stack-Model,v1,"{""demo_score"": 0.5692949500211927, ""apps_score...",Cash September 25 Models,56d74c7f-363e-4c15-9613-af5c3b8d4216,2025-12-13T11:38:36.021342,{},Trench 2,android
4,2795880,39e5b17d-cb31-4987-add7-952da9cb2721,0252dcf0-04e5-402a-913a-a5dd69a272a7,0.147858,2025-12-13T11:38:36.021342,2025-12-13T11:38:36.021342,Beta-Cash-Stack-Model,v1,"{""demo_score"": 0.5441454542981201, ""apps_score...",Cash September 25 Models,930cf11c-2b9c-4130-9dd1-7d4e6744848f,2025-12-13T11:38:36.021342,{},Trench 2,android


In [65]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=17fc5063-d3f2-4bc2-8538-6d464d237f3d>

###### Trench 3

In [66]:
sq = """ 
select r.customer_id,
r.digitalLoanAccountId,
r.demo_score,
r.apps_score,
r.credo_score,
r.stack_score Beta_cash_stack_score,
r.stack_score_norm,
    case when lower(coalesce(lmt.osversion_v2, lmt.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(lmt.osversion_v2, lmt.osVersion)) like '%os%' then 'ios'
    when lower(lmt.deviceType) like '%andro%' then 'android'
    else 'ios' end osType
 from worktable_data_analysis.cash_beta_trench3_applied_loans_backscored_20241001_20250930 r
 left join `risk_credit_mis.loan_master_table` lmt on lmt.digitalLoanAccountId = r.digitalLoanAccountId;

"""
data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")


Job ID c734980a-4a33-4b3e-974d-b969aeb42969 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (42414, 8)


In [67]:
feature_column = ['demo_score',
       'apps_score', 'credo_score',
       'stack_score', 'stack_score_norm']

dfd = transform_data(data, feature_column, a='Beta_cash_stack_score', modelDisplayName='Beta-Cash-Stack-Model', tc='Trench 3', subscription_name = 'Cash September 25 Models') 
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs
0,2930414,ed57fca3-92f2-4a63-b371-5c8fedced0f1,d2a3254e-4e34-407d-8eda-ec3c0f4f2f1b,0.582014,2025-12-13T11:38:58.098481,2025-12-13T11:38:58.098481,Beta-Cash-Stack-Model,v1,"{""demo_score"": 0.3534398469902259, ""apps_score...",Cash September 25 Models,33e44d7c-5d04-4f00-80e5-cee6b535989d,2025-12-13T11:38:58.098481,{},Trench 3,android
1,3133620,1257eadd-7f55-41a1-91a2-f384ce6cd83d,595e91f7-c479-43da-9407-0a061f890674,0.503586,2025-12-13T11:38:58.098481,2025-12-13T11:38:58.098481,Beta-Cash-Stack-Model,v1,"{""demo_score"": 0.5449882433807196, ""apps_score...",Cash September 25 Models,2f902736-ba04-4c29-92ce-cd2ebb0df0bb,2025-12-13T11:38:58.098481,{},Trench 3,android
2,1860665,54957191-79e4-4889-870e-116992d7697d,9061a1a2-ce79-42ef-b31a-6888842a2f89,0.307870,2025-12-13T11:38:58.099479,2025-12-13T11:38:58.099479,Beta-Cash-Stack-Model,v1,"{""demo_score"": 0.372657506730734, ""apps_score""...",Cash September 25 Models,af2bc1d7-d7b0-4993-971f-a9cc90723b21,2025-12-13T11:38:58.099479,{},Trench 3,android
3,1237037,edb6b1a9-a613-4ffa-9391-1c30f18e2841,3a853d09-acf2-4bb0-8c75-895b98c7052c,0.294218,2025-12-13T11:38:58.099479,2025-12-13T11:38:58.099479,Beta-Cash-Stack-Model,v1,"{""demo_score"": 0.3282397117831538, ""apps_score...",Cash September 25 Models,46435696-fb26-4b89-86a5-bc4ffc405c93,2025-12-13T11:38:58.099479,{},Trench 3,android
4,2381621,4e2735c7-6c1d-43f9-8fa4-8328bfcf5a81,4a8eedea-f422-466a-add3-e41edf8d0350,0.683741,2025-12-13T11:38:58.099479,2025-12-13T11:38:58.099479,Beta-Cash-Stack-Model,v1,"{""demo_score"": 0.42109124689211064, ""apps_scor...",Cash September 25 Models,0bd21f4f-4fd7-46c6-af1c-bbd1b82595ad,2025-12-13T11:38:58.099479,{},Trench 3,android


In [68]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=831a31c9-5b6d-4aa3-be7e-2e79d6fa3670>

#####  Beta Cash AppScore Model

###### Trench 1

In [69]:
## query to add the trench category trench 1 and trench 2

# create or replace table risk_mart.applied_quick_loan_new_applicants_20230101_20250930_app_scored_copy as 
# select * from risk_mart.applied_quick_loan_new_applicants_20230101_20250930_app_scored;



# ALTER TABLE risk_mart.applied_quick_loan_new_applicants_20230101_20250930_app_scored_copy
# ADD COLUMN trench_category STRING;

# MERGE risk_mart.applied_quick_loan_new_applicants_20230101_20250930_app_scored_copy AS target
# USING (
#   SELECT 
#     a.digitalLoanAccountId,
#     CASE 
#       WHEN b.ln_loan_level_user_type = '2_New Applicant'
#            AND DATE_DIFF(DATE(b.ln_appln_submit_datetime), DATE(b.onb_tsa_onboarding_datetime), DAY) <= 30 THEN 'Trench 1'
#       WHEN b.ln_loan_level_user_type = '2_New Applicant'
#            AND DATE_DIFF(DATE(b.ln_appln_submit_datetime), DATE(b.onb_tsa_onboarding_datetime), DAY) > 30 THEN 'Trench 2'
#       WHEN b.ln_loan_level_user_type = '1_Repeat Applicant' THEN 'Trench 3'
#       ELSE NULL
#     END AS trench_category,
#     ROW_NUMBER() OVER (PARTITION BY a.digitalLoanAccountId ORDER BY b.ln_appln_submit_datetime DESC) AS row_num
#   FROM risk_mart.applied_quick_loan_new_applicants_20230101_20250930_app_scored_copy a
#   LEFT JOIN prj-prod-dataplatform.risk_mart.applied_loans_20210701_20250930_trans b
#     ON a.digitalLoanAccountId = b.digitalLoanAccountId
# ) AS source
# ON target.digitalLoanAccountId = source.digitalLoanAccountId
# WHEN MATCHED AND source.row_num = 1 THEN
#   UPDATE SET target.trench_category = source.trench_category;

In [70]:
sq = """ 
select 
  r.customerId customer_id,
  r.digitalLoanAccountId,
  r.apps_score beta_cash_app_score,  
  r.app_cnt_health_and_fitness_ever app_cnt_health_and_fitness_ever, 
  r.app_cnt_shopping_ever app_cnt_shopping_ever,
  r.app_cnt_crypto_ever app_cnt_crypto_ever, 
  r.app_cnt_driver_ever app_cnt_driver_ever,
  r.app_cnt_payday_180d app_cnt_payday_180d, 
  r.app_cnt_gambling_180d app_cnt_gambling_180d,
  r.app_avg_time_bw_installed_mins_3d app_avg_time_bw_installed_mins_3d,
  r.app_median_time_bw_installed_mins_ever app_median_time_bw_installed_mins_3d,
    case when lower(coalesce(lmt.osversion_v2, lmt.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(lmt.osversion_v2, lmt.osVersion)) like '%os%' then 'ios'
    when lower(lmt.deviceType) like '%andro%' then 'android'
    else 'ios' end osType
from risk_mart.applied_quick_loan_new_applicants_20230101_20250930_app_scored_copy r 
left join `risk_credit_mis.loan_master_table` lmt on lmt.digitalLoanAccountId = r.digitalLoanAccountId
where r.trench_category = 'Trench 1';
"""
data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")

Job ID ca5f5710-82a3-47b6-934e-3b749d773756 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (386694, 12)


In [71]:
data.columns

Index(['customer_id', 'digitalLoanAccountId', 'beta_cash_app_score',
       'app_cnt_health_and_fitness_ever', 'app_cnt_shopping_ever',
       'app_cnt_crypto_ever', 'app_cnt_driver_ever', 'app_cnt_payday_180d',
       'app_cnt_gambling_180d', 'app_avg_time_bw_installed_mins_3d',
       'app_median_time_bw_installed_mins_3d', 'osType'],
      dtype='object')

In [72]:
feature_column = ['beta_cash_app_score',
       'app_cnt_health_and_fitness_ever', 'app_cnt_shopping_ever',
       'app_cnt_crypto_ever', 'app_cnt_driver_ever', 'app_cnt_payday_180d',
       'app_cnt_gambling_180d', 'app_avg_time_bw_installed_mins_3d',
       'app_median_time_bw_installed_mins_3d']

dfd = transform_data(data, feature_column, a='beta_cash_app_score', modelDisplayName='Beta-Cash-AppScore-Model', tc='Trench 1', subscription_name = 'Cash September 25 Models') 
dfd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 386694 entries, 0 to 386693
Data columns (total 15 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   customerId            386694 non-null  int64  
 1   digitalLoanAccountId  386694 non-null  object 
 2   crifApplicationId     386694 non-null  object 
 3   prediction            386694 non-null  float64
 4   start_time            386694 non-null  object 
 5   end_time              386694 non-null  object 
 6   modelDisplayName      386694 non-null  object 
 7   modelVersionId        386694 non-null  object 
 8   calcFeature           386694 non-null  object 
 9   subscription_name     386694 non-null  object 
 10  message_id            386694 non-null  object 
 11  publish_time          386694 non-null  object 
 12  attributes            386694 non-null  object 
 13  trenchCategory        386694 non-null  object 
 14  deviceOs              386694 non-null  object 
dtype

In [73]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=36821706-038c-4bf5-af0d-552dbb6c73a9>

###### Trench 2

In [74]:
sq = """ 
select 
  r.customerId customer_id,
  r.digitalLoanAccountId,
  r.apps_score beta_cash_app_score,  
  r.app_cnt_health_and_fitness_ever app_cnt_health_and_fitness_ever, 
  r.app_cnt_shopping_ever app_cnt_shopping_ever,
  r.app_cnt_crypto_ever app_cnt_crypto_ever, 
  r.app_cnt_driver_ever app_cnt_driver_ever,
  r.app_cnt_payday_180d app_cnt_payday_180d, 
  r.app_cnt_gambling_180d app_cnt_gambling_180d,
  r.app_avg_time_bw_installed_mins_3d app_avg_time_bw_installed_mins_3d,
  r.app_median_time_bw_installed_mins_ever app_median_time_bw_installed_mins_3d,
    case when lower(coalesce(lmt.osversion_v2, lmt.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(lmt.osversion_v2, lmt.osVersion)) like '%os%' then 'ios'
    when lower(lmt.deviceType) like '%andro%' then 'android'
    else 'ios' end osType
from risk_mart.applied_quick_loan_new_applicants_20230101_20250930_app_scored_copy r 
left join `risk_credit_mis.loan_master_table` lmt on lmt.digitalLoanAccountId = r.digitalLoanAccountId
where r.trench_category = 'Trench 2';
"""
data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")

Job ID 18bb4e46-a749-4ee0-9dea-dd0ffe58d120 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (122895, 12)


In [75]:
feature_column = ['beta_cash_app_score',
       'app_cnt_health_and_fitness_ever', 'app_cnt_shopping_ever',
       'app_cnt_crypto_ever', 'app_cnt_driver_ever', 'app_cnt_payday_180d',
       'app_cnt_gambling_180d', 'app_avg_time_bw_installed_mins_3d',
       'app_median_time_bw_installed_mins_3d']

dfd = transform_data(data, feature_column, a='beta_cash_app_score', modelDisplayName='Beta-Cash-AppScore-Model', tc='Trench 2', subscription_name = 'Cash September 25 Models') 
dfd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 122895 entries, 0 to 122894
Data columns (total 15 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   customerId            122895 non-null  int64  
 1   digitalLoanAccountId  122895 non-null  object 
 2   crifApplicationId     122895 non-null  object 
 3   prediction            122895 non-null  float64
 4   start_time            122895 non-null  object 
 5   end_time              122895 non-null  object 
 6   modelDisplayName      122895 non-null  object 
 7   modelVersionId        122895 non-null  object 
 8   calcFeature           122895 non-null  object 
 9   subscription_name     122895 non-null  object 
 10  message_id            122895 non-null  object 
 11  publish_time          122895 non-null  object 
 12  attributes            122895 non-null  object 
 13  trenchCategory        122895 non-null  object 
 14  deviceOs              122895 non-null  object 
dtype

In [76]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=628950a4-1139-46c1-b1de-b47fd4145c2b>

###### Trench 3

In [77]:
sq = """   
 select 
  r.customerId customer_id, 
  r.digitalLoanAccountId,
  r.apps_score beta_cash_app_score,
  r.app_cnt_health_and_fitness_ever_binned app_cnt_health_and_fitness_ever,
  r.app_cnt_productivity_ever_binned app_cnt_productivity_ever, 
  r.app_cnt_rated_for_18plus_ever_binned app_cnt_rated_for_18plus_ever,
  r.app_cnt_books_and_reference_ever_binned app_cnt_books_and_reference_ever, 
  r.app_cnt_gaming_180d_binned app_cnt_gaming_180d,
  r.app_cnt_absence_tag_365d_binned app_cnt_absence_tag_365d,
  r.app_last_payday_install_to_apply_days_binned app_last_payday_install_to_apply_days,
  r.app_cnt_absence_tag_365d_binned,
  r.app_cnt_gaming_180d_binned,
  r.app_cnt_productivity_ever_binned,
  r.app_cnt_rated_for_18plus_ever_binned,
  r.app_cnt_health_and_fitness_ever_binned,
  r.app_cnt_books_and_reference_ever_binned,
  r.app_last_payday_install_to_apply_days_binned,
  r.ln_user_type,
  case when lower(coalesce(lmt.osversion_v2, lmt.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(lmt.osversion_v2, lmt.osVersion)) like '%os%' then 'ios'
    when lower(lmt.deviceType) like '%andro%' then 'android'
    else 'ios' end osType
from risk_mart.b_score_model_applied_loans_cash_20240101_20250930_app_scored r
left join `risk_credit_mis.loan_master_table` lmt on lmt.digitalLoanAccountId = r.digitalLoanAccountId

"""
data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")

Job ID a6202b4c-5d1f-45ed-b417-5710b4112ebb successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (67403, 19)


In [78]:
feature_column = ['beta_cash_app_score',
       'app_cnt_health_and_fitness_ever', 'app_cnt_productivity_ever',
       'app_cnt_rated_for_18plus_ever', 'app_cnt_books_and_reference_ever',
       'app_cnt_gaming_180d', 'app_cnt_absence_tag_365d',
       'app_last_payday_install_to_apply_days',
       'app_cnt_absence_tag_365d_binned', 'app_cnt_gaming_180d_binned',
       'app_cnt_productivity_ever_binned',
       'app_cnt_rated_for_18plus_ever_binned',
       'app_cnt_health_and_fitness_ever_binned',
       'app_cnt_books_and_reference_ever_binned',
       'app_last_payday_install_to_apply_days_binned', 'ln_user_type']

dfd = transform_data(data, feature_column, a='beta_cash_app_score', modelDisplayName='Beta-Cash-AppScore-Model', tc='Trench 3', subscription_name = 'Cash September 25 Models') 
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs
0,3141515,845a6c28-27a2-41cf-bcc0-d5d85ef30781,85a19877-f2ee-422f-918c-a8794de154ce,0.526536,2025-12-13T11:40:42.758220,2025-12-13T11:40:42.758220,Beta-Cash-AppScore-Model,v1,"{""beta_cash_app_score"": 0.526536277198278, ""ap...",Cash September 25 Models,31e0379e-91c1-4593-b52e-5a178bfe545b,2025-12-13T11:40:42.758220,{},Trench 3,android
1,2486450,c4c7a3d6-f01e-497b-a6c4-1d2bc40e168e,dee32052-da67-40ef-a111-0ae6885f95cb,0.472776,2025-12-13T11:40:42.758220,2025-12-13T11:40:42.758220,Beta-Cash-AppScore-Model,v1,"{""beta_cash_app_score"": 0.4727757799276481, ""a...",Cash September 25 Models,08314215-61a1-48c2-9374-2c9291983c73,2025-12-13T11:40:42.758220,{},Trench 3,android
2,2608108,a5b6a13a-cd33-4ea4-9b5d-5073a3d15037,41524d97-a23e-4c2c-ae07-0d4f203569b4,0.520440,2025-12-13T11:40:42.758220,2025-12-13T11:40:42.758220,Beta-Cash-AppScore-Model,v1,"{""beta_cash_app_score"": 0.5204395696069121, ""a...",Cash September 25 Models,d81ed316-6b06-4f9b-9509-272e240eae53,2025-12-13T11:40:42.758220,{},Trench 3,android
3,2566081,72f46190-0cca-4e80-92f5-48d33a6bffc4,2f59fb20-f603-484d-82b1-ec08c971e91f,0.537022,2025-12-13T11:40:42.759219,2025-12-13T11:40:42.759219,Beta-Cash-AppScore-Model,v1,"{""beta_cash_app_score"": 0.537022030841087, ""ap...",Cash September 25 Models,32967289-795a-49fc-a15b-f2a6119515a9,2025-12-13T11:40:42.759219,{},Trench 3,android
4,2255228,213f69b0-7b7b-4509-99a8-b3d4d2a0eeb8,3877f32c-2b9d-4a51-9a96-2ecfc943bc17,0.545455,2025-12-13T11:40:42.759219,2025-12-13T11:40:42.759219,Beta-Cash-AppScore-Model,v1,"{""beta_cash_app_score"": 0.5454553318135508, ""a...",Cash September 25 Models,f3588453-59d4-4b12-8286-50b11e926ecf,2025-12-13T11:40:42.759219,{},Trench 3,android


In [79]:
dfd.info()
dfd['customerId'] = pd.to_numeric(dfd['customerId'], errors='coerce')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67403 entries, 0 to 67402
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   customerId            67403 non-null  object 
 1   digitalLoanAccountId  67403 non-null  object 
 2   crifApplicationId     67403 non-null  object 
 3   prediction            67403 non-null  float64
 4   start_time            67403 non-null  object 
 5   end_time              67403 non-null  object 
 6   modelDisplayName      67403 non-null  object 
 7   modelVersionId        67403 non-null  object 
 8   calcFeature           67403 non-null  object 
 9   subscription_name     67403 non-null  object 
 10  message_id            67403 non-null  object 
 11  publish_time          67403 non-null  object 
 12  attributes            67403 non-null  object 
 13  trenchCategory        67403 non-null  object 
 14  deviceOs              67403 non-null  object 
dtypes: float64(1), obje

In [80]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=0cb544e8-3c90-4488-b446-10209296519d>

# SIL V2

*   Sil-Alpha-CIC-SIL-Model: worktable_data_analysis.sil_alpha_cic_all_applied_backscored_20240901_20250930
*   Sil-Alpha-StackingModel: worktable_data_analysis.sil_alpha_applied_loans_backscored_20240901_20251013_option3
*   Sil-Beta-AppsScoreModel:
*   New Applicants (T1, T2): risk_mart.applied_sil_trench1_trench2_loan_jan2024_30sep2025_app_scored, risk_mart.applied_sil_new_applicants_loan_oct0125_oct1325_app_scored
*   Repeat Applicants (T3): risk_mart.b_score_model_applied_loans_sil_20240101_20250930_app_scored, risk_mart.applied_sil_repeat_applicants_loan_oct0125_oct1325_app_scored
*   Sil-Beta-DemoScoreModel: worktable_data_analysis.sil_beta_demo_all_applied_backscored_20240801_20251015
*   Sil-Beta-StackScoreModel: worktable_data_analysis.sil_beta_applied_loans_backscored_20240801_20251013_option3m

##### 'Alpha - CIC-SIL-Model'

##### Trench 1

In [81]:
sq = """ 
select 
r.customerId customer_id,
r.digitalLoanAccountId, 
r.c_cic_score ,
r.ScoreRange,
       ln_loan_level_user_type, flg_zero_non_granted_ever,
       flg_zero_granted_ever,
       Personal_Loans_granted_contracts_amt_24M,
       granted_contracts_cnt_6M, total_overdue_granted_contracts,
       has_ever_been_overdue, cnt_nongranted_contracts_3M,
       cnt_active_contracts, max_amt_granted_24M,
       tot_active_contracts_util, days_since_last_closed,
       vel_contract_nongranted_cnt_6on12,
       vel_contract_granted_amt_6on12,
       vel_contract_closed_amt_3on12,
case 
  when lower(r.ln_os_type) like '%andro%' then 'android'
  when lower(r.ln_os_type) like '%os%' then 'ios'
  else 'ios' end  osType
from worktable_data_analysis.sil_alpha_cic_all_applied_backscored_20240901_20250930 r
left join `risk_credit_mis.loan_master_table` lmt on lmt.digitalLoanAccountId = r.digitalLoanAccountId
where r.trench_category = 'Trench 1';
"""
data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")

Job ID 4509aa2f-76e8-4cee-81e0-ad2f6a58a2b1 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (224718, 20)


In [82]:
data.columns

Index(['customer_id', 'digitalLoanAccountId', 'c_cic_score', 'ScoreRange',
       'ln_loan_level_user_type', 'flg_zero_non_granted_ever',
       'flg_zero_granted_ever', 'Personal_Loans_granted_contracts_amt_24M',
       'granted_contracts_cnt_6M', 'total_overdue_granted_contracts',
       'has_ever_been_overdue', 'cnt_nongranted_contracts_3M',
       'cnt_active_contracts', 'max_amt_granted_24M',
       'tot_active_contracts_util', 'days_since_last_closed',
       'vel_contract_nongranted_cnt_6on12', 'vel_contract_granted_amt_6on12',
       'vel_contract_closed_amt_3on12', 'osType'],
      dtype='object')

In [83]:
data.head()

,customer_id,digitalLoanAccountId,c_cic_score,ScoreRange,ln_loan_level_user_type,flg_zero_non_granted_ever,flg_zero_granted_ever,Personal_Loans_granted_contracts_amt_24M,granted_contracts_cnt_6M,total_overdue_granted_contracts,has_ever_been_overdue,cnt_nongranted_contracts_3M,cnt_active_contracts,max_amt_granted_24M,tot_active_contracts_util,days_since_last_closed,vel_contract_nongranted_cnt_6on12,vel_contract_granted_amt_6on12,vel_contract_closed_amt_3on12,osType
0,3109711,63f7b670-62d3-4f47-b239-a9e26a0b8c7a,0.457930,NH_Gi,2_New Applicant,0,0,4889.0,15.0,9.0,1.0,1.0,12.0,12700.0,1.030195,1.0,1.994012,0.890462,0.434634,ios
1,2818114,699b5ad2-f50a-485b-bae5-7fddbda9897d,0.136509,Bi,2_New Applicant,0,0,66414.0,1.0,7.0,1.0,NaN,2.0,28856.0,0.289917,6.0,1.000000,0.294034,4.608014,android
2,3250694,585e799e-0236-45ae-8317-80c198dd4a3a,0.400941,Missing,2_New Applicant,0,0,9962.0,NaN,5.0,1.0,NaN,1.0,9000.0,0.000000,113.0,NaN,NaN,NaN,android
3,3666235,8f99c73c-f5e1-481b-8a78-f627f67e944e,0.570266,Missing,2_New Applicant,0,0,NaN,NaN,5.0,1.0,1.0,1.0,60000.0,1.155800,361.0,2.012048,NaN,NaN,android
4,2874293,2a710daf-9e13-41a1-9129-5c72760666f0,0.364744,NH_Hi,2_New Applicant,1,0,NaN,NaN,5.0,1.0,NaN,NaN,NaN,NaN,811.0,NaN,NaN,NaN,android


In [84]:
feature_column = ['ScoreRange',
       'ln_loan_level_user_type', 'flg_zero_non_granted_ever',
       'flg_zero_granted_ever', 'Personal_Loans_granted_contracts_amt_24M',
       'granted_contracts_cnt_6M', 'total_overdue_granted_contracts',
       'has_ever_been_overdue', 'cnt_nongranted_contracts_3M',
       'cnt_active_contracts', 'max_amt_granted_24M',
       'tot_active_contracts_util', 'days_since_last_closed',
       'vel_contract_nongranted_cnt_6on12', 'vel_contract_granted_amt_6on12',
       'vel_contract_closed_amt_3on12',]

dfd = transform_datav2(data, feature_column, a='c_cic_score', modelDisplayName='cic_model_sil', tc='Trench 1', subscription_name = 'Cash November 25 Models') 
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs
0,3109711,63f7b670-62d3-4f47-b239-a9e26a0b8c7a,47c67aac-3cd5-456d-a770-6a958987e6d5,0.457930,2025-12-13T11:41:04.057359,2025-12-13T11:41:04.057359,cic_model_sil,v2,"{""ScoreRange"": ""NH_Gi"", ""ln_loan_level_user_ty...",Cash November 25 Models,a1e6774a-62a4-4ae3-81f8-69782557f36e,2025-12-13T11:41:04.057359,{},Trench 1,ios
1,2818114,699b5ad2-f50a-485b-bae5-7fddbda9897d,d59fac78-9a7a-4141-b6e0-12db24f1bc6f,0.136509,2025-12-13T11:41:04.057359,2025-12-13T11:41:04.057359,cic_model_sil,v2,"{""ScoreRange"": ""Bi"", ""ln_loan_level_user_type""...",Cash November 25 Models,38530d65-1d38-49ea-a8b3-8f2e0f4b7b7d,2025-12-13T11:41:04.057359,{},Trench 1,android
2,3250694,585e799e-0236-45ae-8317-80c198dd4a3a,83a4a84d-c72d-436d-8135-ca054b2b814f,0.400941,2025-12-13T11:41:04.057359,2025-12-13T11:41:04.057359,cic_model_sil,v2,"{""ScoreRange"": ""Missing"", ""ln_loan_level_user_...",Cash November 25 Models,7711510f-2219-488b-a756-1fe9f1d68651,2025-12-13T11:41:04.057359,{},Trench 1,android
3,3666235,8f99c73c-f5e1-481b-8a78-f627f67e944e,6aacba6e-03fe-414e-b5c8-76f26de973f7,0.570266,2025-12-13T11:41:04.058374,2025-12-13T11:41:04.058374,cic_model_sil,v2,"{""ScoreRange"": ""Missing"", ""ln_loan_level_user_...",Cash November 25 Models,4580aac0-00ec-4101-948d-14aa4fe8ca1e,2025-12-13T11:41:04.058374,{},Trench 1,android
4,2874293,2a710daf-9e13-41a1-9129-5c72760666f0,7596b773-f640-40ac-9666-d58d755005c4,0.364744,2025-12-13T11:41:04.058374,2025-12-13T11:41:04.058374,cic_model_sil,v2,"{""ScoreRange"": ""NH_Hi"", ""ln_loan_level_user_ty...",Cash November 25 Models,40eed691-7a1e-4417-bfa2-d743f203f7b4,2025-12-13T11:41:04.058374,{},Trench 1,android


In [85]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=aa19ae62-2ea2-42f9-8132-60f8059d7790>

##### Trench 2

In [86]:
sq = """ 
select 
r.customerId customer_id,
r.digitalLoanAccountId, 
r.c_cic_score ,
r.ScoreRange,
       ln_loan_level_user_type, flg_zero_non_granted_ever,
       flg_zero_granted_ever,
       Personal_Loans_granted_contracts_amt_24M,
       granted_contracts_cnt_6M, total_overdue_granted_contracts,
       has_ever_been_overdue, cnt_nongranted_contracts_3M,
       cnt_active_contracts, max_amt_granted_24M,
       tot_active_contracts_util, days_since_last_closed,
       vel_contract_nongranted_cnt_6on12,
       vel_contract_granted_amt_6on12,
       vel_contract_closed_amt_3on12,
case 
  when lower(r.ln_os_type) like '%andro%' then 'android'
  when lower(r.ln_os_type) like '%os%' then 'ios'
  else 'ios' end  osType
from worktable_data_analysis.sil_alpha_cic_all_applied_backscored_20240901_20250930 r
left join `risk_credit_mis.loan_master_table` lmt on lmt.digitalLoanAccountId = r.digitalLoanAccountId
where r.trench_category = 'Trench 2';
"""
data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")

Job ID 8e36040b-6b5f-45af-ad12-0b13857f990c successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (10175, 20)


In [87]:
data.columns

Index(['customer_id', 'digitalLoanAccountId', 'c_cic_score', 'ScoreRange',
       'ln_loan_level_user_type', 'flg_zero_non_granted_ever',
       'flg_zero_granted_ever', 'Personal_Loans_granted_contracts_amt_24M',
       'granted_contracts_cnt_6M', 'total_overdue_granted_contracts',
       'has_ever_been_overdue', 'cnt_nongranted_contracts_3M',
       'cnt_active_contracts', 'max_amt_granted_24M',
       'tot_active_contracts_util', 'days_since_last_closed',
       'vel_contract_nongranted_cnt_6on12', 'vel_contract_granted_amt_6on12',
       'vel_contract_closed_amt_3on12', 'osType'],
      dtype='object')

In [88]:
data.head()

,customer_id,digitalLoanAccountId,c_cic_score,ScoreRange,ln_loan_level_user_type,flg_zero_non_granted_ever,flg_zero_granted_ever,Personal_Loans_granted_contracts_amt_24M,granted_contracts_cnt_6M,total_overdue_granted_contracts,has_ever_been_overdue,cnt_nongranted_contracts_3M,cnt_active_contracts,max_amt_granted_24M,tot_active_contracts_util,days_since_last_closed,vel_contract_nongranted_cnt_6on12,vel_contract_granted_amt_6on12,vel_contract_closed_amt_3on12,osType
0,2144008,4d8a8f23-f5e1-4a8a-bade-cce36177c661,0.451918,Missing,2_New Applicant,0,0,11836.0,NaN,12.0,1.0,1.0,1.0,5000.0,NaN,203.0,1.332,NaN,NaN,android
1,3219768,50cea963-0136-4c7c-940a-62447d195779,0.364573,NH_Gi,2_New Applicant,0,0,NaN,NaN,17.0,1.0,1.0,NaN,NaN,NaN,937.0,1.000,NaN,NaN,android
2,2243917,eff43ea2-cf96-4d68-be33-74bf1d3085c6,0.435207,Missing,2_New Applicant,0,0,3000.0,NaN,10.0,1.0,1.0,3.0,6255.0,0.849151,375.0,1.332,NaN,NaN,ios
3,3424030,36bb33ff-2592-44d4-b770-e0240786b53a,0.507689,NH_Ei,2_New Applicant,0,0,NaN,NaN,83.0,1.0,NaN,NaN,NaN,NaN,445.0,NaN,NaN,NaN,android
4,2844270,e955fcc8-77cd-4fb4-86f8-2c8897608ada,0.452476,Missing,2_New Applicant,0,0,2064.0,NaN,18.0,1.0,NaN,NaN,4500.0,NaN,150.0,NaN,NaN,NaN,android


In [89]:
feature_column = ['ScoreRange',
       'ln_loan_level_user_type', 'flg_zero_non_granted_ever',
       'flg_zero_granted_ever', 'Personal_Loans_granted_contracts_amt_24M',
       'granted_contracts_cnt_6M', 'total_overdue_granted_contracts',
       'has_ever_been_overdue', 'cnt_nongranted_contracts_3M',
       'cnt_active_contracts', 'max_amt_granted_24M',
       'tot_active_contracts_util', 'days_since_last_closed',
       'vel_contract_nongranted_cnt_6on12', 'vel_contract_granted_amt_6on12',
       'vel_contract_closed_amt_3on12',]

dfd = transform_datav2(data, feature_column, a='c_cic_score', modelDisplayName='cic_model_sil', tc='Trench 2', subscription_name = 'Cash November 25 Models') 
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs
0,2144008,4d8a8f23-f5e1-4a8a-bade-cce36177c661,b40626f5-2663-48ed-94aa-bb70b1024155,0.451918,2025-12-13T11:41:45.586290,2025-12-13T11:41:45.586290,cic_model_sil,v2,"{""ScoreRange"": ""Missing"", ""ln_loan_level_user_...",Cash November 25 Models,0362ebe6-93bb-4643-83ba-dbd83aef0e2f,2025-12-13T11:41:45.586290,{},Trench 2,android
1,3219768,50cea963-0136-4c7c-940a-62447d195779,3aa79d7c-0c29-417e-9a05-90ab678993f6,0.364573,2025-12-13T11:41:45.587291,2025-12-13T11:41:45.587291,cic_model_sil,v2,"{""ScoreRange"": ""NH_Gi"", ""ln_loan_level_user_ty...",Cash November 25 Models,5ff1c914-24e4-401e-810f-ca7e3fb8f42e,2025-12-13T11:41:45.587291,{},Trench 2,android
2,2243917,eff43ea2-cf96-4d68-be33-74bf1d3085c6,e8585d68-d0ba-4f19-a15d-49302f998d84,0.435207,2025-12-13T11:41:45.587291,2025-12-13T11:41:45.587291,cic_model_sil,v2,"{""ScoreRange"": ""Missing"", ""ln_loan_level_user_...",Cash November 25 Models,0bac1cd1-dc5e-4c63-9f1a-7d7f41074991,2025-12-13T11:41:45.587291,{},Trench 2,ios
3,3424030,36bb33ff-2592-44d4-b770-e0240786b53a,511f4f64-0337-4181-b7cf-d61ad767ee40,0.507689,2025-12-13T11:41:45.587291,2025-12-13T11:41:45.587291,cic_model_sil,v2,"{""ScoreRange"": ""NH_Ei"", ""ln_loan_level_user_ty...",Cash November 25 Models,4de9bc3d-ed98-4708-929e-b884a0028de7,2025-12-13T11:41:45.587291,{},Trench 2,android
4,2844270,e955fcc8-77cd-4fb4-86f8-2c8897608ada,3f127a29-125c-49fe-bc5b-976d60c726ae,0.452476,2025-12-13T11:41:45.588290,2025-12-13T11:41:45.588290,cic_model_sil,v2,"{""ScoreRange"": ""Missing"", ""ln_loan_level_user_...",Cash November 25 Models,0becabd5-2637-4811-b203-de6550b8422c,2025-12-13T11:41:45.588290,{},Trench 2,android


In [90]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=f53eabef-cd54-4f3e-b12d-f1375fee8efa>

##### Trench 3

In [91]:
sq = """ 
select 
r.customerId customer_id,
r.digitalLoanAccountId, 
r.c_cic_score ,
r.ScoreRange,
       ln_loan_level_user_type, flg_zero_non_granted_ever,
       flg_zero_granted_ever,
       Personal_Loans_granted_contracts_amt_24M,
       granted_contracts_cnt_6M, total_overdue_granted_contracts,
       has_ever_been_overdue, cnt_nongranted_contracts_3M,
       cnt_active_contracts, max_amt_granted_24M,
       tot_active_contracts_util, days_since_last_closed,
       vel_contract_nongranted_cnt_6on12,
       vel_contract_granted_amt_6on12,
       vel_contract_closed_amt_3on12,
case 
  when lower(r.ln_os_type) like '%andro%' then 'android'
  when lower(r.ln_os_type) like '%os%' then 'ios'
  else 'ios' end  osType
from worktable_data_analysis.sil_alpha_cic_all_applied_backscored_20240901_20250930 r
left join `risk_credit_mis.loan_master_table` lmt on lmt.digitalLoanAccountId = r.digitalLoanAccountId
where r.trench_category = 'Trench 3';
"""
data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")

Job ID 338b1c47-289e-4c45-8b23-7afbd6b1258d successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (11515, 20)


In [92]:
data.columns

Index(['customer_id', 'digitalLoanAccountId', 'c_cic_score', 'ScoreRange',
       'ln_loan_level_user_type', 'flg_zero_non_granted_ever',
       'flg_zero_granted_ever', 'Personal_Loans_granted_contracts_amt_24M',
       'granted_contracts_cnt_6M', 'total_overdue_granted_contracts',
       'has_ever_been_overdue', 'cnt_nongranted_contracts_3M',
       'cnt_active_contracts', 'max_amt_granted_24M',
       'tot_active_contracts_util', 'days_since_last_closed',
       'vel_contract_nongranted_cnt_6on12', 'vel_contract_granted_amt_6on12',
       'vel_contract_closed_amt_3on12', 'osType'],
      dtype='object')

In [93]:
data.head()

,customer_id,digitalLoanAccountId,c_cic_score,ScoreRange,ln_loan_level_user_type,flg_zero_non_granted_ever,flg_zero_granted_ever,Personal_Loans_granted_contracts_amt_24M,granted_contracts_cnt_6M,total_overdue_granted_contracts,has_ever_been_overdue,cnt_nongranted_contracts_3M,cnt_active_contracts,max_amt_granted_24M,tot_active_contracts_util,days_since_last_closed,vel_contract_nongranted_cnt_6on12,vel_contract_granted_amt_6on12,vel_contract_closed_amt_3on12,osType
0,2936666,d17fbffd-2abd-4ff4-9849-ffef3b9675ad,0.102868,Ai,1_Repeat Applicant,1,0,12851.0,1.0,0.0,0.0,NaN,1.0,12851.0,0.688200,256.0,NaN,1.000000,NaN,android
1,3003632,2a636db7-033f-4f7a-a480-35504778d8dc,0.300044,Ai,1_Repeat Applicant,0,0,NaN,1.0,1.0,1.0,NaN,1.0,4000000.0,0.995809,14.0,NaN,1.994068,1.000000,ios
2,2925302,a4eecf66-08bf-412f-80ba-02acae854100,0.044936,NH_Hi,1_Repeat Applicant,0,0,24990.0,NaN,0.0,0.0,NaN,3.0,24990.0,0.209013,186.0,NaN,NaN,NaN,android
3,2730438,f054ff5c-e34f-4ca4-86b7-fd376c82b051,0.104690,Fi,1_Repeat Applicant,1,0,NaN,2.0,0.0,0.0,NaN,3.0,23000.0,0.586132,7.0,NaN,0.600793,0.730776,android
4,2666368,388d5d4a-8c8a-4c9a-8a64-c7b3815a797b,0.165596,Ai,1_Repeat Applicant,1,0,NaN,1.0,0.0,0.0,NaN,1.0,6539.0,0.718458,NaN,NaN,1.000000,NaN,android


In [94]:
feature_column = ['ScoreRange',
       'ln_loan_level_user_type', 'flg_zero_non_granted_ever',
       'flg_zero_granted_ever', 'Personal_Loans_granted_contracts_amt_24M',
       'granted_contracts_cnt_6M', 'total_overdue_granted_contracts',
       'has_ever_been_overdue', 'cnt_nongranted_contracts_3M',
       'cnt_active_contracts', 'max_amt_granted_24M',
       'tot_active_contracts_util', 'days_since_last_closed',
       'vel_contract_nongranted_cnt_6on12', 'vel_contract_granted_amt_6on12',
       'vel_contract_closed_amt_3on12',]

dfd = transform_datav2(data, feature_column, a='c_cic_score', modelDisplayName='cic_model_sil', tc='Trench 3', subscription_name = 'Cash November 25 Models') 
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs
0,2936666,d17fbffd-2abd-4ff4-9849-ffef3b9675ad,6f126f69-8c68-4001-a66f-34cb1be183e5,0.102868,2025-12-13T11:41:52.831485,2025-12-13T11:41:52.831485,cic_model_sil,v2,"{""ScoreRange"": ""Ai"", ""ln_loan_level_user_type""...",Cash November 25 Models,642a39ed-bd25-43b9-a9dd-e321b0cf4739,2025-12-13T11:41:52.831485,{},Trench 3,android
1,3003632,2a636db7-033f-4f7a-a480-35504778d8dc,36562971-0966-49d5-8ded-8be246eb36db,0.300044,2025-12-13T11:41:52.831485,2025-12-13T11:41:52.831485,cic_model_sil,v2,"{""ScoreRange"": ""Ai"", ""ln_loan_level_user_type""...",Cash November 25 Models,6acc224a-57d7-4c7c-ae8e-879fcc6d3f24,2025-12-13T11:41:52.831485,{},Trench 3,ios
2,2925302,a4eecf66-08bf-412f-80ba-02acae854100,d24800b3-ddb7-439c-bbd5-47f7a5714c5a,0.044936,2025-12-13T11:41:52.831485,2025-12-13T11:41:52.831485,cic_model_sil,v2,"{""ScoreRange"": ""NH_Hi"", ""ln_loan_level_user_ty...",Cash November 25 Models,c9ccb3cc-9eaa-48bb-a883-2cd4298af8ef,2025-12-13T11:41:52.831485,{},Trench 3,android
3,2730438,f054ff5c-e34f-4ca4-86b7-fd376c82b051,3ffedbfa-ce5a-4827-84bf-25bb296af9e2,0.104690,2025-12-13T11:41:52.831485,2025-12-13T11:41:52.831485,cic_model_sil,v2,"{""ScoreRange"": ""Fi"", ""ln_loan_level_user_type""...",Cash November 25 Models,ad9cb8c8-de68-45c4-bd87-5051ba7ed6ee,2025-12-13T11:41:52.831485,{},Trench 3,android
4,2666368,388d5d4a-8c8a-4c9a-8a64-c7b3815a797b,16156759-a889-45a2-b70a-3dded73c858f,0.165596,2025-12-13T11:41:52.831485,2025-12-13T11:41:52.831485,cic_model_sil,v2,"{""ScoreRange"": ""Ai"", ""ln_loan_level_user_type""...",Cash November 25 Models,67ac5475-6788-45ee-8e2c-6535fb342410,2025-12-13T11:41:52.831485,{},Trench 3,android


In [95]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=1d7b3a97-6740-4a73-97cd-cec00c4293e0>

##### Alpha Sil Stack Model 

##### Trench 1

In [96]:
sq = """ 
select 
lmt.customerId customer_id,
r.digitalLoanAccountId,  
r.sa_stack_score ,
r.sb_demo_score,
r.apps_score,
r.s_credo_score,
r.sa_cic_score,
case 
  when lower(r.ln_os_type) like '%andro%' then 'android'
  when lower(r.ln_os_type) like '%os%' then 'ios'
  else 'ios' end  osType,
ln_loan_type
from worktable_data_analysis.sil_alpha_applied_loans_backscored_20240901_20251013_option3 r
left join `risk_credit_mis.loan_master_table` lmt on lmt.digitalLoanAccountId = r.digitalLoanAccountId
where r.trench_category = 'Trench 1'
and sa_stack_score is not null;
"""
data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")


Job ID d6347a46-b9fd-4fd7-a710-b95250c84138 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (232107, 9)


In [97]:
data.columns

Index(['customer_id', 'digitalLoanAccountId', 'sa_stack_score',
       'sb_demo_score', 'apps_score', 's_credo_score', 'sa_cic_score',
       'osType', 'ln_loan_type'],
      dtype='object')

In [98]:
data.head()

,customer_id,digitalLoanAccountId,sa_stack_score,sb_demo_score,apps_score,s_credo_score,sa_cic_score,osType,ln_loan_type
0,3481037,f656f4e0-8ecc-47a3-9d08-331ac6a0760d,0.311603,0.537488,0.444509,0.092131,0.133911,android,SIL Competitor
1,3700744,db8c80ba-bbf4-4c30-9913-ada27de12c11,0.281565,0.471865,0.346524,0.106750,0.223022,android,SIL Competitor
2,3396885,d7742b9e-2573-45a6-a700-4d0e9ce00005,0.327458,0.556793,0.446474,0.123172,0.110849,android,SIL Competitor
3,3598389,64d9500f-1451-46e4-81ad-ec9631cff4a4,0.248856,0.432464,0.495249,0.066159,0.106751,android,SIL Competitor
4,3490600,ccbd0a61-a61c-4146-877e-ceeacfcb19ed,0.371440,0.437702,0.403811,0.074748,0.339225,android,SIL Competitor


In [99]:
feature_column = ['sb_demo_score', 'apps_score', 's_credo_score', 'sa_cic_score',]

dfd = transform_datav2(data, feature_column, a='sa_stack_score', modelDisplayName='alpha_stack_model_sil', tc='Trench 1', subscription_name = 'Cash November 25 Models') 
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs
0,3481037,f656f4e0-8ecc-47a3-9d08-331ac6a0760d,29bfada7-2ee6-4d8b-b5d1-5d7c5266cec7,0.311603,2025-12-13T11:42:03.693154,2025-12-13T11:42:03.693154,alpha_stack_model_sil,v2,"{""sb_demo_score"": 0.5374880248061158, ""apps_sc...",Cash November 25 Models,75038420-ec17-45d2-8adf-7e24f615eba3,2025-12-13T11:42:03.693154,{},Trench 1,android
1,3700744,db8c80ba-bbf4-4c30-9913-ada27de12c11,dad39a70-dbbf-4a7e-8a70-1df0c3bae1e1,0.281565,2025-12-13T11:42:03.693154,2025-12-13T11:42:03.693154,alpha_stack_model_sil,v2,"{""sb_demo_score"": 0.4718651282911049, ""apps_sc...",Cash November 25 Models,dec173c9-ae89-4e57-b32b-af8725fa4113,2025-12-13T11:42:03.693154,{},Trench 1,android
2,3396885,d7742b9e-2573-45a6-a700-4d0e9ce00005,3d562536-4410-4c35-941e-87fa3a73dc4b,0.327458,2025-12-13T11:42:03.693154,2025-12-13T11:42:03.693154,alpha_stack_model_sil,v2,"{""sb_demo_score"": 0.5567932399037111, ""apps_sc...",Cash November 25 Models,1f4cd570-2bca-4dc9-ba37-233beb2d13b8,2025-12-13T11:42:03.693154,{},Trench 1,android
3,3598389,64d9500f-1451-46e4-81ad-ec9631cff4a4,9a2a8a23-764a-4e13-9275-5901e9fa85a4,0.248856,2025-12-13T11:42:03.693154,2025-12-13T11:42:03.693154,alpha_stack_model_sil,v2,"{""sb_demo_score"": 0.4324640290793849, ""apps_sc...",Cash November 25 Models,5f668d10-f4f9-4ee3-98b5-e8fed492c85b,2025-12-13T11:42:03.693154,{},Trench 1,android
4,3490600,ccbd0a61-a61c-4146-877e-ceeacfcb19ed,369ea798-b535-4e4a-8a43-d2e7a300e5e9,0.371440,2025-12-13T11:42:03.693154,2025-12-13T11:42:03.693154,alpha_stack_model_sil,v2,"{""sb_demo_score"": 0.4377016554808884, ""apps_sc...",Cash November 25 Models,c2db7341-08d3-4d41-a2fa-a8ad32cb5c8c,2025-12-13T11:42:03.693154,{},Trench 1,android


In [100]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=b6413003-ab79-4432-8a2b-9ea9fca5cd9b>

##### Trench 2

In [101]:
sq = """ 
select 
lmt.customerId customer_id,
r.digitalLoanAccountId,  
r.sa_stack_score ,
r.sb_demo_score,
r.apps_score,
r.s_credo_score,
r.sa_cic_score,
case 
  when lower(r.ln_os_type) like '%andro%' then 'android'
  when lower(r.ln_os_type) like '%os%' then 'ios'
  else 'ios' end  osType,
ln_loan_type
from worktable_data_analysis.sil_alpha_applied_loans_backscored_20240901_20251013_option3 r
left join `risk_credit_mis.loan_master_table` lmt on lmt.digitalLoanAccountId = r.digitalLoanAccountId
where r.trench_category = 'Trench 2'
and sa_stack_score is not null;
"""
data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")


Job ID ee5d0ad4-e248-433c-a261-d838aabe9b7e successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (10592, 9)


In [102]:
data.columns

Index(['customer_id', 'digitalLoanAccountId', 'sa_stack_score',
       'sb_demo_score', 'apps_score', 's_credo_score', 'sa_cic_score',
       'osType', 'ln_loan_type'],
      dtype='object')

In [103]:
data.head()

,customer_id,digitalLoanAccountId,sa_stack_score,sb_demo_score,apps_score,s_credo_score,sa_cic_score,osType,ln_loan_type
0,2547670,6e1c31c9-8008-44b5-9c11-60ed5f51fd26,0.289517,0.196343,NaN,0.078565,0.309831,ios,SIL Competitor
1,2642565,4a358910-114d-47d2-8cd3-c9f49f98f7e6,0.714234,0.510933,0.361727,0.269480,0.509798,android,SIL Competitor
2,1965699,0445ebe8-e3dc-4ea2-9bec-cc20efc0eba1,0.706989,0.586432,0.581838,0.131977,0.355412,android,SIL Competitor
3,3096122,efe5055a-8231-413d-a209-ba0520b81b4d,0.433972,0.368978,0.587596,0.125117,0.210982,android,SIL Competitor
4,2547959,448057ce-609b-4eac-96bf-06d0396d72e2,0.459151,0.536696,0.313047,0.130150,0.350822,android,SIL Competitor


In [104]:
feature_column = ['sb_demo_score', 'apps_score', 's_credo_score', 'sa_cic_score',]

dfd = transform_datav2(data, feature_column, a='sa_stack_score', modelDisplayName='alpha_stack_model_sil', tc='Trench 2', subscription_name = 'Cash November 25 Models') 
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs
0,2547670,6e1c31c9-8008-44b5-9c11-60ed5f51fd26,3a760b74-4536-44c4-8e76-7002c19d2681,0.289517,2025-12-13T11:42:35.367398,2025-12-13T11:42:35.367398,alpha_stack_model_sil,v2,"{""sb_demo_score"": 0.19634270237300064, ""s_cred...",Cash November 25 Models,05ac1b86-b1d9-4725-b8a7-71da91697f9b,2025-12-13T11:42:35.367398,{},Trench 2,ios
1,2642565,4a358910-114d-47d2-8cd3-c9f49f98f7e6,dcb3bf1b-896d-4d12-aa93-81d69d926c46,0.714234,2025-12-13T11:42:35.367398,2025-12-13T11:42:35.367398,alpha_stack_model_sil,v2,"{""sb_demo_score"": 0.5109328094220427, ""apps_sc...",Cash November 25 Models,e63f62ac-ff43-4f16-891a-104e8040154f,2025-12-13T11:42:35.367398,{},Trench 2,android
2,1965699,0445ebe8-e3dc-4ea2-9bec-cc20efc0eba1,d36f748a-62bc-495a-85e6-58361490bd9e,0.706989,2025-12-13T11:42:35.367398,2025-12-13T11:42:35.367398,alpha_stack_model_sil,v2,"{""sb_demo_score"": 0.5864316895990638, ""apps_sc...",Cash November 25 Models,b590bf6e-8c03-4aae-bfee-73df877bfcf0,2025-12-13T11:42:35.367398,{},Trench 2,android
3,3096122,efe5055a-8231-413d-a209-ba0520b81b4d,668be317-8ebe-4dd3-8542-28269ee4b813,0.433972,2025-12-13T11:42:35.368397,2025-12-13T11:42:35.368397,alpha_stack_model_sil,v2,"{""sb_demo_score"": 0.36897801266633007, ""apps_s...",Cash November 25 Models,16e06813-8365-49ea-8fd8-af5033aa1bd8,2025-12-13T11:42:35.368397,{},Trench 2,android
4,2547959,448057ce-609b-4eac-96bf-06d0396d72e2,277eb95e-0d23-4d12-a5ac-eda8b380d352,0.459151,2025-12-13T11:42:35.368397,2025-12-13T11:42:35.368397,alpha_stack_model_sil,v2,"{""sb_demo_score"": 0.5366956870882912, ""apps_sc...",Cash November 25 Models,82d524ea-4b6a-44bd-b6db-a728cb149630,2025-12-13T11:42:35.368397,{},Trench 2,android


In [105]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=58d130cb-6f7d-4b50-b422-7cadce04b3b2>

##### Trench 3

In [106]:
sq = """ 
select 
lmt.customerId customer_id,
r.digitalLoanAccountId,  
r.sa_stack_score ,
r.sb_demo_score,
r.apps_score,
r.s_credo_score,
r.sa_cic_score,
case 
  when lower(r.ln_os_type) like '%andro%' then 'android'
  when lower(r.ln_os_type) like '%os%' then 'ios'
  else 'ios' end  osType,
ln_loan_type
from worktable_data_analysis.sil_alpha_applied_loans_backscored_20240901_20251013_option3 r
left join `risk_credit_mis.loan_master_table` lmt on lmt.digitalLoanAccountId = r.digitalLoanAccountId
where r.trench_category = 'Trench 3'
and sa_stack_score is not null;
"""
data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")


Job ID 07e1df62-3cd3-4e61-b4a1-6fa9bdb17475 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (11874, 9)


In [107]:
data.columns

Index(['customer_id', 'digitalLoanAccountId', 'sa_stack_score',
       'sb_demo_score', 'apps_score', 's_credo_score', 'sa_cic_score',
       'osType', 'ln_loan_type'],
      dtype='object')

In [108]:
data.head()

,customer_id,digitalLoanAccountId,sa_stack_score,sb_demo_score,apps_score,s_credo_score,sa_cic_score,osType,ln_loan_type
0,3011254,ca600978-4622-415c-92d1-c628311975bb,0.179930,0.321014,0.292255,0.106226,0.140160,android,SIL Competitor
1,2566884,60027830-0e28-4949-8e07-3f8ceadb8087,0.277277,0.355877,0.334537,0.190294,0.158369,android,SIL Competitor
2,1838756,519b8a39-5d56-4178-a772-53841510d0f8,0.179532,0.278837,0.297683,0.096740,0.176114,android,SIL Competitor
3,3523628,2e7b702e-4697-41b7-9238-8063a62d6dfc,0.262543,0.337893,0.364588,0.162820,0.147752,android,SIL Competitor
4,3165626,6de083e3-819d-4dae-ab64-c0f73e982ce1,0.099499,0.141245,0.144016,0.027735,0.293880,android,SIL Competitor


In [109]:
feature_column = ['sb_demo_score', 'apps_score', 's_credo_score', 'sa_cic_score',]

dfd = transform_datav2(data, feature_column, a='sa_stack_score', modelDisplayName='alpha_stack_model_sil', tc='Trench 3', subscription_name = 'Cash November 25 Models') 
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs
0,3011254,ca600978-4622-415c-92d1-c628311975bb,67e29398-bc31-479b-9e71-5676b1518f66,0.179930,2025-12-13T11:42:42.393349,2025-12-13T11:42:42.393349,alpha_stack_model_sil,v2,"{""sb_demo_score"": 0.32101435813638163, ""apps_s...",Cash November 25 Models,fe44169a-0bd5-45ce-a579-ae2b257c227a,2025-12-13T11:42:42.393349,{},Trench 3,android
1,2566884,60027830-0e28-4949-8e07-3f8ceadb8087,01953de4-6814-467b-896e-69756b0d305d,0.277277,2025-12-13T11:42:42.393349,2025-12-13T11:42:42.393349,alpha_stack_model_sil,v2,"{""sb_demo_score"": 0.35587717783391953, ""apps_s...",Cash November 25 Models,8aa46750-fc9c-4314-a411-e49328f57122,2025-12-13T11:42:42.393349,{},Trench 3,android
2,1838756,519b8a39-5d56-4178-a772-53841510d0f8,848d00b1-b683-43ff-8e64-7d0bf80773ab,0.179532,2025-12-13T11:42:42.394530,2025-12-13T11:42:42.394530,alpha_stack_model_sil,v2,"{""sb_demo_score"": 0.2788369786573112, ""apps_sc...",Cash November 25 Models,2ecc27a2-adf0-4789-87e2-2a797f78ece7,2025-12-13T11:42:42.394530,{},Trench 3,android
3,3523628,2e7b702e-4697-41b7-9238-8063a62d6dfc,4642c702-2b86-4839-86ce-a3797473fe28,0.262543,2025-12-13T11:42:42.394530,2025-12-13T11:42:42.394530,alpha_stack_model_sil,v2,"{""sb_demo_score"": 0.33789257513632165, ""apps_s...",Cash November 25 Models,3d298c8d-a125-4a25-ad20-4ccd10cb739b,2025-12-13T11:42:42.394530,{},Trench 3,android
4,3165626,6de083e3-819d-4dae-ab64-c0f73e982ce1,1d317981-af73-48f3-a0e1-36710e2ba7cc,0.099499,2025-12-13T11:42:42.395541,2025-12-13T11:42:42.395541,alpha_stack_model_sil,v2,"{""sb_demo_score"": 0.14124460581822879, ""apps_s...",Cash November 25 Models,43b3a46e-58f9-47bf-b4a0-318a0823072e,2025-12-13T11:42:42.395541,{},Trench 3,android


In [110]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=c22a5d05-9d49-4dbf-b890-71c2ce0c4ceb>

##### Beta SIL STACK Score Model

##### Trench 1

In [111]:
sq = """ 
select 
lmt.customerId customer_id,
r.digitalLoanAccountId,  
r.sb_stack_score ,
r.sb_demo_score,
r.apps_score,
r.s_credo_score,
case 
  when lower(r.ln_os_type) like '%andro%' then 'android'
  when lower(r.ln_os_type) like '%os%' then 'ios'
  else 'ios' end  osType,
ln_loan_type
from prj-prod-dataplatform.worktable_data_analysis.sil_beta_applied_loans_backscored_20240801_20251013_option3 r
left join `risk_credit_mis.loan_master_table` lmt on lmt.digitalLoanAccountId = r.digitalLoanAccountId
where r.trench_category = 'Trench 1'
and sb_stack_score is not null;
"""
data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")


Job ID b0605119-36ba-4f0f-857f-1ea7fd913747 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (343968, 8)


In [112]:
data.columns

Index(['customer_id', 'digitalLoanAccountId', 'sb_stack_score',
       'sb_demo_score', 'apps_score', 's_credo_score', 'osType',
       'ln_loan_type'],
      dtype='object')

In [113]:
data.head()

,customer_id,digitalLoanAccountId,sb_stack_score,sb_demo_score,apps_score,s_credo_score,osType,ln_loan_type
0,3681589,ea36cf67-dfab-4cd1-aa7a-0fc2a48231d8,0.296435,0.238267,0.481102,0.130798,android,SIL Competitor
1,3737577,606961e5-11c7-4e9a-850d-4ce44b6a4ace,0.172791,0.319400,0.273983,0.067540,android,SIL Competitor
2,3722855,3b402a45-746c-4834-b2c0-b0e21481f321,0.366284,0.263582,0.507065,0.171303,android,SIL Competitor
3,3611553,7d42df4f-754b-4949-ba30-40539da48421,0.099853,0.214675,0.226834,0.032116,android,SIL Competitor
4,3638865,19dae2d2-a12e-45e4-bb5b-27e8ce21ef4c,0.197939,0.248056,0.385283,0.066864,android,SIL Competitor


In [114]:
feature_column = ['sb_demo_score', 'apps_score', 's_credo_score']

dfd = transform_datav2(data, feature_column, a='sb_stack_score', modelDisplayName='beta_stack_model_sil', tc='Trench 1', subscription_name = 'Cash November 25 Models') 
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs
0,3681589,ea36cf67-dfab-4cd1-aa7a-0fc2a48231d8,638ed2c1-51b3-4a9f-b595-c4950413da42,0.296435,2025-12-13T11:42:52.709438,2025-12-13T11:42:52.709438,beta_stack_model_sil,v2,"{""sb_demo_score"": 0.23826653309813217, ""apps_s...",Cash November 25 Models,2ef4882d-e610-47ad-b35b-59cb67bbbea3,2025-12-13T11:42:52.709438,{},Trench 1,android
1,3737577,606961e5-11c7-4e9a-850d-4ce44b6a4ace,b19541c2-4c88-403c-948d-542c92eb1d37,0.172791,2025-12-13T11:42:52.709438,2025-12-13T11:42:52.709438,beta_stack_model_sil,v2,"{""sb_demo_score"": 0.31939965324868946, ""apps_s...",Cash November 25 Models,f66a85c6-3754-4496-9a05-e07e67db8844,2025-12-13T11:42:52.709438,{},Trench 1,android
2,3722855,3b402a45-746c-4834-b2c0-b0e21481f321,f39bbd04-04f0-4f2d-badd-131b78f05d16,0.366284,2025-12-13T11:42:52.710435,2025-12-13T11:42:52.710435,beta_stack_model_sil,v2,"{""sb_demo_score"": 0.26358219740397604, ""apps_s...",Cash November 25 Models,88f9e9f3-d134-465c-bfb9-1e9cb40f5382,2025-12-13T11:42:52.710435,{},Trench 1,android
3,3611553,7d42df4f-754b-4949-ba30-40539da48421,780c7fc0-3778-49d8-a18e-e3228b905ef7,0.099853,2025-12-13T11:42:52.710435,2025-12-13T11:42:52.710435,beta_stack_model_sil,v2,"{""sb_demo_score"": 0.21467487217539485, ""apps_s...",Cash November 25 Models,b6ee2bf1-f595-4dae-8f48-c7dd9acb2875,2025-12-13T11:42:52.710435,{},Trench 1,android
4,3638865,19dae2d2-a12e-45e4-bb5b-27e8ce21ef4c,cffa9369-b503-46f7-82f9-512b951da798,0.197939,2025-12-13T11:42:52.710435,2025-12-13T11:42:52.710435,beta_stack_model_sil,v2,"{""sb_demo_score"": 0.24805578873168624, ""apps_s...",Cash November 25 Models,dc89a3d7-57ca-46b9-b4ff-842fe91af75e,2025-12-13T11:42:52.710435,{},Trench 1,android


In [115]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=7b71f125-f9ee-4d79-b4bc-b21872082b1f>

##### Trench 2

In [116]:
sq = """ 
select 
lmt.customerId customer_id,
r.digitalLoanAccountId,  
r.sb_stack_score ,
r.sb_demo_score,
r.apps_score,
r.s_credo_score,
case 
  when lower(r.ln_os_type) like '%andro%' then 'android'
  when lower(r.ln_os_type) like '%os%' then 'ios'
  else 'ios' end  osType,
ln_loan_type
from prj-prod-dataplatform.worktable_data_analysis.sil_beta_applied_loans_backscored_20240801_20251013_option3 r
left join `risk_credit_mis.loan_master_table` lmt on lmt.digitalLoanAccountId = r.digitalLoanAccountId
where r.trench_category = 'Trench 2'
and sb_stack_score is not null;
"""
data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")


Job ID e1612d59-273b-4ca5-8bbd-ceeb16408815 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (13099, 8)


In [117]:
data.columns

Index(['customer_id', 'digitalLoanAccountId', 'sb_stack_score',
       'sb_demo_score', 'apps_score', 's_credo_score', 'osType',
       'ln_loan_type'],
      dtype='object')

In [118]:
data.head()

,customer_id,digitalLoanAccountId,sb_stack_score,sb_demo_score,apps_score,s_credo_score,osType,ln_loan_type
0,2658463,ca9ef30a-7ca1-424c-bbf8-1e9a43c362f3,0.153470,0.277940,0.256788,0.081002,android,SIL Competitor
1,2308542,e8dbc21c-0dd8-41aa-b47a-c0ca6dbd878f,0.343566,0.521737,0.307231,0.105908,android,SIL Competitor
2,2787976,2e1789f8-3189-411e-a845-682b23780f0a,0.346750,0.362235,NaN,0.081611,ios,SIL Competitor
3,2851434,d557b085-a6f1-4f96-80b3-2c7526508017,0.641052,0.554357,0.550931,0.176189,android,SIL Competitor
4,3147455,7bd2018d-6fa9-4abb-ad87-2067dfcb0174,0.449669,0.510103,NaN,0.067433,ios,SIL Competitor


In [119]:
feature_column = ['sb_demo_score', 'apps_score', 's_credo_score']

dfd = transform_datav2(data, feature_column, a='sb_stack_score', modelDisplayName='beta_stack_model_sil', tc='Trench 2', subscription_name = 'Cash November 25 Models') 
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs
0,2658463,ca9ef30a-7ca1-424c-bbf8-1e9a43c362f3,555efc2a-2b95-43a9-a51a-fc2b661d0c27,0.153470,2025-12-13T11:43:33.096726,2025-12-13T11:43:33.096726,beta_stack_model_sil,v2,"{""sb_demo_score"": 0.27794027633420615, ""apps_s...",Cash November 25 Models,ff5c8e9e-d0da-4337-8b0b-b524be979e52,2025-12-13T11:43:33.096726,{},Trench 2,android
1,2308542,e8dbc21c-0dd8-41aa-b47a-c0ca6dbd878f,d66d500a-85d5-445d-ba8e-1a3a3124207c,0.343566,2025-12-13T11:43:33.096726,2025-12-13T11:43:33.096726,beta_stack_model_sil,v2,"{""sb_demo_score"": 0.5217373157660004, ""apps_sc...",Cash November 25 Models,8e0ab47d-7427-4aa9-8fae-8701501f5b47,2025-12-13T11:43:33.096726,{},Trench 2,android
2,2787976,2e1789f8-3189-411e-a845-682b23780f0a,5a2d1f81-9b2b-471a-b182-d5fb2e5e1af2,0.346750,2025-12-13T11:43:33.096726,2025-12-13T11:43:33.096726,beta_stack_model_sil,v2,"{""sb_demo_score"": 0.3622347247176597, ""s_credo...",Cash November 25 Models,5ef77897-61f7-471d-9a70-f39957511ddf,2025-12-13T11:43:33.096726,{},Trench 2,ios
3,2851434,d557b085-a6f1-4f96-80b3-2c7526508017,2f0dc6ef-4d90-4c15-98e5-5c554d63a971,0.641052,2025-12-13T11:43:33.096726,2025-12-13T11:43:33.096726,beta_stack_model_sil,v2,"{""sb_demo_score"": 0.5543574742389406, ""apps_sc...",Cash November 25 Models,902198be-3b1e-412e-a6cd-89b43ad9a99c,2025-12-13T11:43:33.096726,{},Trench 2,android
4,3147455,7bd2018d-6fa9-4abb-ad87-2067dfcb0174,8477a2e8-41b3-4472-b500-954e6acb134b,0.449669,2025-12-13T11:43:33.096726,2025-12-13T11:43:33.096726,beta_stack_model_sil,v2,"{""sb_demo_score"": 0.5101027261789729, ""s_credo...",Cash November 25 Models,9e6d6757-0e6a-4b62-afe4-df5acbfbf5da,2025-12-13T11:43:33.096726,{},Trench 2,ios


In [120]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=cc12f523-f72d-470e-8f05-dce4fac67a75>

##### Trench 3

In [121]:
sq = """ 
select 
lmt.customerId customer_id,
r.digitalLoanAccountId,  
r.sb_stack_score ,
r.sb_demo_score,
r.apps_score,
r.s_credo_score,
case 
  when lower(r.ln_os_type) like '%andro%' then 'android'
  when lower(r.ln_os_type) like '%os%' then 'ios'
  else 'ios' end  osType,
ln_loan_type
from prj-prod-dataplatform.worktable_data_analysis.sil_beta_applied_loans_backscored_20240801_20251013_option3 r
left join `risk_credit_mis.loan_master_table` lmt on lmt.digitalLoanAccountId = r.digitalLoanAccountId
where r.trench_category = 'Trench 3'
and sb_stack_score is not null;
"""
data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")


Job ID 20d7be47-feb3-4d39-a4ea-fb8d4a70874b successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (13802, 8)


In [122]:
data.columns

Index(['customer_id', 'digitalLoanAccountId', 'sb_stack_score',
       'sb_demo_score', 'apps_score', 's_credo_score', 'osType',
       'ln_loan_type'],
      dtype='object')

In [123]:
data.head()

,customer_id,digitalLoanAccountId,sb_stack_score,sb_demo_score,apps_score,s_credo_score,osType,ln_loan_type
0,2394995,fc7d91c6-1926-4b29-b687-05384f98a76e,0.303552,0.301064,NaN,0.079688,ios,SIL Competitor
1,2670076,5eb0c2cd-e45b-4c53-b4da-e9874611bf6e,0.176232,0.301342,0.280287,0.079997,android,SIL Competitor
2,3085084,12366fbf-51d8-43c8-822d-f87ed7268a23,0.196220,0.159609,0.435889,0.089245,android,SIL Competitor
3,2992833,bfb17f27-8ad8-4bcc-a675-6d9190ce4aa8,0.245402,0.385636,0.367656,0.030993,android,SIL Competitor
4,2518306,29d5dfe4-843e-4968-9338-3098e51c1868,0.193002,0.268798,0.359347,0.060499,android,SIL Competitor


In [124]:
feature_column = ['sb_demo_score', 'apps_score', 's_credo_score']

dfd = transform_datav2(data, feature_column, a='sb_stack_score', modelDisplayName='beta_stack_model_sil', tc='Trench 3', subscription_name = 'Cash November 25 Models') 
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs
0,2394995,fc7d91c6-1926-4b29-b687-05384f98a76e,1610e567-3633-4d95-9b71-7e6139b076f6,0.303552,2025-12-13T11:43:40.548864,2025-12-13T11:43:40.548864,beta_stack_model_sil,v2,"{""sb_demo_score"": 0.30106432627837937, ""s_cred...",Cash November 25 Models,45551b01-b3f5-4797-80db-d648be0ed021,2025-12-13T11:43:40.548864,{},Trench 3,ios
1,2670076,5eb0c2cd-e45b-4c53-b4da-e9874611bf6e,e3e86a83-c6dd-450e-89d5-b4e3861ef0a2,0.176232,2025-12-13T11:43:40.548864,2025-12-13T11:43:40.548864,beta_stack_model_sil,v2,"{""sb_demo_score"": 0.3013421470784764, ""apps_sc...",Cash November 25 Models,fd68c798-1b34-4174-a948-0a1dc5877af4,2025-12-13T11:43:40.548864,{},Trench 3,android
2,3085084,12366fbf-51d8-43c8-822d-f87ed7268a23,03f7f881-2a06-4fa3-88c0-6bbf4093bd74,0.196220,2025-12-13T11:43:40.549866,2025-12-13T11:43:40.549866,beta_stack_model_sil,v2,"{""sb_demo_score"": 0.1596088009036647, ""apps_sc...",Cash November 25 Models,129c0b77-6f65-4fb8-8b04-6b1ed77a89d4,2025-12-13T11:43:40.549866,{},Trench 3,android
3,2992833,bfb17f27-8ad8-4bcc-a675-6d9190ce4aa8,1a8ccd9a-8591-45eb-9c76-09d98b069014,0.245402,2025-12-13T11:43:40.549866,2025-12-13T11:43:40.549866,beta_stack_model_sil,v2,"{""sb_demo_score"": 0.38563637007244794, ""apps_s...",Cash November 25 Models,715ad061-6738-4c4b-872d-41885537bb6a,2025-12-13T11:43:40.549866,{},Trench 3,android
4,2518306,29d5dfe4-843e-4968-9338-3098e51c1868,cdc38652-d3b9-47df-b988-dc50efec1d78,0.193002,2025-12-13T11:43:40.549866,2025-12-13T11:43:40.549866,beta_stack_model_sil,v2,"{""sb_demo_score"": 0.26879790991352953, ""apps_s...",Cash November 25 Models,e6e55497-b30c-4eea-9104-6933621cdae2,2025-12-13T11:43:40.549866,{},Trench 3,android


In [125]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=05ca5157-5a5e-44fb-9eea-0c8be53fea5e>

##### Beta Sil App Score

##### Trench 1

In [126]:
sq = """ 
select 
distinct 
loanmaster.customerId customer_id,
r.digitalLoanAccountId,
r.apps_score,
app_cnt_payday_ever,
app_cnt_finance_ever,
app_cnt_competitors_sil_ever,
app_cnt_competitors_ever,
app_cnt_finance_365d,
app_cnt_absence_tag_365d,
app_cnt_competitors_sil_365d,
app_cnt_finance_7d,
app_cnt_rated_for_3plus_ever,
app_cnt_payday_7d,
    case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
    when lower(loanmaster.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
from  risk_mart.applied_sil_new_applicants_loan_20241001_20251124_app_scored r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
where apps_score is not null
and trench_category = 'Trench 1'
;
"""
data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")


Job ID 4f6a7b2f-af4d-4555-99e4-7d21777ce434 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (296743, 14)


In [127]:
data.columns

Index(['customer_id', 'digitalLoanAccountId', 'apps_score',
       'app_cnt_payday_ever', 'app_cnt_finance_ever',
       'app_cnt_competitors_sil_ever', 'app_cnt_competitors_ever',
       'app_cnt_finance_365d', 'app_cnt_absence_tag_365d',
       'app_cnt_competitors_sil_365d', 'app_cnt_finance_7d',
       'app_cnt_rated_for_3plus_ever', 'app_cnt_payday_7d', 'osType'],
      dtype='object')

In [128]:
data.head()

,customer_id,digitalLoanAccountId,apps_score,app_cnt_payday_ever,app_cnt_finance_ever,app_cnt_competitors_sil_ever,app_cnt_competitors_ever,app_cnt_finance_365d,app_cnt_absence_tag_365d,app_cnt_competitors_sil_365d,app_cnt_finance_7d,app_cnt_rated_for_3plus_ever,app_cnt_payday_7d,osType
0,3706437,aae332a5-da24-47ca-a35f-7a32c88fe43f,0.358293,7,15.0,4,9,12.0,3.0,3.0,0.0,45.0,0.0,android
1,3804859,68a1765a-2c44-44c2-8b8a-e4e22bda497e,0.112190,2,21.0,1,6,5.0,3.0,0.0,0.0,70.0,0.0,android
2,3096094,432a9ddd-3e69-4541-9a96-fa9e5432410c,0.419632,2,12.0,2,5,11.0,24.0,2.0,1.0,35.0,1.0,android
3,3111135,3fa78901-d96b-4335-a267-5d96901dc96f,0.230816,3,13.0,1,8,8.0,3.0,0.0,0.0,43.0,0.0,android
4,3588574,727b2251-bba9-4b26-8fe1-163f07331baa,0.361614,9,12.0,1,2,1.0,4.0,1.0,1.0,22.0,1.0,android


In [129]:
feature_column = ['app_cnt_payday_ever', 'app_cnt_finance_ever',
       'app_cnt_competitors_sil_ever', 'app_cnt_competitors_ever',
       'app_cnt_finance_365d', 'app_cnt_absence_tag_365d',
       'app_cnt_competitors_sil_365d', 'app_cnt_finance_7d',
       'app_cnt_rated_for_3plus_ever', 'app_cnt_payday_7d',]

dfd = transform_datav2(data, feature_column, a='apps_score', modelDisplayName='apps_score_model_sil', tc='Trench 1', subscription_name = 'Cash November 25 Models') 
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs
0,3706437,aae332a5-da24-47ca-a35f-7a32c88fe43f,366fe2c5-9e98-4f68-ae2f-db4199e536cc,0.358293,2025-12-13T11:43:51.234685,2025-12-13T11:43:51.234685,apps_score_model_sil,v2,"{""app_cnt_payday_ever"": 7, ""app_cnt_finance_ev...",Cash November 25 Models,dbe1b34d-2c15-4532-82db-e6c9777c2e90,2025-12-13T11:43:51.234685,{},Trench 1,android
1,3804859,68a1765a-2c44-44c2-8b8a-e4e22bda497e,7e059592-46bf-4efe-a861-66ef90ad8d2f,0.112190,2025-12-13T11:43:51.234685,2025-12-13T11:43:51.234685,apps_score_model_sil,v2,"{""app_cnt_payday_ever"": 2, ""app_cnt_finance_ev...",Cash November 25 Models,5e8f329b-0f73-4bc1-aff7-4febb5db2adb,2025-12-13T11:43:51.234685,{},Trench 1,android
2,3096094,432a9ddd-3e69-4541-9a96-fa9e5432410c,c4c5fcd7-85f3-4168-a35b-57f9b5af223a,0.419632,2025-12-13T11:43:51.234685,2025-12-13T11:43:51.234685,apps_score_model_sil,v2,"{""app_cnt_payday_ever"": 2, ""app_cnt_finance_ev...",Cash November 25 Models,c5b4e111-22a9-4d49-95e3-e09abefe7d63,2025-12-13T11:43:51.234685,{},Trench 1,android
3,3111135,3fa78901-d96b-4335-a267-5d96901dc96f,cb9afa02-23f7-424c-9e06-42fcfc37f28f,0.230816,2025-12-13T11:43:51.235236,2025-12-13T11:43:51.235236,apps_score_model_sil,v2,"{""app_cnt_payday_ever"": 3, ""app_cnt_finance_ev...",Cash November 25 Models,a6740675-5c57-4806-8cc3-9885cbc4d461,2025-12-13T11:43:51.235236,{},Trench 1,android
4,3588574,727b2251-bba9-4b26-8fe1-163f07331baa,b89044a7-1b53-465c-ae11-843d9901fd8d,0.361614,2025-12-13T11:43:51.235236,2025-12-13T11:43:51.235236,apps_score_model_sil,v2,"{""app_cnt_payday_ever"": 9, ""app_cnt_finance_ev...",Cash November 25 Models,d790871e-9215-42ef-9729-aefcd7f31d1e,2025-12-13T11:43:51.235236,{},Trench 1,android


In [130]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=46cf22be-ea05-4d17-9bb4-685dd13559ec>

##### Trench 2

In [131]:
sq = """ 
select 
distinct 
loanmaster.customerId customer_id,
r.digitalLoanAccountId,
r.apps_score,
app_cnt_payday_ever,
app_cnt_finance_ever,
app_cnt_competitors_sil_ever,
app_cnt_competitors_ever,
app_cnt_finance_365d,
app_cnt_absence_tag_365d,
app_cnt_competitors_sil_365d,
app_cnt_finance_7d,
app_cnt_rated_for_3plus_ever,
app_cnt_payday_7d,
    case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
    when lower(loanmaster.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
from  risk_mart.applied_sil_new_applicants_loan_20241001_20251124_app_scored r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
where apps_score is not null
and trench_category = 'Trench 2'
;
"""
data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")


Job ID 3a946463-21ed-4756-b680-c8a8d1ecde15 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (12240, 14)


In [132]:
data.columns

Index(['customer_id', 'digitalLoanAccountId', 'apps_score',
       'app_cnt_payday_ever', 'app_cnt_finance_ever',
       'app_cnt_competitors_sil_ever', 'app_cnt_competitors_ever',
       'app_cnt_finance_365d', 'app_cnt_absence_tag_365d',
       'app_cnt_competitors_sil_365d', 'app_cnt_finance_7d',
       'app_cnt_rated_for_3plus_ever', 'app_cnt_payday_7d', 'osType'],
      dtype='object')

In [133]:
data.head()

,customer_id,digitalLoanAccountId,apps_score,app_cnt_payday_ever,app_cnt_finance_ever,app_cnt_competitors_sil_ever,app_cnt_competitors_ever,app_cnt_finance_365d,app_cnt_absence_tag_365d,app_cnt_competitors_sil_365d,app_cnt_finance_7d,app_cnt_rated_for_3plus_ever,app_cnt_payday_7d,osType
0,2970212,d34c2e92-cb43-43c5-8169-dd754e0b4331,0.469220,23,32.0,4,7,32.0,20.0,4.0,1.0,72.0,0.0,android
1,2124714,a0e428dd-96c5-49e4-a54f-0dc5a90fbbda,0.605375,4,19.0,1,9,19.0,10.0,1.0,0.0,38.0,0.0,android
2,2336638,0b1f81a3-47a8-44b0-95f7-654bf20c633d,0.251713,6,29.0,3,12,13.0,9.0,3.0,0.0,77.0,0.0,android
3,3387619,12e15e17-b274-4f0f-9240-1861b069c1aa,0.544086,26,35.0,4,8,33.0,14.0,4.0,0.0,60.0,0.0,android
4,2605433,6487aadf-ede1-4775-8d67-4711771a9fa8,0.525955,12,25.0,1,9,6.0,2.0,0.0,0.0,41.0,0.0,android


In [134]:
feature_column = ['app_cnt_payday_ever', 'app_cnt_finance_ever',
       'app_cnt_competitors_sil_ever', 'app_cnt_competitors_ever',
       'app_cnt_finance_365d', 'app_cnt_absence_tag_365d',
       'app_cnt_competitors_sil_365d', 'app_cnt_finance_7d',
       'app_cnt_rated_for_3plus_ever', 'app_cnt_payday_7d',]

dfd = transform_datav2(data, feature_column, a='apps_score', modelDisplayName='apps_score_model_sil', tc='Trench 2', subscription_name = 'Cash November 25 Models') 
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs
0,2970212,d34c2e92-cb43-43c5-8169-dd754e0b4331,93e12908-677f-4fae-97bc-fc68e5972b28,0.469220,2025-12-13T11:44:43.179440,2025-12-13T11:44:43.179440,apps_score_model_sil,v2,"{""app_cnt_payday_ever"": 23, ""app_cnt_finance_e...",Cash November 25 Models,ce851f6a-4cf5-4aab-929d-c83c436f185e,2025-12-13T11:44:43.179440,{},Trench 2,android
1,2124714,a0e428dd-96c5-49e4-a54f-0dc5a90fbbda,abb39de9-6330-49f7-bb96-d27e77e7ce60,0.605375,2025-12-13T11:44:43.179440,2025-12-13T11:44:43.179440,apps_score_model_sil,v2,"{""app_cnt_payday_ever"": 4, ""app_cnt_finance_ev...",Cash November 25 Models,7818fe11-b722-4110-934a-6f363270e286,2025-12-13T11:44:43.179440,{},Trench 2,android
2,2336638,0b1f81a3-47a8-44b0-95f7-654bf20c633d,82a6b805-6fdf-41b3-bf4a-1859c06d59b6,0.251713,2025-12-13T11:44:43.179440,2025-12-13T11:44:43.179440,apps_score_model_sil,v2,"{""app_cnt_payday_ever"": 6, ""app_cnt_finance_ev...",Cash November 25 Models,0420f9ea-59b7-46b4-9540-9cdda2676e62,2025-12-13T11:44:43.179440,{},Trench 2,android
3,3387619,12e15e17-b274-4f0f-9240-1861b069c1aa,0b55dac6-3b0f-4d3a-a29e-b57f0d873742,0.544086,2025-12-13T11:44:43.179440,2025-12-13T11:44:43.179440,apps_score_model_sil,v2,"{""app_cnt_payday_ever"": 26, ""app_cnt_finance_e...",Cash November 25 Models,b3053ac4-d712-4ad4-952f-572bb85772f6,2025-12-13T11:44:43.179440,{},Trench 2,android
4,2605433,6487aadf-ede1-4775-8d67-4711771a9fa8,7d1538d4-e2bb-4611-8254-26f9be7a4552,0.525955,2025-12-13T11:44:43.180444,2025-12-13T11:44:43.180444,apps_score_model_sil,v2,"{""app_cnt_payday_ever"": 12, ""app_cnt_finance_e...",Cash November 25 Models,5543df67-29cf-4731-b3f3-92f112214110,2025-12-13T11:44:43.180444,{},Trench 2,android


In [135]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=bb3ed279-de48-419c-8875-f07021f86b1d>

##### Trench 3

In [136]:
sq = """ 
select 
r.customerId customer_id,
r.digitalLoanAccountId,
r.apps_score,
app_cnt_productivity_ever,
app_cnt_rated_for_3plus_ever, 
app_cnt_books_and_reference_ever,
app_cnt_tools_ever, 
app_median_time_bw_installed_mins_3d,
app_median_time_bw_installed_mins_30d,
app_cnt_communication_ever, 
app_cnt_finance_90d,
app_cnt_absence_tag_180d, 
app_cnt_shopping_ever,
app_cnt_social_ever, 
app_cnt_driver_90d, 
app_cnt_payday_365d,
app_cnt_driver_365d, 
app_cnt_music_and_audio_ever,
app_cnt_finance_180d, 
app_cnt_art_and_design_ever,
app_cnt_gaming_90d, 
app_avg_time_bw_installed_mins_30d,
app_cnt_education_ever,
case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
    when lower(loanmaster.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
from risk_mart.applied_sil_repeat_applicants_loan_20241001_20251124_app_scored r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
where apps_score is not null
;
"""
data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")

Job ID 3a236767-0800-49f6-91cd-c55aa6c6c9ed successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (20246, 24)


In [137]:
data.columns

Index(['customer_id', 'digitalLoanAccountId', 'apps_score',
       'app_cnt_productivity_ever', 'app_cnt_rated_for_3plus_ever',
       'app_cnt_books_and_reference_ever', 'app_cnt_tools_ever',
       'app_median_time_bw_installed_mins_3d',
       'app_median_time_bw_installed_mins_30d', 'app_cnt_communication_ever',
       'app_cnt_finance_90d', 'app_cnt_absence_tag_180d',
       'app_cnt_shopping_ever', 'app_cnt_social_ever', 'app_cnt_driver_90d',
       'app_cnt_payday_365d', 'app_cnt_driver_365d',
       'app_cnt_music_and_audio_ever', 'app_cnt_finance_180d',
       'app_cnt_art_and_design_ever', 'app_cnt_gaming_90d',
       'app_avg_time_bw_installed_mins_30d', 'app_cnt_education_ever',
       'osType'],
      dtype='object')

In [138]:
feature_column = ['app_cnt_productivity_ever', 'app_cnt_rated_for_3plus_ever',
       'app_cnt_books_and_reference_ever', 'app_cnt_tools_ever',
       'app_median_time_bw_installed_mins_3d',
       'app_median_time_bw_installed_mins_30d', 'app_cnt_communication_ever',
       'app_cnt_finance_90d', 'app_cnt_absence_tag_180d',
       'app_cnt_shopping_ever', 'app_cnt_social_ever', 'app_cnt_driver_90d',
       'app_cnt_payday_365d', 'app_cnt_driver_365d',
       'app_cnt_music_and_audio_ever', 'app_cnt_finance_180d',
       'app_cnt_art_and_design_ever', 'app_cnt_gaming_90d',
       'app_avg_time_bw_installed_mins_30d', 'app_cnt_education_ever',]

dfd = transform_datav2(data, feature_column, a='apps_score', modelDisplayName='apps_score_model_sil', tc='Trench 3', subscription_name = 'Cash November 25 Models') 
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs
0,3271588,30752ee4-93ab-4069-84ac-ccd75499c781,1208f5a5-d481-4cec-84ba-ee1a380a6f1a,0.455138,2025-12-13T11:44:50.659798,2025-12-13T11:44:50.659798,apps_score_model_sil,v2,"{""app_cnt_productivity_ever"": 3.0, ""app_cnt_ra...",Cash November 25 Models,6394f2b0-7870-4d98-a315-1ea675e3d15b,2025-12-13T11:44:50.659798,{},Trench 3,android
1,2976508,37784023-7be0-4692-b191-ba7f968b62c7,55ec45eb-1b71-49cc-8d6c-046d6a3a346d,0.494863,2025-12-13T11:44:50.660800,2025-12-13T11:44:50.660800,apps_score_model_sil,v2,"{""app_cnt_productivity_ever"": 2.0, ""app_cnt_ra...",Cash November 25 Models,83408b82-bdb8-414c-b442-004eaebf28fb,2025-12-13T11:44:50.660800,{},Trench 3,android
2,2275792,b63f06eb-e7b8-4ed0-a1ee-0ec1a1feecde,f502518b-d429-487e-8f76-99594d10172e,0.487225,2025-12-13T11:44:50.660800,2025-12-13T11:44:50.660800,apps_score_model_sil,v2,"{""app_cnt_productivity_ever"": 3.0, ""app_cnt_ra...",Cash November 25 Models,3288f54e-707e-4546-aae7-bb8906a4c656,2025-12-13T11:44:50.660800,{},Trench 3,android
3,3036258,dfad0338-c1b7-47a9-8f92-d23faf4d8cdd,da3ea2c8-b8ef-4370-8da5-2b2eb77a4188,0.406026,2025-12-13T11:44:50.661797,2025-12-13T11:44:50.661797,apps_score_model_sil,v2,"{""app_cnt_productivity_ever"": 3.0, ""app_cnt_ra...",Cash November 25 Models,e4cf0f1a-affe-40e5-a092-ad245c792af7,2025-12-13T11:44:50.661797,{},Trench 3,android
4,2708581,aecc1fe3-bd63-42c1-a901-6a1ccd0f104f,ed614a07-3364-44f3-9307-c04d96901556,0.477627,2025-12-13T11:44:50.661797,2025-12-13T11:44:50.661797,apps_score_model_sil,v2,"{""app_cnt_productivity_ever"": 3.0, ""app_cnt_ra...",Cash November 25 Models,f0c99c1f-621a-4d11-ab88-faa9fdd6ccf0,2025-12-13T11:44:50.661797,{},Trench 3,android


In [139]:
dfd['customerId'] = pd.to_numeric(dfd['customerId'], errors='coerce')

In [140]:
dfd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20246 entries, 0 to 20245
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   customerId            20246 non-null  int64  
 1   digitalLoanAccountId  20246 non-null  object 
 2   crifApplicationId     20246 non-null  object 
 3   prediction            20246 non-null  float64
 4   start_time            20246 non-null  object 
 5   end_time              20246 non-null  object 
 6   modelDisplayName      20246 non-null  object 
 7   modelVersionId        20246 non-null  object 
 8   calcFeature           20246 non-null  object 
 9   subscription_name     20246 non-null  object 
 10  message_id            20246 non-null  object 
 11  publish_time          20246 non-null  object 
 12  attributes            20246 non-null  object 
 13  trenchCategory        20246 non-null  object 
 14  deviceOs              20246 non-null  object 
dtypes: float64(1), int6

In [141]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=1b7bd5c5-07ce-42ea-8421-2fd173908eba>

##### Beta SIL Demo Score

##### Trench 1

In [142]:
sq = """select 
distinct 
loanmaster.customerId customer_id,
r.digitalLoanAccountId,
r.s_demo_score,
ln_vas_opted_flag,
ln_doc_type_rolled,
ln_industry_new_cat_bin,
ln_marital_status,
ln_age,
ln_education_level,
ln_cnt_dependents,
ln_ref2_type,
ln_loan_level_user_type,
ln_ref1_type,
ln_name_email_match_score,
ln_telconame,
ln_city_cat,
ln_brand_bin,
ln_apply_Is_Weekend,
r.ln_os_type osType
from worktable_data_analysis.sil_beta_demo_all_applied_backscored_20240801_20251015 r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
where s_demo_score is not null
and trench_category = 'Trench 1';
"""
data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")

Job ID ac66c0fb-2a22-4ffc-8075-198d4fe330eb successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (345905, 19)


In [143]:
data.columns

Index(['customer_id', 'digitalLoanAccountId', 's_demo_score',
       'ln_vas_opted_flag', 'ln_doc_type_rolled', 'ln_industry_new_cat_bin',
       'ln_marital_status', 'ln_age', 'ln_education_level',
       'ln_cnt_dependents', 'ln_ref2_type', 'ln_loan_level_user_type',
       'ln_ref1_type', 'ln_name_email_match_score', 'ln_telconame',
       'ln_city_cat', 'ln_brand_bin', 'ln_apply_Is_Weekend', 'osType'],
      dtype='object')

In [144]:
feature_column = [ 'ln_vas_opted_flag', 'ln_doc_type_rolled', 'ln_industry_new_cat_bin',
       'ln_marital_status', 'ln_age', 'ln_education_level',
       'ln_cnt_dependents', 'ln_ref2_type', 'ln_loan_level_user_type',
       'ln_ref1_type', 'ln_name_email_match_score', 'ln_telconame',
       'ln_city_cat', 'ln_brand_bin', 'ln_apply_Is_Weekend', 'osType']

dfd = transform_datav2(data, feature_column, a='s_demo_score', modelDisplayName='beta_demo_model_sil', tc='Trench 1', subscription_name = 'Cash November 25 Models') 
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs
0,2964331,71b8b858-29bc-4d91-9b8b-cb4017720ebf,841c4d44-a544-4464-b20c-9fcd8fcfaefe,0.300526,2025-12-13T11:45:05.901565,2025-12-13T11:45:05.901565,beta_demo_model_sil,v2,"{""ln_vas_opted_flag"": ""0"", ""ln_doc_type_rolled...",Cash November 25 Models,259f8486-8309-46d2-b76b-7e244aa17d2d,2025-12-13T11:45:05.901565,{},Trench 1,Android
1,3721936,e9d03a27-05c1-4b04-829d-505126cc518e,416b81b2-c05b-4d87-a28a-d873a5d05926,0.416546,2025-12-13T11:45:05.901565,2025-12-13T11:45:05.901565,beta_demo_model_sil,v2,"{""ln_vas_opted_flag"": ""0"", ""ln_doc_type_rolled...",Cash November 25 Models,b14f9ed7-093d-4552-9d98-6f0b835caf5d,2025-12-13T11:45:05.901565,{},Trench 1,Android
2,3352656,2a8999e5-2596-4920-bc0a-a755d3019ddc,dab60073-16c8-4110-a61e-3264447a14b7,0.475673,2025-12-13T11:45:05.901565,2025-12-13T11:45:05.901565,beta_demo_model_sil,v2,"{""ln_vas_opted_flag"": ""0"", ""ln_doc_type_rolled...",Cash November 25 Models,99320597-d0eb-4b42-9da9-50d8418394e3,2025-12-13T11:45:05.901565,{},Trench 1,Android
3,3044110,e8cfa383-b099-4fdb-a0ae-7af59e770cb7,bb0c0877-dc36-496f-b0ab-0103e9a048cd,0.468105,2025-12-13T11:45:05.901565,2025-12-13T11:45:05.901565,beta_demo_model_sil,v2,"{""ln_vas_opted_flag"": ""0"", ""ln_doc_type_rolled...",Cash November 25 Models,72e857ef-35ff-490e-a48c-3aafad17d59f,2025-12-13T11:45:05.901565,{},Trench 1,Android
4,3306365,9086e1f4-b747-4d3a-a871-a13dc7506719,6c1b9dd8-18c6-4238-bcaa-25f4153ebcd5,0.541279,2025-12-13T11:45:05.902566,2025-12-13T11:45:05.902566,beta_demo_model_sil,v2,"{""ln_vas_opted_flag"": ""0"", ""ln_doc_type_rolled...",Cash November 25 Models,50b3e555-98dc-40ef-b664-a58324f328df,2025-12-13T11:45:05.902566,{},Trench 1,Android


In [145]:
dfd['customerId'] = pd.to_numeric(dfd['customerId'], errors='coerce')

In [146]:
dfd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 345905 entries, 0 to 345904
Data columns (total 15 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   customerId            345905 non-null  int64  
 1   digitalLoanAccountId  345905 non-null  object 
 2   crifApplicationId     345905 non-null  object 
 3   prediction            345905 non-null  float64
 4   start_time            345905 non-null  object 
 5   end_time              345905 non-null  object 
 6   modelDisplayName      345905 non-null  object 
 7   modelVersionId        345905 non-null  object 
 8   calcFeature           345905 non-null  object 
 9   subscription_name     345905 non-null  object 
 10  message_id            345905 non-null  object 
 11  publish_time          345905 non-null  object 
 12  attributes            345905 non-null  object 
 13  trenchCategory        345905 non-null  object 
 14  deviceOs              345905 non-null  object 
dtype

In [147]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=e0155098-b685-4801-92ec-e49c130f2925>

##### Trench 2

In [148]:
sq = """select 
distinct 
loanmaster.customerId customer_id,
r.digitalLoanAccountId,
r.s_demo_score,
ln_vas_opted_flag,
ln_doc_type_rolled,
ln_industry_new_cat_bin,
ln_marital_status,
ln_age,
ln_education_level,
ln_cnt_dependents,
ln_ref2_type,
ln_loan_level_user_type,
ln_ref1_type,
ln_name_email_match_score,
ln_telconame,
ln_city_cat,
ln_brand_bin,
ln_apply_Is_Weekend,
r.ln_os_type osType
from worktable_data_analysis.sil_beta_demo_all_applied_backscored_20240801_20251015 r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
where s_demo_score is not null
and trench_category = 'Trench 2';
"""
data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")

Job ID 415953d1-1c02-444c-ada7-42b8bae823e7 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (13192, 19)


In [149]:
data.columns

Index(['customer_id', 'digitalLoanAccountId', 's_demo_score',
       'ln_vas_opted_flag', 'ln_doc_type_rolled', 'ln_industry_new_cat_bin',
       'ln_marital_status', 'ln_age', 'ln_education_level',
       'ln_cnt_dependents', 'ln_ref2_type', 'ln_loan_level_user_type',
       'ln_ref1_type', 'ln_name_email_match_score', 'ln_telconame',
       'ln_city_cat', 'ln_brand_bin', 'ln_apply_Is_Weekend', 'osType'],
      dtype='object')

In [150]:
feature_column = [ 'ln_vas_opted_flag', 'ln_doc_type_rolled', 'ln_industry_new_cat_bin',
       'ln_marital_status', 'ln_age', 'ln_education_level',
       'ln_cnt_dependents', 'ln_ref2_type', 'ln_loan_level_user_type',
       'ln_ref1_type', 'ln_name_email_match_score', 'ln_telconame',
       'ln_city_cat', 'ln_brand_bin', 'ln_apply_Is_Weekend', 'osType']

dfd = transform_datav2(data, feature_column, a='s_demo_score', modelDisplayName='beta_demo_model_sil', tc='Trench 2', subscription_name = 'Cash November 25 Models') 
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs
0,2948381,5a7fd43c-4b1c-437e-958d-4ed37fd62cca,3834ab9b-23d2-4b42-8868-4176b64bb3de,0.533307,2025-12-13T11:46:15.021594,2025-12-13T11:46:15.021594,beta_demo_model_sil,v2,"{""ln_vas_opted_flag"": ""1"", ""ln_doc_type_rolled...",Cash November 25 Models,ba43a648-6baa-4144-8bd0-34af71229547,2025-12-13T11:46:15.021594,{},Trench 2,Android
1,2459467,9f794aa4-0569-475b-916e-fa98d58e8ed3,163a4611-5073-4180-baa0-d1a131a622bc,0.398044,2025-12-13T11:46:15.021594,2025-12-13T11:46:15.021594,beta_demo_model_sil,v2,"{""ln_vas_opted_flag"": ""1"", ""ln_doc_type_rolled...",Cash November 25 Models,62655ea1-305d-4071-8edc-fbe3ed519f3a,2025-12-13T11:46:15.021594,{},Trench 2,Android
2,2327873,9e2d8dd6-32da-4e1b-81df-719f281747cd,9fc661aa-9643-445f-81a4-a2457df5b4b1,0.388540,2025-12-13T11:46:15.022604,2025-12-13T11:46:15.022604,beta_demo_model_sil,v2,"{""ln_vas_opted_flag"": ""1"", ""ln_doc_type_rolled...",Cash November 25 Models,92255b5e-d54c-4b60-bafb-e5c41f867978,2025-12-13T11:46:15.022604,{},Trench 2,Android
3,2327873,c0384c96-7e52-4403-8885-06318137a5f4,975b7371-7268-4fb5-8360-eff3e579a90d,0.315774,2025-12-13T11:46:15.022604,2025-12-13T11:46:15.022604,beta_demo_model_sil,v2,"{""ln_vas_opted_flag"": ""1"", ""ln_doc_type_rolled...",Cash November 25 Models,61622735-5c7c-4714-b39d-ae35bf43967e,2025-12-13T11:46:15.022604,{},Trench 2,Android
4,3581826,1316a7ed-b056-4d30-b1b1-e5c1c86572f7,2b905e62-91f4-4cfb-99fa-f385fe7df3db,0.356379,2025-12-13T11:46:15.022604,2025-12-13T11:46:15.022604,beta_demo_model_sil,v2,"{""ln_vas_opted_flag"": ""1"", ""ln_doc_type_rolled...",Cash November 25 Models,119a396a-7921-4d57-8e14-cbd040656946,2025-12-13T11:46:15.022604,{},Trench 2,Android


In [151]:
dfd['customerId'] = pd.to_numeric(dfd['customerId'], errors='coerce')

In [152]:
dfd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13192 entries, 0 to 13191
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   customerId            13192 non-null  int64  
 1   digitalLoanAccountId  13192 non-null  object 
 2   crifApplicationId     13192 non-null  object 
 3   prediction            13192 non-null  float64
 4   start_time            13192 non-null  object 
 5   end_time              13192 non-null  object 
 6   modelDisplayName      13192 non-null  object 
 7   modelVersionId        13192 non-null  object 
 8   calcFeature           13192 non-null  object 
 9   subscription_name     13192 non-null  object 
 10  message_id            13192 non-null  object 
 11  publish_time          13192 non-null  object 
 12  attributes            13192 non-null  object 
 13  trenchCategory        13192 non-null  object 
 14  deviceOs              13192 non-null  object 
dtypes: float64(1), int6

In [153]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=f32153da-206a-41b1-82c6-701de3bc9036>

##### Trench 3

In [154]:
sq = """select 
distinct 
loanmaster.customerId customer_id,
r.digitalLoanAccountId,
r.s_demo_score,
ln_vas_opted_flag,
ln_doc_type_rolled,
ln_industry_new_cat_bin,
ln_marital_status,
ln_age,
ln_education_level,
ln_cnt_dependents,
ln_ref2_type,
ln_loan_level_user_type,
ln_ref1_type,
ln_name_email_match_score,
ln_telconame,
ln_city_cat,
ln_brand_bin,
ln_apply_Is_Weekend,
r.ln_os_type osType
from worktable_data_analysis.sil_beta_demo_all_applied_backscored_20240801_20251015 r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
where s_demo_score is not null
and trench_category = 'Trench 3';
"""
data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")

Job ID 8e644bf6-1a54-4624-9077-82e0b07e7b3e successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (13866, 19)


In [155]:
data.columns

Index(['customer_id', 'digitalLoanAccountId', 's_demo_score',
       'ln_vas_opted_flag', 'ln_doc_type_rolled', 'ln_industry_new_cat_bin',
       'ln_marital_status', 'ln_age', 'ln_education_level',
       'ln_cnt_dependents', 'ln_ref2_type', 'ln_loan_level_user_type',
       'ln_ref1_type', 'ln_name_email_match_score', 'ln_telconame',
       'ln_city_cat', 'ln_brand_bin', 'ln_apply_Is_Weekend', 'osType'],
      dtype='object')

In [156]:
feature_column = [ 'ln_vas_opted_flag', 'ln_doc_type_rolled', 'ln_industry_new_cat_bin',
       'ln_marital_status', 'ln_age', 'ln_education_level',
       'ln_cnt_dependents', 'ln_ref2_type', 'ln_loan_level_user_type',
       'ln_ref1_type', 'ln_name_email_match_score', 'ln_telconame',
       'ln_city_cat', 'ln_brand_bin', 'ln_apply_Is_Weekend', 'osType']

dfd = transform_datav2(data, feature_column, a='s_demo_score', modelDisplayName='beta_demo_model_sil', tc='Trench 3', subscription_name = 'Cash November 25 Models') 
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs
0,3135276,370b66cf-88f9-4303-92da-8ea30459f842,a70b1ea2-8f80-43ca-927f-3512daa474dc,0.245603,2025-12-13T11:46:23.334136,2025-12-13T11:46:23.334136,beta_demo_model_sil,v2,"{""ln_vas_opted_flag"": ""0"", ""ln_doc_type_rolled...",Cash November 25 Models,f47c7754-a816-48ec-8d38-38733ba61a1c,2025-12-13T11:46:23.334136,{},Trench 3,Android
1,2555439,3f2b61d1-d70d-418d-819d-7d90865f8c54,36df94dc-53ca-4398-8771-1304d5924d2f,0.109076,2025-12-13T11:46:23.334136,2025-12-13T11:46:23.334136,beta_demo_model_sil,v2,"{""ln_vas_opted_flag"": ""0"", ""ln_doc_type_rolled...",Cash November 25 Models,e3c9835b-65e9-46c8-8911-b942e97bc2d8,2025-12-13T11:46:23.334136,{},Trench 3,Android
2,2543709,5d3e7150-6eb0-4a3a-83fb-ec183ea90066,0e818708-ed7c-4a19-8641-b0e0d3719f81,0.361493,2025-12-13T11:46:23.334136,2025-12-13T11:46:23.334136,beta_demo_model_sil,v2,"{""ln_vas_opted_flag"": ""1"", ""ln_doc_type_rolled...",Cash November 25 Models,81af72c1-f2da-49b7-870d-61eb3e37c34f,2025-12-13T11:46:23.334136,{},Trench 3,iOS
3,3050538,b398c779-0cd1-4aeb-9ea7-5dfd7a5a49d1,17f3e01a-76b8-47d8-a5f6-254e54f40e22,0.206584,2025-12-13T11:46:23.334136,2025-12-13T11:46:23.334136,beta_demo_model_sil,v2,"{""ln_vas_opted_flag"": ""1"", ""ln_doc_type_rolled...",Cash November 25 Models,13c9a4ea-a2f5-44af-8752-b08e1d705660,2025-12-13T11:46:23.334136,{},Trench 3,Android
4,2459261,4ad24418-0d7c-4a38-b309-2ffbbff21daf,0757dfae-f8ab-4443-ab32-40992491bdca,0.129502,2025-12-13T11:46:23.335133,2025-12-13T11:46:23.335133,beta_demo_model_sil,v2,"{""ln_vas_opted_flag"": ""0"", ""ln_doc_type_rolled...",Cash November 25 Models,a4b92411-d8aa-46b2-8532-52eac71d493e,2025-12-13T11:46:23.335133,{},Trench 3,Android


In [157]:
dfd['customerId'] = pd.to_numeric(dfd['customerId'], errors='coerce')

In [158]:
dfd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13866 entries, 0 to 13865
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   customerId            13866 non-null  int64  
 1   digitalLoanAccountId  13866 non-null  object 
 2   crifApplicationId     13866 non-null  object 
 3   prediction            13866 non-null  float64
 4   start_time            13866 non-null  object 
 5   end_time              13866 non-null  object 
 6   modelDisplayName      13866 non-null  object 
 7   modelVersionId        13866 non-null  object 
 8   calcFeature           13866 non-null  object 
 9   subscription_name     13866 non-null  object 
 10  message_id            13866 non-null  object 
 11  publish_time          13866 non-null  object 
 12  attributes            13866 non-null  object 
 13  trenchCategory        13866 non-null  object 
 14  deviceOs              13866 non-null  object 
dtypes: float64(1), int6

In [159]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=07bbd322-1962-45a5-9e16-c0a04043a5b8>

# Delete digitalLoanAccountId from version 1 of sil models as the terms and condition date was very resent.

In [ ]:
sq = """delete from prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details where digitalLoanAccountId = '6b2d3c9c-efd4-4d62-a4d3-db28826ffd7a'
and modelVersionId = 'v1';"""

job = client.query(sq)
job.result()

# End